In [ ]:
## This file implements neural networks before and after lasso selection for p0006presabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0006presabsSTCC_qual.csv')
df.shape

(253, 98)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_10140,group_10143,group_3985,group_4427,group_7172,group_8210,group_9586,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1


In [6]:
df['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 97)

In [9]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_10140,group_10143,group_3985,group_4427,group_7172,group_8210,group_9586,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 97) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,NRS383,1
1,NRS254,1
2,NRS218,1
3,NRS215,0
4,BCH-SA-14,2
...,...,...
112,NRS227,1
113,NRS272,1
114,CFBRSa24,0
115,CFBRSa74,0


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 477us/step - loss: 3.9735 - accuracy: 0.3556 - val_loss: 3.3483 - val_accuracy: 0.2821
Epoch 2/1000
270/270 [==============================] - 0s 92us/step - loss: 2.8293 - accuracy: 0.3407 - val_loss: 2.7728 - val_accuracy: 0.3077
Epoch 3/1000
270/270 [==============================] - 0s 116us/step - loss: 2.2618 - accuracy: 0.4074 - val_loss: 2.3767 - val_accuracy: 0.3675
Epoch 4/1000
270/270 [==============================] - 0s 117us/step - loss: 1.8130 - accuracy: 0.4407 - val_loss: 2.0140 - val_accuracy: 0.3248
Epoch 5/1000
270/270 [==============================] - 0s 136us/step - loss: 1.5554 - accuracy: 0.4370 - val_loss: 1.8760 - val_accuracy: 0.4103
Epoch 6/1000
270/270 [==============================] - 0s 91us/step - loss: 1.6423 - accuracy: 0.5111 - val_loss: 1.8048 - val_accuracy: 0.4274
Epoch 7/1000
270/270 [==============================] - 0s 62us/step - loss: 1.5

Epoch 57/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7291 - accuracy: 0.7407 - val_loss: 0.9021 - val_accuracy: 0.6154
Epoch 58/1000
270/270 [==============================] - 0s 51us/step - loss: 0.6704 - accuracy: 0.7444 - val_loss: 0.9294 - val_accuracy: 0.5897
Epoch 59/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8402 - accuracy: 0.7074 - val_loss: 0.9178 - val_accuracy: 0.5812
Epoch 60/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7663 - accuracy: 0.7222 - val_loss: 0.9215 - val_accuracy: 0.5983
Epoch 61/1000
270/270 [==============================] - 0s 57us/step - loss: 0.7553 - accuracy: 0.7481 - val_loss: 0.9277 - val_accuracy: 0.6154
Epoch 62/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7098 - accuracy: 0.7370 - val_loss: 0.9142 - val_accuracy: 0.5897
Epoch 63/1000
270/270 [==============================] - 0s 64us/step - loss: 0.9170 - accuracy: 0.7185 - val_loss: 1.0288 -

Epoch 113/1000
270/270 [==============================] - 0s 81us/step - loss: 0.5875 - accuracy: 0.7778 - val_loss: 0.8933 - val_accuracy: 0.6581
Epoch 114/1000
270/270 [==============================] - 0s 107us/step - loss: 0.5731 - accuracy: 0.7741 - val_loss: 0.9183 - val_accuracy: 0.6239
Epoch 115/1000
270/270 [==============================] - 0s 121us/step - loss: 0.5583 - accuracy: 0.7852 - val_loss: 0.8752 - val_accuracy: 0.6752
Epoch 116/1000
270/270 [==============================] - 0s 103us/step - loss: 0.5573 - accuracy: 0.7926 - val_loss: 0.9028 - val_accuracy: 0.6667
Epoch 117/1000
270/270 [==============================] - 0s 115us/step - loss: 0.5532 - accuracy: 0.7852 - val_loss: 0.8769 - val_accuracy: 0.6752
Epoch 118/1000
270/270 [==============================] - 0s 123us/step - loss: 0.5552 - accuracy: 0.7963 - val_loss: 0.8854 - val_accuracy: 0.6410
Epoch 119/1000
270/270 [==============================] - 0s 104us/step - loss: 0.5460 - accuracy: 0.7852 - val_l

Epoch 224/1000
270/270 [==============================] - 0s 73us/step - loss: 0.5673 - accuracy: 0.8148 - val_loss: 0.9446 - val_accuracy: 0.6752
Epoch 225/1000
270/270 [==============================] - 0s 69us/step - loss: 0.5296 - accuracy: 0.8111 - val_loss: 0.9231 - val_accuracy: 0.7094
Epoch 226/1000
270/270 [==============================] - 0s 85us/step - loss: 0.4544 - accuracy: 0.8222 - val_loss: 0.9358 - val_accuracy: 0.6581
Epoch 227/1000
270/270 [==============================] - 0s 102us/step - loss: 0.4409 - accuracy: 0.8370 - val_loss: 0.9241 - val_accuracy: 0.6667
Epoch 228/1000
270/270 [==============================] - 0s 106us/step - loss: 0.4799 - accuracy: 0.8185 - val_loss: 0.9538 - val_accuracy: 0.6239
Epoch 229/1000
270/270 [==============================] - 0s 79us/step - loss: 0.4315 - accuracy: 0.8444 - val_loss: 0.9081 - val_accuracy: 0.6923
Epoch 230/1000
270/270 [==============================] - 0s 72us/step - loss: 0.5014 - accuracy: 0.8296 - val_loss:

Epoch 335/1000
270/270 [==============================] - 0s 79us/step - loss: 0.3547 - accuracy: 0.8556 - val_loss: 0.9909 - val_accuracy: 0.6581
Epoch 336/1000
270/270 [==============================] - 0s 71us/step - loss: 0.3509 - accuracy: 0.8593 - val_loss: 0.9959 - val_accuracy: 0.6496
Epoch 337/1000
270/270 [==============================] - 0s 55us/step - loss: 0.3783 - accuracy: 0.8222 - val_loss: 0.9814 - val_accuracy: 0.6838
Epoch 338/1000
270/270 [==============================] - 0s 67us/step - loss: 0.4071 - accuracy: 0.8556 - val_loss: 0.9837 - val_accuracy: 0.6838
Epoch 339/1000
270/270 [==============================] - 0s 84us/step - loss: 0.3646 - accuracy: 0.8630 - val_loss: 1.0613 - val_accuracy: 0.6410
Epoch 340/1000
270/270 [==============================] - 0s 103us/step - loss: 0.3659 - accuracy: 0.8593 - val_loss: 0.9817 - val_accuracy: 0.6838
Epoch 341/1000
270/270 [==============================] - 0s 89us/step - loss: 0.4510 - accuracy: 0.8444 - val_loss: 

...

Epoch 559/1000
270/270 [==============================] - 0s 86us/step - loss: 0.3348 - accuracy: 0.8815 - val_loss: 1.0701 - val_accuracy: 0.6667
Epoch 560/1000
270/270 [==============================] - 0s 87us/step - loss: 0.3361 - accuracy: 0.8778 - val_loss: 1.1162 - val_accuracy: 0.6410
Epoch 561/1000
270/270 [==============================] - 0s 76us/step - loss: 0.3377 - accuracy: 0.8630 - val_loss: 1.0733 - val_accuracy: 0.6752
Epoch 562/1000
270/270 [==============================] - 0s 71us/step - loss: 0.3328 - accuracy: 0.8704 - val_loss: 1.0714 - val_accuracy: 0.6581
Epoch 563/1000
270/270 [==============================] - 0s 74us/step - loss: 0.3038 - accuracy: 0.8778 - val_loss: 1.1189 - val_accuracy: 0.6496
Epoch 564/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2980 - accuracy: 0.8815 - val_loss: 1.0832 - val_accuracy: 0.6752
Epoch 565/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3042 - accuracy: 0.8704 - val_loss: 1

Epoch 615/1000
270/270 [==============================] - 0s 49us/step - loss: 0.4116 - accuracy: 0.8556 - val_loss: 1.1061 - val_accuracy: 0.6496
Epoch 616/1000
270/270 [==============================] - 0s 58us/step - loss: 0.3106 - accuracy: 0.8667 - val_loss: 1.1609 - val_accuracy: 0.5983
Epoch 617/1000
270/270 [==============================] - 0s 52us/step - loss: 0.2907 - accuracy: 0.8815 - val_loss: 1.1324 - val_accuracy: 0.6325
Epoch 618/1000
270/270 [==============================] - 0s 52us/step - loss: 0.2985 - accuracy: 0.8741 - val_loss: 1.1543 - val_accuracy: 0.6325
Epoch 619/1000
270/270 [==============================] - 0s 53us/step - loss: 0.2830 - accuracy: 0.8815 - val_loss: 1.1165 - val_accuracy: 0.6581
Epoch 620/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2966 - accuracy: 0.8852 - val_loss: 1.1073 - val_accuracy: 0.6581
Epoch 621/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2797 - accuracy: 0.8852 - val_loss: 1

Epoch 671/1000
270/270 [==============================] - 0s 51us/step - loss: 0.2757 - accuracy: 0.8852 - val_loss: 1.1314 - val_accuracy: 0.6496
Epoch 672/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2861 - accuracy: 0.8852 - val_loss: 1.1576 - val_accuracy: 0.6239
Epoch 673/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2734 - accuracy: 0.8889 - val_loss: 1.1633 - val_accuracy: 0.6154
Epoch 674/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2749 - accuracy: 0.8815 - val_loss: 1.1427 - val_accuracy: 0.6496
Epoch 675/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2731 - accuracy: 0.8852 - val_loss: 1.1283 - val_accuracy: 0.6496
Epoch 676/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2712 - accuracy: 0.8852 - val_loss: 1.1416 - val_accuracy: 0.6239
Epoch 677/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2670 - accuracy: 0.8889 - val_loss: 1

Epoch 782/1000
270/270 [==============================] - 0s 53us/step - loss: 0.2563 - accuracy: 0.9000 - val_loss: 1.1817 - val_accuracy: 0.6325
Epoch 783/1000
270/270 [==============================] - 0s 44us/step - loss: 0.2652 - accuracy: 0.8852 - val_loss: 1.1549 - val_accuracy: 0.6410
Epoch 784/1000
270/270 [==============================] - 0s 44us/step - loss: 0.2606 - accuracy: 0.8926 - val_loss: 1.1801 - val_accuracy: 0.6325
Epoch 785/1000
270/270 [==============================] - ETA: 0s - loss: 0.2234 - accuracy: 0.90 - 0s 41us/step - loss: 0.2517 - accuracy: 0.9000 - val_loss: 1.1673 - val_accuracy: 0.6325
Epoch 786/1000
270/270 [==============================] - 0s 50us/step - loss: 0.2574 - accuracy: 0.8926 - val_loss: 1.1608 - val_accuracy: 0.6239
Epoch 787/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2511 - accuracy: 0.8963 - val_loss: 1.1896 - val_accuracy: 0.6410
Epoch 788/1000
270/270 [==============================] - 0s 46us/step - los

Epoch 893/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2397 - accuracy: 0.9074 - val_loss: 1.2369 - val_accuracy: 0.6410
Epoch 894/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2414 - accuracy: 0.9037 - val_loss: 1.2267 - val_accuracy: 0.6581
Epoch 895/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2392 - accuracy: 0.9037 - val_loss: 1.2622 - val_accuracy: 0.6496
Epoch 896/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2412 - accuracy: 0.9037 - val_loss: 1.2484 - val_accuracy: 0.6410
Epoch 897/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2429 - accuracy: 0.9037 - val_loss: 1.2383 - val_accuracy: 0.6410
Epoch 898/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2412 - accuracy: 0.9037 - val_loss: 1.2365 - val_accuracy: 0.6325
Epoch 899/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2395 - accuracy: 0.9111 - val_loss: 1

In [37]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 62us/step
over-sampling test accuracy: 69.23%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 1, 1, 0, 1, 1, 0, 2, 2, 2, 0, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1,
       0, 1, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 2, 0, 2, 0, 1, 0, 2, 0, 0, 1, 0, 2, 2, 1, 1, 2, 0, 0, 2, 2, 2,
       1, 1, 1, 2, 0, 2, 1, 1, 2, 2, 0, 1, 0, 0, 1, 1, 1, 2, 1, 0, 2, 1,
       0, 0, 1, 0, 1, 0, 2, 1, 1, 2, 1, 0, 2, 0, 1, 2, 0, 0, 2, 2, 1, 2,
       0, 1, 2, 1, 1, 0, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS383,1,1
1,NRS254,1,1
2,NRS218,1,1
3,NRS215,0,0
4,BCH-SA-14,2,1
...,...,...,...
112,NRS227,1,2
113,NRS272,1,1
114,CFBRSa24,0,1
115,CFBRSa74,0,0


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.174705,0.810493,1.480301e-02
1,0.149512,0.849576,9.126217e-04
2,0.238834,0.760457,7.096314e-04
3,0.999793,0.000203,4.801617e-06
4,0.000334,0.667405,3.322608e-01
...,...,...,...
112,0.468998,0.054691,4.763110e-01
113,0.011606,0.629533,3.588608e-01
114,0.487851,0.493897,1.825207e-02
115,0.997426,0.002574,4.817531e-08


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p006pST.csv", index = False,
         header=None)

In [41]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 112us/step - loss: 0.2097 - accuracy: 0.9111 - val_loss: 1.2526 - val_accuracy: 0.7009
Epoch 2/1000
270/270 [==============================] - 0s 116us/step - loss: 0.2183 - accuracy: 0.8963 - val_loss: 1.2679 - val_accuracy: 0.6923
Epoch 3/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2053 - accuracy: 0.9037 - val_loss: 1.2702 - val_accuracy: 0.6838
Epoch 4/1000
270/270 [==============================] - 0s 115us/step - loss: 0.2093 - accuracy: 0.9000 - val_loss: 1.2474 - val_accuracy: 0.7094
Epoch 5/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2324 - accuracy: 0.9074 - val_loss: 1.2301 - val_accuracy: 0.6838
Epoch 6/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2999 - accuracy: 0.8852 - val_loss: 1.2511 - val_accuracy: 0.7094
Epoch 7/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2

Epoch 57/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2978 - accuracy: 0.8926 - val_loss: 1.3841 - val_accuracy: 0.7009
Epoch 58/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2388 - accuracy: 0.8963 - val_loss: 1.3657 - val_accuracy: 0.6838
Epoch 59/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2038 - accuracy: 0.8963 - val_loss: 1.2716 - val_accuracy: 0.7094
Epoch 60/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2602 - accuracy: 0.8889 - val_loss: 1.2622 - val_accuracy: 0.7094
Epoch 61/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2167 - accuracy: 0.8926 - val_loss: 1.3414 - val_accuracy: 0.6838
Epoch 62/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2174 - accuracy: 0.8926 - val_loss: 1.2916 - val_accuracy: 0.6923
Epoch 63/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2358 - accuracy: 0.9037 - val_loss: 1.2941 -

Epoch 113/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2215 - accuracy: 0.8889 - val_loss: 1.3292 - val_accuracy: 0.6923
Epoch 114/1000
270/270 [==============================] - 0s 139us/step - loss: 0.2148 - accuracy: 0.9037 - val_loss: 1.3215 - val_accuracy: 0.6923
Epoch 115/1000
270/270 [==============================] - 0s 190us/step - loss: 0.2302 - accuracy: 0.8926 - val_loss: 1.3082 - val_accuracy: 0.7009
Epoch 116/1000
270/270 [==============================] - 0s 125us/step - loss: 0.3138 - accuracy: 0.8926 - val_loss: 1.3397 - val_accuracy: 0.7094
Epoch 117/1000
270/270 [==============================] - 0s 143us/step - loss: 0.4290 - accuracy: 0.8852 - val_loss: 1.3211 - val_accuracy: 0.7094
Epoch 118/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2404 - accuracy: 0.8926 - val_loss: 1.4966 - val_accuracy: 0.6838
Epoch 119/1000
270/270 [==============================] - 0s 126us/step - loss: 0.3019 - accuracy: 0.8815 - val_l

Epoch 224/1000
270/270 [==============================] - 0s 51us/step - loss: 0.3920 - accuracy: 0.8852 - val_loss: 1.3764 - val_accuracy: 0.7094
Epoch 225/1000
270/270 [==============================] - 0s 51us/step - loss: 0.2251 - accuracy: 0.9037 - val_loss: 1.5234 - val_accuracy: 0.6752
Epoch 226/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2640 - accuracy: 0.8926 - val_loss: 1.3552 - val_accuracy: 0.7009
Epoch 227/1000
270/270 [==============================] - 0s 52us/step - loss: 0.2617 - accuracy: 0.9000 - val_loss: 1.3567 - val_accuracy: 0.7009
Epoch 228/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2360 - accuracy: 0.9037 - val_loss: 1.4000 - val_accuracy: 0.7009
Epoch 229/1000
270/270 [==============================] - 0s 46us/step - loss: 0.2066 - accuracy: 0.9037 - val_loss: 1.3950 - val_accuracy: 0.7009
Epoch 230/1000
270/270 [==============================] - 0s 46us/step - loss: 0.2175 - accuracy: 0.9148 - val_loss: 1

270/270 [==============================] - 0s 89us/step - loss: 0.1851 - accuracy: 0.9074 - val_loss: 1.4197 - val_accuracy: 0.7009
Epoch 335/1000
270/270 [==============================] - 0s 75us/step - loss: 0.1870 - accuracy: 0.9074 - val_loss: 1.3926 - val_accuracy: 0.7179
Epoch 336/1000
270/270 [==============================] - 0s 65us/step - loss: 0.1840 - accuracy: 0.9185 - val_loss: 1.4108 - val_accuracy: 0.7179
Epoch 337/1000
270/270 [==============================] - 0s 66us/step - loss: 0.1816 - accuracy: 0.9074 - val_loss: 1.4252 - val_accuracy: 0.7009
Epoch 338/1000
270/270 [==============================] - 0s 77us/step - loss: 0.1793 - accuracy: 0.9148 - val_loss: 1.4053 - val_accuracy: 0.7265
Epoch 339/1000
270/270 [==============================] - 0s 80us/step - loss: 0.1865 - accuracy: 0.9222 - val_loss: 1.4141 - val_accuracy: 0.7265
Epoch 340/1000
270/270 [==============================] - 0s 78us/step - loss: 0.1852 - accuracy: 0.9111 - val_loss: 1.4228 - val_acc

...

Epoch 556/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2221 - accuracy: 0.8963 - val_loss: 1.4775 - val_accuracy: 0.6923
Epoch 557/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2643 - accuracy: 0.9111 - val_loss: 1.4832 - val_accuracy: 0.6838
Epoch 558/1000
270/270 [==============================] - 0s 89us/step - loss: 0.4326 - accuracy: 0.8852 - val_loss: 1.4800 - val_accuracy: 0.7179
Epoch 559/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2382 - accuracy: 0.9111 - val_loss: 1.6147 - val_accuracy: 0.6923
Epoch 560/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2359 - accuracy: 0.9000 - val_loss: 1.5460 - val_accuracy: 0.7009
Epoch 561/1000
270/270 [==============================] - 0s 92us/step - loss: 0.1918 - accuracy: 0.9074 - val_loss: 1.4829 - val_accuracy: 0.7179
Epoch 562/1000
270/270 [==============================] - 0s 83us/step - loss: 0.1753 - accuracy: 0.9148 - val_loss: 

Epoch 667/1000
270/270 [==============================] - 0s 68us/step - loss: 0.1849 - accuracy: 0.9148 - val_loss: 1.5153 - val_accuracy: 0.7179
Epoch 668/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2154 - accuracy: 0.9148 - val_loss: 1.5361 - val_accuracy: 0.7179
Epoch 669/1000
270/270 [==============================] - 0s 435us/step - loss: 0.1665 - accuracy: 0.9296 - val_loss: 1.6135 - val_accuracy: 0.6923
Epoch 670/1000
270/270 [==============================] - 0s 120us/step - loss: 0.1760 - accuracy: 0.9111 - val_loss: 1.5303 - val_accuracy: 0.7179
Epoch 671/1000
270/270 [==============================] - 0s 177us/step - loss: 0.2380 - accuracy: 0.9222 - val_loss: 1.5179 - val_accuracy: 0.7265
Epoch 672/1000
270/270 [==============================] - 0s 161us/step - loss: 0.2006 - accuracy: 0.9222 - val_loss: 1.6119 - val_accuracy: 0.6838
Epoch 673/1000
270/270 [==============================] - 0s 152us/step - loss: 0.2294 - accuracy: 0.9037 - val_lo

270/270 [==============================] - 0s 100us/step - loss: 0.1843 - accuracy: 0.9222 - val_loss: 1.5517 - val_accuracy: 0.7179
Epoch 778/1000
270/270 [==============================] - 0s 82us/step - loss: 0.1725 - accuracy: 0.9259 - val_loss: 1.5947 - val_accuracy: 0.7094
Epoch 779/1000
270/270 [==============================] - 0s 91us/step - loss: 0.1644 - accuracy: 0.9259 - val_loss: 1.5917 - val_accuracy: 0.7094
Epoch 780/1000
270/270 [==============================] - 0s 100us/step - loss: 0.1639 - accuracy: 0.9296 - val_loss: 1.5619 - val_accuracy: 0.7265
Epoch 781/1000
270/270 [==============================] - 0s 73us/step - loss: 0.1621 - accuracy: 0.9333 - val_loss: 1.5725 - val_accuracy: 0.7179
Epoch 782/1000
270/270 [==============================] - 0s 133us/step - loss: 0.1660 - accuracy: 0.9259 - val_loss: 1.5843 - val_accuracy: 0.7179
Epoch 783/1000
270/270 [==============================] - 0s 115us/step - loss: 0.1610 - accuracy: 0.9296 - val_loss: 1.6294 - val

Epoch 888/1000
270/270 [==============================] - 0s 78us/step - loss: 0.1813 - accuracy: 0.9148 - val_loss: 1.6242 - val_accuracy: 0.7094
Epoch 889/1000
270/270 [==============================] - 0s 73us/step - loss: 0.1774 - accuracy: 0.9259 - val_loss: 1.5767 - val_accuracy: 0.7094
Epoch 890/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2021 - accuracy: 0.9222 - val_loss: 1.6020 - val_accuracy: 0.7179
Epoch 891/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2340 - accuracy: 0.9148 - val_loss: 1.7092 - val_accuracy: 0.6838
Epoch 892/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2158 - accuracy: 0.9037 - val_loss: 1.5622 - val_accuracy: 0.7094
Epoch 893/1000
270/270 [==============================] - 0s 171us/step - loss: 0.3604 - accuracy: 0.9074 - val_loss: 1.5887 - val_accuracy: 0.7094
Epoch 894/1000
270/270 [==============================] - 0s 213us/step - loss: 0.1893 - accuracy: 0.9185 - val_loss:

270/270 [==============================] - 0s 78us/step - loss: 0.1568 - accuracy: 0.9333 - val_loss: 1.6557 - val_accuracy: 0.7094
Epoch 999/1000
270/270 [==============================] - 0s 69us/step - loss: 0.1562 - accuracy: 0.9333 - val_loss: 1.6601 - val_accuracy: 0.7094
Epoch 1000/1000
270/270 [==============================] - 0s 117us/step - loss: 0.1613 - accuracy: 0.9222 - val_loss: 1.6365 - val_accuracy: 0.7179


In [42]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 91.32%


In [19]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [20]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [21]:
y_prob = df_proba[df_proba['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[1.74704540e-01, 8.10492500e-01, 1.48030110e-02],
       [1.49511810e-01, 8.49575600e-01, 9.12621730e-04],
       [2.38833700e-01, 7.60456700e-01, 7.09631400e-04],
       [9.99792500e-01, 2.02748280e-04, 4.80161730e-06],
       [3.34387470e-04, 6.67404830e-01, 3.32260760e-01],
       [4.85844650e-02, 5.94312700e-01, 3.57102840e-01],
       [9.82816700e-01, 1.71829090e-02, 3.13184160e-07],
       [1.30378840e-01, 7.42902100e-02, 7.95330900e-01],
       [1.03087080e-03, 3.87456840e-06, 9.98965260e-01],
       [4.65065400e-03, 4.66648800e-02, 9.48684500e-01],
       [8.90647950e-01, 1.09351985e-01, 1.67793610e-09],
       [1.56612220e-04, 1.58889370e-03, 9.98254500e-01],
       [1.15666166e-01, 8.84300950e-01, 3.28678430e-05],
       [4.87851350e-01, 4.93896500e-01, 1.82520710e-02],
       [1.28195380e-01, 8.13084900e-01, 5.87196700e-02],
       [3.34387470e-04, 6.67404830e-01, 3.32260760e-01],
       [4.89163450e-03, 1.82635160e-01, 8.12473240e-01],
       [8.79944000e-05, 2.88255

In [15]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.829607714223099

In [16]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [25]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.829607714223099

In [26]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [27]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [28]:
dat2

,0,test
0,BCH-SA-11,2
1,NRS161,1
2,BCH-SA-14,2
3,BCH-SA-11,2
4,CFBRSa49,1
...,...,...
112,NRS064,2
113,NRS266,2
114,NRS222,0
115,GA27,2


In [29]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [30]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 854us/step - loss: 23.9236 - accuracy: 0.3333 - val_loss: 13.9145 - val_accuracy: 0.3846
Epoch 2/1000
270/270 [==============================] - 0s 184us/step - loss: 17.8150 - accuracy: 0.4185 - val_loss: 9.9569 - val_accuracy: 0.4957
Epoch 3/1000
270/270 [==============================] - 0s 216us/step - loss: 12.1661 - accuracy: 0.4481 - val_loss: 6.3496 - val_accuracy: 0.4957
Epoch 4/1000
270/270 [==============================] - 0s 297us/step - loss: 6.2161 - accuracy: 0.4259 - val_loss: 3.4705 - val_accuracy: 0.4274
Epoch 5/1000
270/270 [==============================] - 0s 222us/step - loss: 3.0819 - accuracy: 0.4111 - val_loss: 3.2913 - val_accuracy: 0.4103
Epoch 6/1000
270/270 [==============================] - 0s 186us/step - loss: 2.4649 - accuracy: 0.4222 - val_loss: 2.2918 - val_accuracy: 0.4103
Epoch 7/1000
270/270 [==============================] - 0s 174us/step - lo

Epoch 112/1000
270/270 [==============================] - 0s 92us/step - loss: 0.5652 - accuracy: 0.7963 - val_loss: 0.9723 - val_accuracy: 0.7521
Epoch 113/1000
270/270 [==============================] - 0s 71us/step - loss: 0.4975 - accuracy: 0.7889 - val_loss: 0.7588 - val_accuracy: 0.7009
Epoch 114/1000
270/270 [==============================] - 0s 69us/step - loss: 0.5320 - accuracy: 0.8037 - val_loss: 1.1713 - val_accuracy: 0.7350
Epoch 115/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7463 - accuracy: 0.7556 - val_loss: 1.0044 - val_accuracy: 0.7436
Epoch 116/1000
270/270 [==============================] - 0s 99us/step - loss: 0.6096 - accuracy: 0.7630 - val_loss: 0.9705 - val_accuracy: 0.6410
Epoch 117/1000
270/270 [==============================] - 0s 73us/step - loss: 0.7331 - accuracy: 0.7148 - val_loss: 0.9275 - val_accuracy: 0.7179
Epoch 118/1000
270/270 [==============================] - 0s 69us/step - loss: 0.5043 - accuracy: 0.7889 - val_loss: 0

Epoch 168/1000
270/270 [==============================] - 0s 97us/step - loss: 0.4533 - accuracy: 0.8370 - val_loss: 0.8649 - val_accuracy: 0.7179
Epoch 169/1000
270/270 [==============================] - 0s 77us/step - loss: 0.4447 - accuracy: 0.8259 - val_loss: 0.7829 - val_accuracy: 0.6838
Epoch 170/1000
270/270 [==============================] - 0s 76us/step - loss: 0.4293 - accuracy: 0.8222 - val_loss: 0.8265 - val_accuracy: 0.7350
Epoch 171/1000
270/270 [==============================] - 0s 82us/step - loss: 0.4266 - accuracy: 0.8370 - val_loss: 0.7783 - val_accuracy: 0.7350
Epoch 172/1000
270/270 [==============================] - 0s 79us/step - loss: 0.4148 - accuracy: 0.8370 - val_loss: 0.7960 - val_accuracy: 0.6838
Epoch 173/1000
270/270 [==============================] - 0s 91us/step - loss: 0.4272 - accuracy: 0.8259 - val_loss: 0.7731 - val_accuracy: 0.7009
Epoch 174/1000
270/270 [==============================] - 0s 82us/step - loss: 0.4146 - accuracy: 0.8333 - val_loss: 1

270/270 [==============================] - 0s 96us/step - loss: 0.3445 - accuracy: 0.8593 - val_loss: 0.8982 - val_accuracy: 0.6923
Epoch 279/1000
270/270 [==============================] - 0s 98us/step - loss: 0.3388 - accuracy: 0.8593 - val_loss: 0.8868 - val_accuracy: 0.6923
Epoch 280/1000
270/270 [==============================] - 0s 87us/step - loss: 0.3431 - accuracy: 0.8741 - val_loss: 0.8959 - val_accuracy: 0.6923
Epoch 281/1000
270/270 [==============================] - 0s 102us/step - loss: 0.3482 - accuracy: 0.8630 - val_loss: 0.8792 - val_accuracy: 0.7350
Epoch 282/1000
270/270 [==============================] - 0s 86us/step - loss: 0.3678 - accuracy: 0.8630 - val_loss: 0.9237 - val_accuracy: 0.6667
Epoch 283/1000
270/270 [==============================] - 0s 94us/step - loss: 0.3700 - accuracy: 0.8407 - val_loss: 0.8762 - val_accuracy: 0.7009
Epoch 284/1000
270/270 [==============================] - 0s 110us/step - loss: 0.3547 - accuracy: 0.8630 - val_loss: 0.8906 - val_a

Epoch 334/1000
270/270 [==============================] - 0s 95us/step - loss: 0.3135 - accuracy: 0.8815 - val_loss: 0.9267 - val_accuracy: 0.6667
Epoch 335/1000
270/270 [==============================] - 0s 98us/step - loss: 0.3143 - accuracy: 0.8889 - val_loss: 0.9323 - val_accuracy: 0.7350
Epoch 336/1000
270/270 [==============================] - 0s 94us/step - loss: 0.3133 - accuracy: 0.8926 - val_loss: 0.9322 - val_accuracy: 0.7094
Epoch 337/1000
270/270 [==============================] - 0s 94us/step - loss: 0.3314 - accuracy: 0.8778 - val_loss: 1.0563 - val_accuracy: 0.7265
Epoch 338/1000
270/270 [==============================] - 0s 88us/step - loss: 0.3375 - accuracy: 0.8667 - val_loss: 0.9482 - val_accuracy: 0.7350
Epoch 339/1000
270/270 [==============================] - 0s 102us/step - loss: 0.3221 - accuracy: 0.8704 - val_loss: 1.0258 - val_accuracy: 0.7265
Epoch 340/1000
270/270 [==============================] - 0s 97us/step - loss: 0.3301 - accuracy: 0.8778 - val_loss: 

...

Epoch 612/1000
270/270 [==============================] - 0s 102us/step - loss: 0.2551 - accuracy: 0.8963 - val_loss: 1.2017 - val_accuracy: 0.7009
Epoch 613/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2517 - accuracy: 0.8926 - val_loss: 1.2729 - val_accuracy: 0.7009
Epoch 614/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2705 - accuracy: 0.8815 - val_loss: 1.1978 - val_accuracy: 0.6923
Epoch 615/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2581 - accuracy: 0.8815 - val_loss: 1.2330 - val_accuracy: 0.7009
Epoch 616/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2743 - accuracy: 0.8815 - val_loss: 1.1975 - val_accuracy: 0.7436
Epoch 617/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2949 - accuracy: 0.8778 - val_loss: 1.3330 - val_accuracy: 0.7009
Epoch 618/1000
270/270 [==============================] - 0s 115us/step - loss: 0.2726 - accuracy: 0.8926 - val_loss

270/270 [==============================] - 0s 89us/step - loss: 0.2418 - accuracy: 0.8926 - val_loss: 1.2618 - val_accuracy: 0.7094
Epoch 723/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2493 - accuracy: 0.8963 - val_loss: 1.2753 - val_accuracy: 0.7094
Epoch 724/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2456 - accuracy: 0.9000 - val_loss: 1.2574 - val_accuracy: 0.7094
Epoch 725/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2422 - accuracy: 0.9000 - val_loss: 1.3064 - val_accuracy: 0.7009
Epoch 726/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2433 - accuracy: 0.9000 - val_loss: 1.2630 - val_accuracy: 0.7094
Epoch 727/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2527 - accuracy: 0.8963 - val_loss: 1.2849 - val_accuracy: 0.7094
Epoch 728/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2521 - accuracy: 0.8852 - val_loss: 1.3516 - val_acc

Epoch 778/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2463 - accuracy: 0.8926 - val_loss: 1.4348 - val_accuracy: 0.6923
Epoch 779/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2854 - accuracy: 0.8815 - val_loss: 1.4321 - val_accuracy: 0.6923
Epoch 780/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2542 - accuracy: 0.8963 - val_loss: 1.3441 - val_accuracy: 0.7009
Epoch 781/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2641 - accuracy: 0.8926 - val_loss: 1.5968 - val_accuracy: 0.6838
Epoch 782/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3127 - accuracy: 0.8741 - val_loss: 1.3615 - val_accuracy: 0.7350
Epoch 783/1000
270/270 [==============================] - 0s 124us/step - loss: 0.2423 - accuracy: 0.8963 - val_loss: 1.3353 - val_accuracy: 0.7009
Epoch 784/1000
270/270 [==============================] - 0s 196us/step - loss: 0.2457 - accuracy: 0.8667 - val_los

Epoch 889/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2321 - accuracy: 0.8963 - val_loss: 1.5190 - val_accuracy: 0.7265
Epoch 890/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2635 - accuracy: 0.8926 - val_loss: 1.4399 - val_accuracy: 0.6923
Epoch 891/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2312 - accuracy: 0.9000 - val_loss: 1.3936 - val_accuracy: 0.6923
Epoch 892/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2346 - accuracy: 0.9000 - val_loss: 1.4017 - val_accuracy: 0.7436
Epoch 893/1000
270/270 [==============================] - 0s 55us/step - loss: 0.2279 - accuracy: 0.9000 - val_loss: 1.3951 - val_accuracy: 0.7350
Epoch 894/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2275 - accuracy: 0.9074 - val_loss: 1.3951 - val_accuracy: 0.7094
Epoch 895/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2256 - accuracy: 0.9000 - val_loss:

270/270 [==============================] - 0s 154us/step - loss: 0.2508 - accuracy: 0.8667 - val_loss: 1.4854 - val_accuracy: 0.7350
Epoch 1000/1000
270/270 [==============================] - 0s 125us/step - loss: 0.2313 - accuracy: 0.8889 - val_loss: 1.4974 - val_accuracy: 0.6923


In [59]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 99us/step
over-sampling test accuracy: 76.92%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 2, 2, 2, 1, 2, 0, 1, 1, 1, 2, 1, 0, 1, 0, 2, 2, 0, 2, 1, 2, 1,
       0, 1, 0, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 1,
       2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 0, 0, 1, 0, 1, 1, 2, 2, 1, 1, 2, 1,
       1, 2, 2, 1, 1, 0, 2, 2, 0, 1, 1, 2, 0, 0, 1, 2, 1, 2, 0, 0, 2, 0,
       1, 0, 2, 1, 1, 0, 2, 0, 2, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 2, 1, 2,
       0, 1, 2, 2, 0, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,BCH-SA-11,2,2
1,NRS161,1,2
2,BCH-SA-14,2,2
3,BCH-SA-11,2,2
4,CFBRSa49,1,1
...,...,...,...
112,NRS064,2,2
113,NRS266,2,2
114,NRS222,0,0
115,GA27,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,2.158608e-03,0.056989,9.408523e-01
1,7.647003e-06,0.000100,9.998921e-01
2,4.092795e-06,0.043807,9.561888e-01
3,2.158608e-03,0.056989,9.408523e-01
4,5.226637e-03,0.994773,5.888736e-12
...,...,...,...
112,1.162997e-07,0.000050,9.999496e-01
113,5.491051e-02,0.006768,9.383212e-01
114,9.997597e-01,0.000114,1.264912e-04
115,2.730268e-04,0.006485,9.932420e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p006pST.csv", index = False,
         header=None)

In [63]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2227 - accuracy: 0.9000 - val_loss: 1.2491 - val_accuracy: 0.7692
Epoch 2/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2213 - accuracy: 0.8963 - val_loss: 1.2332 - val_accuracy: 0.7692
Epoch 3/1000
270/270 [==============================] - 0s 131us/step - loss: 0.2170 - accuracy: 0.8963 - val_loss: 1.2479 - val_accuracy: 0.7692
Epoch 4/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2249 - accuracy: 0.8963 - val_loss: 1.2482 - val_accuracy: 0.7692
Epoch 5/1000
270/270 [==============================] - 0s 129us/step - loss: 0.2242 - accuracy: 0.9000 - val_loss: 1.2543 - val_accuracy: 0.7350
Epoch 6/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2262 - accuracy: 0.9000 - val_loss: 1.2713 - val_accuracy: 0.7350
Epoch 7/1000
270/270 [==============================] - 0s 130us/step - loss: 

270/270 [==============================] - 0s 159us/step - loss: 0.2424 - accuracy: 0.8926 - val_loss: 1.4158 - val_accuracy: 0.7521
Epoch 112/1000
270/270 [==============================] - 0s 373us/step - loss: 0.2183 - accuracy: 0.8815 - val_loss: 1.3845 - val_accuracy: 0.7179
Epoch 113/1000
270/270 [==============================] - 0s 168us/step - loss: 0.3102 - accuracy: 0.8630 - val_loss: 1.6949 - val_accuracy: 0.7521
Epoch 114/1000
270/270 [==============================] - 0s 282us/step - loss: 0.2696 - accuracy: 0.8852 - val_loss: 1.4614 - val_accuracy: 0.7350
Epoch 115/1000
270/270 [==============================] - 0s 265us/step - loss: 0.2534 - accuracy: 0.8815 - val_loss: 1.4369 - val_accuracy: 0.7521
Epoch 116/1000
270/270 [==============================] - 0s 477us/step - loss: 0.2256 - accuracy: 0.8926 - val_loss: 1.3613 - val_accuracy: 0.7350
Epoch 117/1000
270/270 [==============================] - 0s 153us/step - loss: 0.2269 - accuracy: 0.9000 - val_loss: 1.3448 - 

270/270 [==============================] - 0s 132us/step - loss: 0.2121 - accuracy: 0.9000 - val_loss: 1.4120 - val_accuracy: 0.7607
Epoch 222/1000
270/270 [==============================] - 0s 142us/step - loss: 0.2127 - accuracy: 0.9000 - val_loss: 1.4124 - val_accuracy: 0.7607
Epoch 223/1000
270/270 [==============================] - 0s 159us/step - loss: 0.2235 - accuracy: 0.9000 - val_loss: 1.4042 - val_accuracy: 0.7350
Epoch 224/1000
270/270 [==============================] - 0s 155us/step - loss: 0.2154 - accuracy: 0.8963 - val_loss: 1.4127 - val_accuracy: 0.7692
Epoch 225/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2204 - accuracy: 0.9037 - val_loss: 1.5871 - val_accuracy: 0.7521
Epoch 226/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2476 - accuracy: 0.9000 - val_loss: 1.4425 - val_accuracy: 0.7692
Epoch 227/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2121 - accuracy: 0.9000 - val_loss: 1.4255 - 

270/270 [==============================] - 0s 92us/step - loss: 0.2161 - accuracy: 0.8963 - val_loss: 1.4451 - val_accuracy: 0.7265
Epoch 332/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2107 - accuracy: 0.9000 - val_loss: 1.4520 - val_accuracy: 0.7607
Epoch 333/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2099 - accuracy: 0.8741 - val_loss: 1.4637 - val_accuracy: 0.7094
Epoch 334/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2081 - accuracy: 0.9037 - val_loss: 1.4801 - val_accuracy: 0.7692
Epoch 335/1000
270/270 [==============================] - 0s 131us/step - loss: 0.2099 - accuracy: 0.9037 - val_loss: 1.7129 - val_accuracy: 0.7265
Epoch 336/1000
270/270 [==============================] - 0s 196us/step - loss: 0.3552 - accuracy: 0.8741 - val_loss: 1.6696 - val_accuracy: 0.7521
Epoch 337/1000
270/270 [==============================] - 0s 146us/step - loss: 0.2372 - accuracy: 0.8852 - val_loss: 1.5307 - val

270/270 [==============================] - 0s 111us/step - loss: 0.1989 - accuracy: 0.9000 - val_loss: 1.5386 - val_accuracy: 0.7692
Epoch 442/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2215 - accuracy: 0.8741 - val_loss: 1.5474 - val_accuracy: 0.7607
Epoch 443/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2182 - accuracy: 0.8926 - val_loss: 1.7326 - val_accuracy: 0.7350
Epoch 444/1000
270/270 [==============================] - 0s 118us/step - loss: 0.3905 - accuracy: 0.8778 - val_loss: 1.5830 - val_accuracy: 0.7607
Epoch 445/1000
270/270 [==============================] - 0s 123us/step - loss: 0.2245 - accuracy: 0.8926 - val_loss: 1.5696 - val_accuracy: 0.7607
Epoch 446/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2077 - accuracy: 0.9000 - val_loss: 1.5404 - val_accuracy: 0.7350
Epoch 447/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2087 - accuracy: 0.8778 - val_loss: 1.5260 - va

270/270 [==============================] - 0s 96us/step - loss: 0.2085 - accuracy: 0.9000 - val_loss: 1.5855 - val_accuracy: 0.7692
Epoch 552/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2013 - accuracy: 0.9000 - val_loss: 1.5675 - val_accuracy: 0.7692
Epoch 553/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2011 - accuracy: 0.8963 - val_loss: 1.5709 - val_accuracy: 0.7607
Epoch 554/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2015 - accuracy: 0.9000 - val_loss: 1.5806 - val_accuracy: 0.7436
Epoch 555/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2030 - accuracy: 0.8963 - val_loss: 1.5954 - val_accuracy: 0.7692
Epoch 556/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2005 - accuracy: 0.8926 - val_loss: 1.5865 - val_accuracy: 0.7436
Epoch 557/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2000 - accuracy: 0.9000 - val_loss: 1.5876 - va

270/270 [==============================] - 0s 99us/step - loss: 0.2057 - accuracy: 0.9074 - val_loss: 1.7116 - val_accuracy: 0.7265
Epoch 662/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2168 - accuracy: 0.9000 - val_loss: 1.6740 - val_accuracy: 0.7778
Epoch 663/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2053 - accuracy: 0.9000 - val_loss: 1.7057 - val_accuracy: 0.7350
Epoch 664/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2440 - accuracy: 0.8963 - val_loss: 1.6834 - val_accuracy: 0.7607
Epoch 665/1000
270/270 [==============================] - 0s 144us/step - loss: 0.2032 - accuracy: 0.9000 - val_loss: 1.7298 - val_accuracy: 0.7350
Epoch 666/1000
270/270 [==============================] - 0s 123us/step - loss: 0.2066 - accuracy: 0.8963 - val_loss: 1.6638 - val_accuracy: 0.7350
Epoch 667/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2204 - accuracy: 0.9037 - val_loss: 1.6361 - va

270/270 [==============================] - 0s 129us/step - loss: 0.1955 - accuracy: 0.9000 - val_loss: 1.6971 - val_accuracy: 0.7350
Epoch 772/1000
270/270 [==============================] - 0s 126us/step - loss: 0.1965 - accuracy: 0.9074 - val_loss: 1.7016 - val_accuracy: 0.7692
Epoch 773/1000
270/270 [==============================] - 0s 129us/step - loss: 0.1962 - accuracy: 0.8926 - val_loss: 1.7217 - val_accuracy: 0.7692
Epoch 774/1000
270/270 [==============================] - 0s 124us/step - loss: 0.2033 - accuracy: 0.9000 - val_loss: 1.7207 - val_accuracy: 0.7692
Epoch 775/1000
270/270 [==============================] - 0s 126us/step - loss: 0.1994 - accuracy: 0.8963 - val_loss: 1.6824 - val_accuracy: 0.7350
Epoch 776/1000
270/270 [==============================] - 0s 108us/step - loss: 0.1942 - accuracy: 0.9037 - val_loss: 1.6931 - val_accuracy: 0.7692
Epoch 777/1000
270/270 [==============================] - 0s 113us/step - loss: 0.1949 - accuracy: 0.8963 - val_loss: 1.6879 - 

270/270 [==============================] - 0s 91us/step - loss: 0.2133 - accuracy: 0.8926 - val_loss: 1.7534 - val_accuracy: 0.7607
Epoch 882/1000
270/270 [==============================] - 0s 90us/step - loss: 0.1997 - accuracy: 0.8963 - val_loss: 1.8164 - val_accuracy: 0.7265
Epoch 883/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2164 - accuracy: 0.8704 - val_loss: 1.7949 - val_accuracy: 0.7778
Epoch 884/1000
270/270 [==============================] - 0s 84us/step - loss: 0.2016 - accuracy: 0.9000 - val_loss: 1.7961 - val_accuracy: 0.7863
Epoch 885/1000
270/270 [==============================] - 0s 87us/step - loss: 0.1971 - accuracy: 0.9000 - val_loss: 1.7632 - val_accuracy: 0.7521
Epoch 886/1000
270/270 [==============================] - 0s 88us/step - loss: 0.1976 - accuracy: 0.9000 - val_loss: 1.7559 - val_accuracy: 0.7521
Epoch 887/1000
270/270 [==============================] - 0s 88us/step - loss: 0.1990 - accuracy: 0.8963 - val_loss: 1.7821 - val_acc

Epoch 992/1000
270/270 [==============================] - 0s 89us/step - loss: 0.1902 - accuracy: 0.9037 - val_loss: 1.8405 - val_accuracy: 0.7436
Epoch 993/1000
270/270 [==============================] - 0s 94us/step - loss: 0.1915 - accuracy: 0.9037 - val_loss: 1.8380 - val_accuracy: 0.7521
Epoch 994/1000
270/270 [==============================] - 0s 97us/step - loss: 0.1911 - accuracy: 0.9000 - val_loss: 1.8421 - val_accuracy: 0.7778
Epoch 995/1000
270/270 [==============================] - 0s 92us/step - loss: 0.1913 - accuracy: 0.9037 - val_loss: 1.8382 - val_accuracy: 0.7778
Epoch 996/1000
270/270 [==============================] - 0s 93us/step - loss: 0.1927 - accuracy: 0.8963 - val_loss: 1.8363 - val_accuracy: 0.7436
Epoch 997/1000
270/270 [==============================] - 0s 88us/step - loss: 0.1887 - accuracy: 0.9037 - val_loss: 1.8742 - val_accuracy: 0.7521
Epoch 998/1000
270/270 [==============================] - 0s 87us/step - loss: 0.1983 - accuracy: 0.8963 - val_loss: 1

In [64]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 89.24%


In [31]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [32]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [33]:
y_prob2 = df_proba2[df_proba2['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[2.15860830e-03, 5.69891330e-02, 9.40852300e-01],
       [7.64700300e-06, 1.00275676e-04, 9.99892100e-01],
       [4.09279500e-06, 4.38070800e-02, 9.56188800e-01],
       [2.15860830e-03, 5.69891330e-02, 9.40852300e-01],
       [5.22663700e-03, 9.94773300e-01, 5.88873600e-12],
       [1.61924070e-05, 2.14124260e-04, 9.99769750e-01],
       [9.99935030e-01, 5.48772250e-10, 6.49313700e-05],
       [6.10989840e-03, 9.93808300e-01, 8.16970000e-05],
       [1.25642390e-02, 9.83071270e-01, 4.36453850e-03],
       [4.58489700e-01, 5.41494800e-01, 1.55094100e-05],
       [3.74113570e-07, 3.12022000e-10, 9.99999640e-01],
       [4.58489700e-01, 5.41494800e-01, 1.55094100e-05],
       [9.97113600e-01, 2.73161170e-03, 1.54796040e-04],
       [3.63304050e-01, 6.29359660e-01, 7.33629800e-03],
       [9.99933840e-01, 5.40984770e-05, 1.20237120e-05],
       [3.18029620e-08, 1.61292490e-07, 9.99999760e-01],
       [1.59327710e-03, 1.81019070e-04, 9.98225750e-01],
       [9.66296140e-01, 3.29109

In [34]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.8385930309007232

In [35]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.8385930309007232

In [36]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [37]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [38]:
dat3

,0,test
0,SR4187,0
1,NRS177,0
2,NRS109,2
3,CFBREBSa131,2
4,SR4152,1
...,...,...
112,NRS110,2
113,CFBRSa25,1
114,834N,0
115,CFBREBSa114,1


In [39]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [40]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 738us/step - loss: 13.7552 - accuracy: 0.3370 - val_loss: 12.2980 - val_accuracy: 0.3419
Epoch 2/1000
270/270 [==============================] - 0s 147us/step - loss: 7.3648 - accuracy: 0.4037 - val_loss: 4.4679 - val_accuracy: 0.2991
Epoch 3/1000
270/270 [==============================] - 0s 129us/step - loss: 3.7361 - accuracy: 0.3000 - val_loss: 1.9737 - val_accuracy: 0.2564
Epoch 4/1000
270/270 [==============================] - 0s 139us/step - loss: 1.5345 - accuracy: 0.2963 - val_loss: 1.8348 - val_accuracy: 0.3248
Epoch 5/1000
270/270 [==============================] - 0s 137us/step - loss: 1.5190 - accuracy: 0.3889 - val_loss: 1.5765 - val_accuracy: 0.4359
Epoch 6/1000
270/270 [==============================] - 0s 128us/step - loss: 1.2550 - accuracy: 0.4815 - val_loss: 1.1785 - val_accuracy: 0.4444
Epoch 7/1000
270/270 [==============================] - 0s 179us/step - loss

270/270 [==============================] - 0s 82us/step - loss: 0.6527 - accuracy: 0.7852 - val_loss: 1.0418 - val_accuracy: 0.7179
Epoch 112/1000
270/270 [==============================] - 0s 78us/step - loss: 0.5321 - accuracy: 0.8111 - val_loss: 0.8310 - val_accuracy: 0.6923
Epoch 113/1000
270/270 [==============================] - 0s 107us/step - loss: 0.6497 - accuracy: 0.7667 - val_loss: 1.1993 - val_accuracy: 0.7009
Epoch 114/1000
270/270 [==============================] - 0s 89us/step - loss: 0.5649 - accuracy: 0.8037 - val_loss: 0.8209 - val_accuracy: 0.6923
Epoch 115/1000
270/270 [==============================] - 0s 82us/step - loss: 0.4745 - accuracy: 0.8185 - val_loss: 0.8077 - val_accuracy: 0.7009
Epoch 116/1000
270/270 [==============================] - 0s 94us/step - loss: 0.4631 - accuracy: 0.8185 - val_loss: 0.8048 - val_accuracy: 0.7179
Epoch 117/1000
270/270 [==============================] - 0s 74us/step - loss: 0.4458 - accuracy: 0.8259 - val_loss: 0.8122 - val_ac

Epoch 167/1000
270/270 [==============================] - 0s 77us/step - loss: 0.4800 - accuracy: 0.8296 - val_loss: 0.8533 - val_accuracy: 0.7179
Epoch 168/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4694 - accuracy: 0.8259 - val_loss: 0.9299 - val_accuracy: 0.7265
Epoch 169/1000
270/270 [==============================] - 0s 78us/step - loss: 0.4248 - accuracy: 0.8444 - val_loss: 0.8847 - val_accuracy: 0.7009
Epoch 170/1000
270/270 [==============================] - 0s 83us/step - loss: 0.4366 - accuracy: 0.8259 - val_loss: 0.8979 - val_accuracy: 0.7265
Epoch 171/1000
270/270 [==============================] - 0s 87us/step - loss: 0.4020 - accuracy: 0.8407 - val_loss: 0.8482 - val_accuracy: 0.7179
Epoch 172/1000
270/270 [==============================] - 0s 106us/step - loss: 0.4020 - accuracy: 0.8481 - val_loss: 0.8701 - val_accuracy: 0.7436
Epoch 173/1000
270/270 [==============================] - 0s 80us/step - loss: 0.3908 - accuracy: 0.8444 - val_loss:

270/270 [==============================] - 0s 150us/step - loss: 0.3659 - accuracy: 0.8593 - val_loss: 0.9493 - val_accuracy: 0.7521
Epoch 278/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3499 - accuracy: 0.8593 - val_loss: 1.0498 - val_accuracy: 0.7521
Epoch 279/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3920 - accuracy: 0.8444 - val_loss: 0.9684 - val_accuracy: 0.7692
Epoch 280/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4261 - accuracy: 0.8519 - val_loss: 1.0758 - val_accuracy: 0.7436
Epoch 281/1000
270/270 [==============================] - 0s 104us/step - loss: 0.5847 - accuracy: 0.7889 - val_loss: 1.3644 - val_accuracy: 0.7179
Epoch 282/1000
270/270 [==============================] - 0s 91us/step - loss: 0.4497 - accuracy: 0.8556 - val_loss: 1.0741 - val_accuracy: 0.7265
Epoch 283/1000
270/270 [==============================] - 0s 83us/step - loss: 0.4229 - accuracy: 0.8185 - val_loss: 1.1752 - val_

Epoch 333/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2907 - accuracy: 0.8815 - val_loss: 0.9748 - val_accuracy: 0.7607
Epoch 334/1000
270/270 [==============================] - 0s 103us/step - loss: 0.2931 - accuracy: 0.8778 - val_loss: 1.0088 - val_accuracy: 0.7521
Epoch 335/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2949 - accuracy: 0.8741 - val_loss: 0.9908 - val_accuracy: 0.7436
Epoch 336/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2886 - accuracy: 0.8741 - val_loss: 0.9737 - val_accuracy: 0.7692
Epoch 337/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2916 - accuracy: 0.8815 - val_loss: 0.9798 - val_accuracy: 0.7692
Epoch 338/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2868 - accuracy: 0.8815 - val_loss: 0.9977 - val_accuracy: 0.7521
Epoch 339/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2773 - accuracy: 0.8926 - val_

...

Epoch 665/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2543 - accuracy: 0.8926 - val_loss: 1.4635 - val_accuracy: 0.7350
Epoch 666/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2449 - accuracy: 0.8963 - val_loss: 1.3415 - val_accuracy: 0.7521
Epoch 667/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2337 - accuracy: 0.9074 - val_loss: 1.3500 - val_accuracy: 0.7436
Epoch 668/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2287 - accuracy: 0.9074 - val_loss: 1.3292 - val_accuracy: 0.7436
Epoch 669/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2296 - accuracy: 0.9074 - val_loss: 1.3495 - val_accuracy: 0.7436
Epoch 670/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2292 - accuracy: 0.9148 - val_loss: 1.3224 - val_accuracy: 0.7521
Epoch 671/1000
270/270 [==============================] - 0s 98us/step - loss: 0.2327 - accuracy: 0.9111 - val_loss: 1

Epoch 721/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2156 - accuracy: 0.9111 - val_loss: 1.3834 - val_accuracy: 0.7436
Epoch 722/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2176 - accuracy: 0.9111 - val_loss: 1.3702 - val_accuracy: 0.7521
Epoch 723/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2171 - accuracy: 0.9111 - val_loss: 1.3720 - val_accuracy: 0.7521
Epoch 724/1000
270/270 [==============================] - 0s 101us/step - loss: 0.2229 - accuracy: 0.9074 - val_loss: 1.4054 - val_accuracy: 0.7436
Epoch 725/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2648 - accuracy: 0.8963 - val_loss: 1.3509 - val_accuracy: 0.7521
Epoch 726/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2237 - accuracy: 0.9111 - val_loss: 1.4334 - val_accuracy: 0.7436
Epoch 727/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2188 - accuracy: 0.9074 - val_loss: 

270/270 [==============================] - 0s 93us/step - loss: 0.2011 - accuracy: 0.9148 - val_loss: 1.4544 - val_accuracy: 0.7436
Epoch 832/1000
270/270 [==============================] - 0s 98us/step - loss: 0.1971 - accuracy: 0.9148 - val_loss: 1.4977 - val_accuracy: 0.7436
Epoch 833/1000
270/270 [==============================] - 0s 99us/step - loss: 0.1970 - accuracy: 0.9111 - val_loss: 1.5029 - val_accuracy: 0.7521
Epoch 834/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2003 - accuracy: 0.9111 - val_loss: 1.4542 - val_accuracy: 0.7436
Epoch 835/1000
270/270 [==============================] - 0s 99us/step - loss: 0.1983 - accuracy: 0.9148 - val_loss: 1.4563 - val_accuracy: 0.7521
Epoch 836/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2101 - accuracy: 0.9111 - val_loss: 1.4838 - val_accuracy: 0.7521
Epoch 837/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2009 - accuracy: 0.9148 - val_loss: 1.4928 - val_acc

Epoch 887/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3010 - accuracy: 0.8593 - val_loss: 1.5379 - val_accuracy: 0.7436
Epoch 888/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2558 - accuracy: 0.8815 - val_loss: 1.6977 - val_accuracy: 0.6923
Epoch 889/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2645 - accuracy: 0.8852 - val_loss: 1.4161 - val_accuracy: 0.7436
Epoch 890/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2541 - accuracy: 0.8926 - val_loss: 1.5080 - val_accuracy: 0.7350
Epoch 891/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2433 - accuracy: 0.8963 - val_loss: 1.5010 - val_accuracy: 0.7436
Epoch 892/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2336 - accuracy: 0.9037 - val_loss: 1.5353 - val_accuracy: 0.7436
Epoch 893/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2677 - accuracy: 0.9000 - val_loss: 1

Epoch 998/1000
270/270 [==============================] - 0s 111us/step - loss: 0.1960 - accuracy: 0.9037 - val_loss: 1.5485 - val_accuracy: 0.7607
Epoch 999/1000
270/270 [==============================] - 0s 99us/step - loss: 0.1938 - accuracy: 0.9148 - val_loss: 1.5689 - val_accuracy: 0.7521
Epoch 1000/1000
270/270 [==============================] - 0s 102us/step - loss: 0.1992 - accuracy: 0.9148 - val_loss: 1.5550 - val_accuracy: 0.7607


In [83]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

117/117 [==============================] - 0s 115us/step
over-sampling test accuracy: 75.21%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 1, 2, 2, 1, 1, 1, 2, 0, 2, 2, 1, 2, 0, 1, 1, 2, 1, 1, 0, 1, 1,
       2, 2, 0, 1, 1, 2, 1, 1, 0, 2, 1, 1, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2,
       2, 2, 0, 2, 1, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 1, 1, 1, 2,
       1, 2, 0, 0, 1, 0, 0, 2, 2, 1, 2, 0, 2, 1, 1, 2, 1, 2, 0, 1, 2, 0,
       0, 1, 2, 0, 1, 1, 2, 2, 1, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 1,
       1, 0, 1, 1, 0, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,SR4187,0,0
1,NRS177,0,1
2,NRS109,2,2
3,CFBREBSa131,2,2
4,SR4152,1,1
...,...,...,...
112,NRS110,2,1
113,CFBRSa25,1,1
114,834N,0,0
115,CFBREBSa114,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,9.999993e-01,6.737826e-07,1.940007e-31
1,4.537925e-04,9.995067e-01,3.944386e-05
2,1.173801e-07,8.213616e-07,9.999990e-01
3,5.930540e-11,7.765037e-11,1.000000e+00
4,5.647478e-02,5.480984e-01,3.954268e-01
...,...,...,...
112,3.213580e-01,5.903757e-01,8.826631e-02
113,1.876621e-01,8.106524e-01,1.685426e-03
114,9.999999e-01,1.425835e-07,1.091444e-08
115,2.468554e-01,7.531447e-01,2.310240e-08


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p006pST.csv", index = False,
         header=None)

In [87]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 116us/step - loss: 0.1950 - accuracy: 0.9148 - val_loss: 1.7646 - val_accuracy: 0.7521
Epoch 2/1000
270/270 [==============================] - 0s 129us/step - loss: 0.1971 - accuracy: 0.9148 - val_loss: 1.7548 - val_accuracy: 0.7521
Epoch 3/1000
270/270 [==============================] - 0s 131us/step - loss: 0.1954 - accuracy: 0.9148 - val_loss: 1.7398 - val_accuracy: 0.7607
Epoch 4/1000
270/270 [==============================] - 0s 165us/step - loss: 0.1998 - accuracy: 0.9148 - val_loss: 1.7524 - val_accuracy: 0.7521
Epoch 5/1000
270/270 [==============================] - 0s 119us/step - loss: 0.1972 - accuracy: 0.9148 - val_loss: 1.7355 - val_accuracy: 0.7607
Epoch 6/1000
270/270 [==============================] - 0s 121us/step - loss: 0.2019 - accuracy: 0.9074 - val_loss: 1.7865 - val_accuracy: 0.7521
Epoch 7/1000
270/270 [==============================] - 0s 150us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 141us/step - loss: 0.1837 - accuracy: 0.9148 - val_loss: 1.8620 - val_accuracy: 0.7436
Epoch 113/1000
270/270 [==============================] - 0s 136us/step - loss: 0.2003 - accuracy: 0.9111 - val_loss: 1.9088 - val_accuracy: 0.7521
Epoch 114/1000
270/270 [==============================] - 0s 149us/step - loss: 0.1931 - accuracy: 0.9074 - val_loss: 1.8902 - val_accuracy: 0.7436
Epoch 115/1000
270/270 [==============================] - 0s 173us/step - loss: 0.1916 - accuracy: 0.9111 - val_loss: 1.9083 - val_accuracy: 0.7436
Epoch 116/1000
270/270 [==============================] - 0s 563us/step - loss: 0.1929 - accuracy: 0.9148 - val_loss: 1.8839 - val_accuracy: 0.7436
Epoch 117/1000
270/270 [==============================] - 0s 519us/step - loss: 0.1835 - accuracy: 0.9148 - val_loss: 1.8670 - val_accuracy: 0.7607
Epoch 118/1000
270/270 [==============================] - 0s 142us/step - loss: 0.2029 - accuracy: 0.9074 - val_

Epoch 222/1000
270/270 [==============================] - 0s 227us/step - loss: 0.1800 - accuracy: 0.9111 - val_loss: 1.9788 - val_accuracy: 0.7521
Epoch 223/1000
270/270 [==============================] - 0s 107us/step - loss: 0.1863 - accuracy: 0.9148 - val_loss: 1.9771 - val_accuracy: 0.7436
Epoch 224/1000
270/270 [==============================] - 0s 120us/step - loss: 0.1832 - accuracy: 0.9185 - val_loss: 1.9205 - val_accuracy: 0.7521
Epoch 225/1000
270/270 [==============================] - 0s 126us/step - loss: 0.1832 - accuracy: 0.9185 - val_loss: 1.9738 - val_accuracy: 0.7521
Epoch 226/1000
270/270 [==============================] - 0s 156us/step - loss: 0.1808 - accuracy: 0.9000 - val_loss: 2.0178 - val_accuracy: 0.7521
Epoch 227/1000
270/270 [==============================] - 0s 186us/step - loss: 0.1928 - accuracy: 0.9148 - val_loss: 1.9877 - val_accuracy: 0.7521
Epoch 228/1000
270/270 [==============================] - 0s 252us/step - loss: 0.1793 - accuracy: 0.9148 - val_

270/270 [==============================] - 0s 203us/step - loss: 0.1790 - accuracy: 0.9148 - val_loss: 1.9891 - val_accuracy: 0.7521
Epoch 333/1000
270/270 [==============================] - 0s 210us/step - loss: 0.1786 - accuracy: 0.9148 - val_loss: 2.0096 - val_accuracy: 0.7692
Epoch 334/1000
270/270 [==============================] - 0s 181us/step - loss: 0.1758 - accuracy: 0.9185 - val_loss: 2.0322 - val_accuracy: 0.7607
Epoch 335/1000
270/270 [==============================] - 0s 156us/step - loss: 0.1813 - accuracy: 0.9185 - val_loss: 2.0170 - val_accuracy: 0.7607
Epoch 336/1000
270/270 [==============================] - 0s 275us/step - loss: 0.1795 - accuracy: 0.9148 - val_loss: 1.9707 - val_accuracy: 0.7607
Epoch 337/1000
270/270 [==============================] - 0s 117us/step - loss: 0.1802 - accuracy: 0.9148 - val_loss: 2.0033 - val_accuracy: 0.7521
Epoch 338/1000
270/270 [==============================] - 0s 112us/step - loss: 0.1746 - accuracy: 0.9148 - val_loss: 2.0086 - 

270/270 [==============================] - 0s 103us/step - loss: 0.1809 - accuracy: 0.9148 - val_loss: 2.0817 - val_accuracy: 0.7521
Epoch 443/1000
270/270 [==============================] - 0s 118us/step - loss: 0.1860 - accuracy: 0.9148 - val_loss: 2.0351 - val_accuracy: 0.7607
Epoch 444/1000
270/270 [==============================] - 0s 129us/step - loss: 0.1826 - accuracy: 0.9185 - val_loss: 2.0527 - val_accuracy: 0.7692
Epoch 445/1000
270/270 [==============================] - 0s 161us/step - loss: 0.1930 - accuracy: 0.9074 - val_loss: 2.1179 - val_accuracy: 0.7607
Epoch 446/1000
270/270 [==============================] - 0s 193us/step - loss: 0.3220 - accuracy: 0.8963 - val_loss: 2.1293 - val_accuracy: 0.7607
Epoch 447/1000
270/270 [==============================] - 0s 389us/step - loss: 0.1927 - accuracy: 0.9148 - val_loss: 2.1164 - val_accuracy: 0.7350
Epoch 448/1000
270/270 [==============================] - 0s 280us/step - loss: 0.2037 - accuracy: 0.9148 - val_loss: 2.1636 - 

270/270 [==============================] - 0s 154us/step - loss: 0.1805 - accuracy: 0.9148 - val_loss: 2.0299 - val_accuracy: 0.7521
Epoch 553/1000
270/270 [==============================] - 0s 114us/step - loss: 0.1858 - accuracy: 0.9148 - val_loss: 2.0549 - val_accuracy: 0.7350
Epoch 554/1000
270/270 [==============================] - 0s 128us/step - loss: 0.1857 - accuracy: 0.9222 - val_loss: 2.0836 - val_accuracy: 0.7350
Epoch 555/1000
270/270 [==============================] - 0s 113us/step - loss: 0.1839 - accuracy: 0.9185 - val_loss: 2.0508 - val_accuracy: 0.7521
Epoch 556/1000
270/270 [==============================] - 0s 136us/step - loss: 0.1871 - accuracy: 0.9148 - val_loss: 2.0469 - val_accuracy: 0.7521
Epoch 557/1000
270/270 [==============================] - 0s 116us/step - loss: 0.1833 - accuracy: 0.9148 - val_loss: 2.0748 - val_accuracy: 0.7436
Epoch 558/1000
270/270 [==============================] - 0s 131us/step - loss: 0.1825 - accuracy: 0.9222 - val_loss: 2.0510 - 

270/270 [==============================] - 0s 125us/step - loss: 0.1904 - accuracy: 0.9074 - val_loss: 2.0249 - val_accuracy: 0.7607
Epoch 663/1000
270/270 [==============================] - 0s 116us/step - loss: 0.1759 - accuracy: 0.9185 - val_loss: 2.1084 - val_accuracy: 0.7436
Epoch 664/1000
270/270 [==============================] - 0s 362us/step - loss: 0.1922 - accuracy: 0.9148 - val_loss: 2.0882 - val_accuracy: 0.7521
Epoch 665/1000
270/270 [==============================] - 0s 172us/step - loss: 0.1882 - accuracy: 0.9111 - val_loss: 2.1084 - val_accuracy: 0.7350
Epoch 666/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2036 - accuracy: 0.9148 - val_loss: 2.1846 - val_accuracy: 0.7350
Epoch 667/1000
270/270 [==============================] - 0s 106us/step - loss: 0.2534 - accuracy: 0.8963 - val_loss: 2.0846 - val_accuracy: 0.7436
Epoch 668/1000
270/270 [==============================] - 0s 139us/step - loss: 0.2130 - accuracy: 0.9074 - val_loss: 1.9809 - 

270/270 [==============================] - 0s 183us/step - loss: 0.1837 - accuracy: 0.9148 - val_loss: 2.1056 - val_accuracy: 0.7436
Epoch 773/1000
270/270 [==============================] - 0s 125us/step - loss: 0.1736 - accuracy: 0.9185 - val_loss: 2.0720 - val_accuracy: 0.7521
Epoch 774/1000
270/270 [==============================] - 0s 121us/step - loss: 0.1783 - accuracy: 0.9074 - val_loss: 2.0835 - val_accuracy: 0.7521
Epoch 775/1000
270/270 [==============================] - 0s 116us/step - loss: 0.1816 - accuracy: 0.9074 - val_loss: 2.1055 - val_accuracy: 0.7521
Epoch 776/1000
270/270 [==============================] - 0s 129us/step - loss: 0.1739 - accuracy: 0.9148 - val_loss: 2.0733 - val_accuracy: 0.7521
Epoch 777/1000
270/270 [==============================] - 0s 124us/step - loss: 0.1761 - accuracy: 0.9185 - val_loss: 2.0865 - val_accuracy: 0.7521
Epoch 778/1000
270/270 [==============================] - 0s 259us/step - loss: 0.1742 - accuracy: 0.9148 - val_loss: 2.0909 - 

270/270 [==============================] - 0s 100us/step - loss: 0.1761 - accuracy: 0.9148 - val_loss: 2.1105 - val_accuracy: 0.7521
Epoch 883/1000
270/270 [==============================] - 0s 124us/step - loss: 0.1791 - accuracy: 0.9185 - val_loss: 2.1269 - val_accuracy: 0.7521
Epoch 884/1000
270/270 [==============================] - 0s 100us/step - loss: 0.1725 - accuracy: 0.9185 - val_loss: 2.1075 - val_accuracy: 0.7607
Epoch 885/1000
270/270 [==============================] - 0s 92us/step - loss: 0.1737 - accuracy: 0.9185 - val_loss: 2.1072 - val_accuracy: 0.7521
Epoch 886/1000
270/270 [==============================] - 0s 97us/step - loss: 0.1755 - accuracy: 0.9148 - val_loss: 2.1386 - val_accuracy: 0.7436
Epoch 887/1000
270/270 [==============================] - 0s 100us/step - loss: 0.1727 - accuracy: 0.9148 - val_loss: 2.1201 - val_accuracy: 0.7521
Epoch 888/1000
270/270 [==============================] - 0s 98us/step - loss: 0.1760 - accuracy: 0.9111 - val_loss: 2.1351 - val

Epoch 938/1000
270/270 [==============================] - 0s 91us/step - loss: 0.1801 - accuracy: 0.9074 - val_loss: 2.1324 - val_accuracy: 0.7521
Epoch 939/1000
270/270 [==============================] - 0s 97us/step - loss: 0.1766 - accuracy: 0.9185 - val_loss: 2.1422 - val_accuracy: 0.7607
Epoch 940/1000
270/270 [==============================] - 0s 86us/step - loss: 0.1768 - accuracy: 0.9185 - val_loss: 2.1896 - val_accuracy: 0.7521
Epoch 941/1000
270/270 [==============================] - 0s 97us/step - loss: 0.1783 - accuracy: 0.9074 - val_loss: 2.2274 - val_accuracy: 0.7436
Epoch 942/1000
270/270 [==============================] - 0s 108us/step - loss: 0.1694 - accuracy: 0.9185 - val_loss: 2.1658 - val_accuracy: 0.7436
Epoch 943/1000
270/270 [==============================] - 0s 98us/step - loss: 0.1863 - accuracy: 0.9037 - val_loss: 2.1518 - val_accuracy: 0.7607
Epoch 944/1000
270/270 [==============================] - 0s 100us/step - loss: 0.1844 - accuracy: 0.9222 - val_loss:

In [88]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 91.30%


In [41]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [42]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [43]:
y_prob3 = df_proba3[df_proba3['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.99999300e-01, 6.73782600e-07, 1.94000740e-31],
       [4.53792450e-04, 9.99506700e-01, 3.94438650e-05],
       [1.17380090e-07, 8.21361600e-07, 9.99999050e-01],
       [5.93054000e-11, 7.76503700e-11, 1.00000000e+00],
       [5.64747830e-02, 5.48098440e-01, 3.95426750e-01],
       [2.54870040e-04, 9.99745100e-01, 1.50869730e-10],
       [3.97096260e-08, 9.99995700e-01, 4.26113960e-06],
       [5.93054000e-11, 7.76503700e-11, 1.00000000e+00],
       [9.41297400e-01, 5.86939600e-02, 8.63650700e-06],
       [5.93054000e-11, 7.76503700e-11, 1.00000000e+00],
       [3.15733660e-10, 9.97200300e-05, 9.99900200e-01],
       [9.24742000e-03, 9.90596400e-01, 1.56284720e-04],
       [6.06061220e-05, 5.25788700e-03, 9.94681540e-01],
       [9.95665700e-01, 4.33435570e-03, 4.68417800e-08],
       [3.63526050e-02, 9.63647370e-01, 9.72463100e-14],
       [8.36295600e-03, 9.59754470e-01, 3.18826000e-02],
       [1.63116100e-08, 1.13026510e-05, 9.99988700e-01],
       [2.46855380e-01, 7.53144

In [44]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.8768354152969536

In [45]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.8768354152969536

In [46]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [47]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [48]:
dat4

,0,test
0,CFBRSa25,1
1,CFBRSa07,0
2,NRS247,0
3,NY439,2
4,CFBREBSa110,1
...,...,...
112,SR1129,0
113,NRS172,0
114,NRS205,2
115,NY439,2


In [49]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [50]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 647us/step - loss: 13.1761 - accuracy: 0.2963 - val_loss: 4.8338 - val_accuracy: 0.3590
Epoch 2/1000
270/270 [==============================] - 0s 121us/step - loss: 9.6635 - accuracy: 0.3778 - val_loss: 3.5890 - val_accuracy: 0.3333
Epoch 3/1000
270/270 [==============================] - 0s 120us/step - loss: 7.1466 - accuracy: 0.4000 - val_loss: 2.5613 - val_accuracy: 0.3504
Epoch 4/1000
270/270 [==============================] - 0s 146us/step - loss: 4.2875 - accuracy: 0.4037 - val_loss: 1.8755 - val_accuracy: 0.3504
Epoch 5/1000
270/270 [==============================] - 0s 102us/step - loss: 2.7178 - accuracy: 0.4074 - val_loss: 1.2517 - val_accuracy: 0.3419
Epoch 6/1000
270/270 [==============================] - 0s 164us/step - loss: 1.4507 - accuracy: 0.3778 - val_loss: 1.7183 - val_accuracy: 0.3675
Epoch 7/1000
270/270 [==============================] - 0s 120us/step - loss:

Epoch 112/1000
270/270 [==============================] - 0s 58us/step - loss: 0.5308 - accuracy: 0.7741 - val_loss: 0.9970 - val_accuracy: 0.7094
Epoch 113/1000
270/270 [==============================] - 0s 55us/step - loss: 0.5424 - accuracy: 0.7741 - val_loss: 0.9971 - val_accuracy: 0.6581
Epoch 114/1000
270/270 [==============================] - 0s 77us/step - loss: 0.5401 - accuracy: 0.7815 - val_loss: 1.0235 - val_accuracy: 0.7265
Epoch 115/1000
270/270 [==============================] - 0s 61us/step - loss: 0.5389 - accuracy: 0.7778 - val_loss: 1.0106 - val_accuracy: 0.6410
Epoch 116/1000
270/270 [==============================] - 0s 57us/step - loss: 0.5185 - accuracy: 0.7889 - val_loss: 0.9955 - val_accuracy: 0.6838
Epoch 117/1000
270/270 [==============================] - 0s 58us/step - loss: 0.5225 - accuracy: 0.7963 - val_loss: 0.9960 - val_accuracy: 0.6581
Epoch 118/1000
270/270 [==============================] - 0s 86us/step - loss: 0.5135 - accuracy: 0.7852 - val_loss: 0

Epoch 223/1000
270/270 [==============================] - 0s 74us/step - loss: 0.3951 - accuracy: 0.8556 - val_loss: 1.2289 - val_accuracy: 0.6667
Epoch 224/1000
270/270 [==============================] - 0s 57us/step - loss: 0.4036 - accuracy: 0.8481 - val_loss: 1.2348 - val_accuracy: 0.7009
Epoch 225/1000
270/270 [==============================] - 0s 57us/step - loss: 0.4044 - accuracy: 0.8519 - val_loss: 1.2278 - val_accuracy: 0.6923
Epoch 226/1000
270/270 [==============================] - 0s 60us/step - loss: 0.4068 - accuracy: 0.8407 - val_loss: 1.1998 - val_accuracy: 0.6325
Epoch 227/1000
270/270 [==============================] - 0s 58us/step - loss: 0.3990 - accuracy: 0.8222 - val_loss: 1.2449 - val_accuracy: 0.6410
Epoch 228/1000
270/270 [==============================] - 0s 72us/step - loss: 0.3974 - accuracy: 0.8519 - val_loss: 1.2398 - val_accuracy: 0.6752
Epoch 229/1000
270/270 [==============================] - 0s 58us/step - loss: 0.3896 - accuracy: 0.8481 - val_loss: 1

Epoch 334/1000
270/270 [==============================] - 0s 67us/step - loss: 0.3623 - accuracy: 0.8593 - val_loss: 1.3768 - val_accuracy: 0.6581
Epoch 335/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3570 - accuracy: 0.8444 - val_loss: 1.3638 - val_accuracy: 0.6581
Epoch 336/1000
270/270 [==============================] - 0s 84us/step - loss: 0.3433 - accuracy: 0.8667 - val_loss: 1.3534 - val_accuracy: 0.7179
Epoch 337/1000
270/270 [==============================] - 0s 90us/step - loss: 0.3567 - accuracy: 0.8704 - val_loss: 1.3598 - val_accuracy: 0.7009
Epoch 338/1000
270/270 [==============================] - 0s 73us/step - loss: 0.3474 - accuracy: 0.8889 - val_loss: 1.3893 - val_accuracy: 0.6667
Epoch 339/1000
270/270 [==============================] - 0s 76us/step - loss: 0.3473 - accuracy: 0.8741 - val_loss: 1.3866 - val_accuracy: 0.7009
Epoch 340/1000
270/270 [==============================] - 0s 72us/step - loss: 0.3475 - accuracy: 0.8667 - val_loss: 1

Epoch 390/1000
270/270 [==============================] - 0s 51us/step - loss: 0.3226 - accuracy: 0.8778 - val_loss: 1.4485 - val_accuracy: 0.6154
Epoch 391/1000
270/270 [==============================] - 0s 51us/step - loss: 0.3196 - accuracy: 0.8704 - val_loss: 1.4496 - val_accuracy: 0.6154
Epoch 392/1000
270/270 [==============================] - 0s 51us/step - loss: 0.3241 - accuracy: 0.8815 - val_loss: 1.4345 - val_accuracy: 0.6752
Epoch 393/1000
270/270 [==============================] - 0s 48us/step - loss: 0.3166 - accuracy: 0.8815 - val_loss: 1.4594 - val_accuracy: 0.6325
Epoch 394/1000
270/270 [==============================] - 0s 53us/step - loss: 0.3254 - accuracy: 0.8519 - val_loss: 1.4549 - val_accuracy: 0.6410
Epoch 395/1000
270/270 [==============================] - 0s 56us/step - loss: 0.3174 - accuracy: 0.8778 - val_loss: 1.4433 - val_accuracy: 0.6667
Epoch 396/1000
270/270 [==============================] - 0s 50us/step - loss: 0.3279 - accuracy: 0.8630 - val_loss: 1

...

Epoch 614/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2680 - accuracy: 0.8852 - val_loss: 1.5742 - val_accuracy: 0.6752
Epoch 615/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2648 - accuracy: 0.8889 - val_loss: 1.5457 - val_accuracy: 0.6923
Epoch 616/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2678 - accuracy: 0.8889 - val_loss: 1.5167 - val_accuracy: 0.7094
Epoch 617/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2631 - accuracy: 0.8926 - val_loss: 1.5509 - val_accuracy: 0.7009
Epoch 618/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2666 - accuracy: 0.8963 - val_loss: 1.5570 - val_accuracy: 0.6923
Epoch 619/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2607 - accuracy: 0.9000 - val_loss: 1.5461 - val_accuracy: 0.7350
Epoch 620/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2667 - accuracy: 0.8963 - val_loss: 1

Epoch 670/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2884 - accuracy: 0.8778 - val_loss: 1.5898 - val_accuracy: 0.7179
Epoch 671/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2564 - accuracy: 0.8926 - val_loss: 1.6073 - val_accuracy: 0.7436
Epoch 672/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2622 - accuracy: 0.8852 - val_loss: 1.5889 - val_accuracy: 0.7009
Epoch 673/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2531 - accuracy: 0.9000 - val_loss: 1.5942 - val_accuracy: 0.7436
Epoch 674/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2566 - accuracy: 0.8926 - val_loss: 1.6018 - val_accuracy: 0.7350
Epoch 675/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2644 - accuracy: 0.8889 - val_loss: 1.5991 - val_accuracy: 0.7009
Epoch 676/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2518 - accuracy: 0.8926 - val_loss: 1

Epoch 726/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2783 - accuracy: 0.8778 - val_loss: 1.7842 - val_accuracy: 0.6667
Epoch 727/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2713 - accuracy: 0.8741 - val_loss: 1.6992 - val_accuracy: 0.7265
Epoch 728/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2919 - accuracy: 0.8741 - val_loss: 1.7592 - val_accuracy: 0.7350
Epoch 729/1000
270/270 [==============================] - 0s 88us/step - loss: 0.3006 - accuracy: 0.8889 - val_loss: 1.7567 - val_accuracy: 0.7350
Epoch 730/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2574 - accuracy: 0.8963 - val_loss: 1.7825 - val_accuracy: 0.7094
Epoch 731/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2825 - accuracy: 0.8852 - val_loss: 1.7463 - val_accuracy: 0.7350
Epoch 732/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2770 - accuracy: 0.8963 - val_loss: 1

Epoch 782/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2457 - accuracy: 0.8889 - val_loss: 1.8670 - val_accuracy: 0.6667
Epoch 783/1000
270/270 [==============================] - 0s 76us/step - loss: 0.3085 - accuracy: 0.8741 - val_loss: 1.8285 - val_accuracy: 0.6923
Epoch 784/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2605 - accuracy: 0.9000 - val_loss: 1.7901 - val_accuracy: 0.7179
Epoch 785/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2455 - accuracy: 0.9000 - val_loss: 1.8209 - val_accuracy: 0.6752
Epoch 786/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2516 - accuracy: 0.8926 - val_loss: 1.7983 - val_accuracy: 0.6923
Epoch 787/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2625 - accuracy: 0.8963 - val_loss: 1.8254 - val_accuracy: 0.6923
Epoch 788/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2618 - accuracy: 0.8963 - val_loss: 1

Epoch 893/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2394 - accuracy: 0.8963 - val_loss: 1.9874 - val_accuracy: 0.6581
Epoch 894/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2303 - accuracy: 0.8926 - val_loss: 2.0166 - val_accuracy: 0.7009
Epoch 895/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2462 - accuracy: 0.8963 - val_loss: 1.9127 - val_accuracy: 0.7436
Epoch 896/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2348 - accuracy: 0.9000 - val_loss: 1.8875 - val_accuracy: 0.7436
Epoch 897/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2308 - accuracy: 0.9037 - val_loss: 1.9299 - val_accuracy: 0.7436
Epoch 898/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2238 - accuracy: 0.9037 - val_loss: 1.9568 - val_accuracy: 0.7009
Epoch 899/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2291 - accuracy: 0.9037 - val_loss: 1

In [112]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

117/117 [==============================] - 0s 101us/step
over-sampling test accuracy: 70.09%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 1, 1, 2, 2, 2, 0, 1, 2, 2,
       1, 1, 0, 0, 0, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 2, 0, 0,
       1, 2, 0, 1, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 2, 1, 0, 1, 1, 2, 1, 1,
       2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 1, 1, 0, 0, 2, 2, 2, 1,
       2, 2, 2, 2, 1, 2, 2, 2, 1, 0, 1, 1, 2, 2, 1, 1, 2, 1, 0, 2, 1, 1,
       0, 1, 0, 1, 2, 2, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa25,1,1
1,CFBRSa07,0,0
2,NRS247,0,1
3,NY439,2,2
4,CFBREBSa110,1,0
...,...,...,...
112,SR1129,0,0
113,NRS172,0,1
114,NRS205,2,2
115,NY439,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,6.561665e-02,9.325171e-01,1.866363e-03
1,9.670122e-01,3.033776e-02,2.650134e-03
2,1.829924e-01,8.158398e-01,1.167765e-03
3,4.501102e-05,3.545064e-04,9.996005e-01
4,8.193218e-01,1.806760e-01,2.293046e-06
...,...,...,...
112,9.976769e-01,2.323148e-03,3.024173e-14
113,6.095036e-03,9.939049e-01,1.122750e-07
114,9.795157e-13,2.333501e-12,1.000000e+00
115,4.501102e-05,3.545064e-04,9.996005e-01


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p006pST.csv", index = False,
         header=None)

In [116]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2222 - accuracy: 0.8889 - val_loss: 1.8358 - val_accuracy: 0.6752
Epoch 2/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2268 - accuracy: 0.8926 - val_loss: 1.8247 - val_accuracy: 0.6838
Epoch 3/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2229 - accuracy: 0.9074 - val_loss: 1.8844 - val_accuracy: 0.6752
Epoch 4/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2248 - accuracy: 0.8963 - val_loss: 1.9013 - val_accuracy: 0.7179
Epoch 5/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2263 - accuracy: 0.9074 - val_loss: 1.8350 - val_accuracy: 0.7179
Epoch 6/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2556 - accuracy: 0.9074 - val_loss: 1.8539 - val_accuracy: 0.7094
Epoch 7/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2324

Epoch 57/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2107 - accuracy: 0.9111 - val_loss: 1.8780 - val_accuracy: 0.6752
Epoch 58/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2108 - accuracy: 0.9111 - val_loss: 1.8774 - val_accuracy: 0.7179
Epoch 59/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2084 - accuracy: 0.9111 - val_loss: 1.8427 - val_accuracy: 0.7179
Epoch 60/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2139 - accuracy: 0.9111 - val_loss: 1.8228 - val_accuracy: 0.7265
Epoch 61/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2142 - accuracy: 0.9111 - val_loss: 1.8376 - val_accuracy: 0.7265
Epoch 62/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2101 - accuracy: 0.9111 - val_loss: 1.8582 - val_accuracy: 0.7179
Epoch 63/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2096 - accuracy: 0.9111 - val_loss: 1.8773 -

270/270 [==============================] - 0s 69us/step - loss: 0.2660 - accuracy: 0.8815 - val_loss: 2.0109 - val_accuracy: 0.6667
Epoch 114/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2349 - accuracy: 0.8926 - val_loss: 1.8716 - val_accuracy: 0.6923
Epoch 115/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2508 - accuracy: 0.8926 - val_loss: 1.8769 - val_accuracy: 0.6923
Epoch 116/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2250 - accuracy: 0.9037 - val_loss: 1.9402 - val_accuracy: 0.6752
Epoch 117/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2489 - accuracy: 0.8852 - val_loss: 1.8548 - val_accuracy: 0.6752
Epoch 118/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2381 - accuracy: 0.9000 - val_loss: 1.9404 - val_accuracy: 0.6667
Epoch 119/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2470 - accuracy: 0.9000 - val_loss: 2.0993 - val_acc

Epoch 169/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2149 - accuracy: 0.9000 - val_loss: 1.8767 - val_accuracy: 0.6667
Epoch 170/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2265 - accuracy: 0.9037 - val_loss: 1.9133 - val_accuracy: 0.6752
Epoch 171/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2086 - accuracy: 0.9074 - val_loss: 1.9110 - val_accuracy: 0.6752
Epoch 172/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2033 - accuracy: 0.9037 - val_loss: 1.8696 - val_accuracy: 0.7265
Epoch 173/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2080 - accuracy: 0.9000 - val_loss: 1.8556 - val_accuracy: 0.7094
Epoch 174/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2083 - accuracy: 0.9037 - val_loss: 1.8865 - val_accuracy: 0.7179
Epoch 175/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2034 - accuracy: 0.9111 - val_loss: 1

Epoch 225/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2043 - accuracy: 0.9074 - val_loss: 1.9266 - val_accuracy: 0.7265
Epoch 226/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2102 - accuracy: 0.9074 - val_loss: 1.9458 - val_accuracy: 0.7094
Epoch 227/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2164 - accuracy: 0.9037 - val_loss: 1.9582 - val_accuracy: 0.7179
Epoch 228/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2163 - accuracy: 0.9000 - val_loss: 1.9663 - val_accuracy: 0.6752
Epoch 229/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2641 - accuracy: 0.8889 - val_loss: 2.0507 - val_accuracy: 0.6752
Epoch 230/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2626 - accuracy: 0.8963 - val_loss: 2.0437 - val_accuracy: 0.6752
Epoch 231/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2254 - accuracy: 0.8963 - val_loss: 1

...

Epoch 504/1000
270/270 [==============================] - 0s 66us/step - loss: 0.1984 - accuracy: 0.9111 - val_loss: 2.1969 - val_accuracy: 0.7350
Epoch 505/1000
270/270 [==============================] - 0s 61us/step - loss: 0.1969 - accuracy: 0.9037 - val_loss: 2.2374 - val_accuracy: 0.7350
Epoch 506/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2372 - accuracy: 0.9148 - val_loss: 2.3105 - val_accuracy: 0.6496
Epoch 507/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2318 - accuracy: 0.9037 - val_loss: 2.2466 - val_accuracy: 0.6496
Epoch 508/1000
270/270 [==============================] - 0s 71us/step - loss: 0.1963 - accuracy: 0.9037 - val_loss: 2.2865 - val_accuracy: 0.6581
Epoch 509/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2112 - accuracy: 0.9037 - val_loss: 2.2688 - val_accuracy: 0.6838
Epoch 510/1000
270/270 [==============================] - 0s 70us/step - loss: 0.1960 - accuracy: 0.9111 - val_loss: 2

Epoch 560/1000
270/270 [==============================] - 0s 81us/step - loss: 0.1951 - accuracy: 0.9074 - val_loss: 2.2351 - val_accuracy: 0.7179
Epoch 561/1000
270/270 [==============================] - 0s 71us/step - loss: 0.1902 - accuracy: 0.9074 - val_loss: 2.1962 - val_accuracy: 0.7179
Epoch 562/1000
270/270 [==============================] - 0s 58us/step - loss: 0.1957 - accuracy: 0.9074 - val_loss: 2.2213 - val_accuracy: 0.7094
Epoch 563/1000
270/270 [==============================] - 0s 76us/step - loss: 0.1896 - accuracy: 0.9074 - val_loss: 2.2813 - val_accuracy: 0.6667
Epoch 564/1000
270/270 [==============================] - 0s 64us/step - loss: 0.1916 - accuracy: 0.9037 - val_loss: 2.2687 - val_accuracy: 0.6838
Epoch 565/1000
270/270 [==============================] - 0s 62us/step - loss: 0.1926 - accuracy: 0.9037 - val_loss: 2.2439 - val_accuracy: 0.7009
Epoch 566/1000
270/270 [==============================] - 0s 82us/step - loss: 0.1933 - accuracy: 0.9000 - val_loss: 2

Epoch 671/1000
270/270 [==============================] - 0s 76us/step - loss: 0.1963 - accuracy: 0.9074 - val_loss: 2.4032 - val_accuracy: 0.6667
Epoch 672/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2053 - accuracy: 0.9074 - val_loss: 2.3497 - val_accuracy: 0.6838
Epoch 673/1000
270/270 [==============================] - 0s 75us/step - loss: 0.1924 - accuracy: 0.9000 - val_loss: 2.3253 - val_accuracy: 0.7265
Epoch 674/1000
270/270 [==============================] - 0s 65us/step - loss: 0.1962 - accuracy: 0.9111 - val_loss: 2.3298 - val_accuracy: 0.7265
Epoch 675/1000
270/270 [==============================] - 0s 71us/step - loss: 0.1971 - accuracy: 0.9148 - val_loss: 2.3314 - val_accuracy: 0.7265
Epoch 676/1000
270/270 [==============================] - 0s 74us/step - loss: 0.1889 - accuracy: 0.9111 - val_loss: 2.4067 - val_accuracy: 0.6752
Epoch 677/1000
270/270 [==============================] - 0s 78us/step - loss: 0.1881 - accuracy: 0.9111 - val_loss: 2

Epoch 782/1000
270/270 [==============================] - 0s 70us/step - loss: 0.1841 - accuracy: 0.9111 - val_loss: 2.4200 - val_accuracy: 0.6667
Epoch 783/1000
270/270 [==============================] - 0s 67us/step - loss: 0.1909 - accuracy: 0.9111 - val_loss: 2.4438 - val_accuracy: 0.6667
Epoch 784/1000
270/270 [==============================] - 0s 98us/step - loss: 0.1883 - accuracy: 0.9148 - val_loss: 2.3775 - val_accuracy: 0.7179
Epoch 785/1000
270/270 [==============================] - 0s 158us/step - loss: 0.1900 - accuracy: 0.9111 - val_loss: 2.3087 - val_accuracy: 0.7179
Epoch 786/1000
270/270 [==============================] - 0s 122us/step - loss: 0.1953 - accuracy: 0.9037 - val_loss: 2.3279 - val_accuracy: 0.7179
Epoch 787/1000
270/270 [==============================] - 0s 93us/step - loss: 0.1868 - accuracy: 0.9148 - val_loss: 2.4271 - val_accuracy: 0.7094
Epoch 788/1000
270/270 [==============================] - 0s 156us/step - loss: 0.2138 - accuracy: 0.9074 - val_loss

270/270 [==============================] - 0s 184us/step - loss: 0.1842 - accuracy: 0.9148 - val_loss: 2.3770 - val_accuracy: 0.7179
Epoch 893/1000
270/270 [==============================] - 0s 166us/step - loss: 0.1803 - accuracy: 0.9111 - val_loss: 2.3858 - val_accuracy: 0.7179
Epoch 894/1000
270/270 [==============================] - 0s 157us/step - loss: 0.1822 - accuracy: 0.9037 - val_loss: 2.3848 - val_accuracy: 0.7265
Epoch 895/1000
270/270 [==============================] - 0s 135us/step - loss: 0.1933 - accuracy: 0.9111 - val_loss: 2.3582 - val_accuracy: 0.7009
Epoch 896/1000
270/270 [==============================] - 0s 92us/step - loss: 0.1927 - accuracy: 0.9000 - val_loss: 2.4308 - val_accuracy: 0.7350
Epoch 897/1000
270/270 [==============================] - 0s 71us/step - loss: 0.1834 - accuracy: 0.9037 - val_loss: 2.4543 - val_accuracy: 0.6838
Epoch 898/1000
270/270 [==============================] - 0s 85us/step - loss: 0.1883 - accuracy: 0.9148 - val_loss: 2.4093 - val

In [117]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 90.40%


In [51]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [52]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [53]:
y_prob4 = df_proba4[df_proba4['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[6.56166500e-02, 9.32517050e-01, 1.86636300e-03],
       [9.67012170e-01, 3.03377560e-02, 2.65013380e-03],
       [1.82992350e-01, 8.15839800e-01, 1.16776470e-03],
       [4.50110240e-05, 3.54506370e-04, 9.99600470e-01],
       [8.19321750e-01, 1.80675970e-01, 2.29304580e-06],
       [9.99999400e-01, 2.70702710e-11, 6.37441100e-07],
       [9.87103000e-15, 1.58808800e-10, 1.00000000e+00],
       [9.99995230e-01, 4.15172800e-06, 5.80358860e-07],
       [8.27239060e-05, 2.33881570e-02, 9.76529100e-01],
       [5.14501900e-01, 4.85249550e-01, 2.48478750e-04],
       [9.99864460e-01, 1.35532800e-04, 6.10054050e-09],
       [9.23980700e-06, 8.03031500e-02, 9.19687600e-01],
       [8.27239060e-05, 2.33881570e-02, 9.76529100e-01],
       [4.15684200e-05, 9.99957300e-01, 1.01330970e-06],
       [4.02427340e-04, 9.98903900e-01, 6.93639860e-04],
       [7.26356700e-08, 1.85878200e-01, 8.14121800e-01],
       [1.58807010e-17, 4.90023200e-12, 1.00000000e+00],
       [2.20035420e-10, 3.59601

In [54]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.8275257506026737

In [55]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.8275257506026737

In [56]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.8431404777558624

In [57]:
np.std(ovos)

0.01989336247156682

In [58]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.8431404777558624

In [59]:
np.std(ovrs)

0.01989336247156682

In [118]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [119]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 72.86%


In [120]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.03275578804067092


In [121]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [122]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 90.57%


In [123]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.008512871


In [49]:
############ Feature selection using lasso ##########

In [17]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [18]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X_over[:,1:], y_over)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [19]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [20]:
X_train_features_over = np.vstack((names, X_over[:,1:]))
X_train_features_over = pd.DataFrame(X_train_features_over)

In [21]:
sel_features = X_train_features_over.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_over.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 96
selected features: 62


In [22]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 18,
        20, 21, 23, 24, 25, 27, 29, 31, 32, 36, 38, 40, 41, 42, 45, 46,
        47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 64,
        67, 80, 81, 82, 83, 84, 85, 86, 87, 88, 91, 93, 94, 95]])

In [23]:
names_arr = np.array(names)
names_arr[cols]

array(['TTAATTTAATAGA', 'TTAACATAATAAT', 'TGCAATCTCTTTAT',
       'TATTATGTTAATG', 'TACATACCGAT', 'GTGTATCATAAT', 'GCTGTTGAAATGGC',
       'GCAAACATGCG', 'GAGTCCTGTT', 'GAGTCCTGTTT',
       'GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG',
       'CTTTTTCACCTGT', 'CTTGTGAATTTAG', 'CGCCATTATGTT', 'CAGAAAAGCGT',
       'X1_102334_C_T', 'X1_143918_T_C', 'X1_144092_A_G', 'X1_238886_T_C',
       'X1_287706_A_G', 'X1_327716_C_T', 'X1_449085_T_A', 'X1_569187_A_G',
       'X1_655318_C_T', 'X1_655324_T_C', 'X1_681794_G_A', 'X1_698337_T_C',
       'X1_791797_A_T', 'X1_810661_T_C', 'X1_829088_C_T', 'kdpA',
       'group_5991', 'group_3022', 'sdrE_1', 'group_1251', 'group_482',
       'group_6857', 'group_1087', 'group_2822', 'group_1748', 'clpA',
       'group_3572', 'group_3937', 'group_1320', 'group_34', 'dus_1',
       'group_9051', 'group_6900', 'group_7995', 'group_11094',
       'group_7803', 'group_870', 'group_465', 'group_2236', 'group

In [24]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [25]:
df_sel['strain'] = X.iloc[:,0]

In [26]:
df_sel

,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,GAGTCCTGTTT,...,group_7824,group_7845,group_10140,group_10143,group_7172,group_9586,ST,CC,pheno,strain
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,1,107
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,8,8,0,109
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,1,115
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,1,120335
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,1,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,1,SR4152
249,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,3812,5,0,SR4153
250,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,0,SR4155
251,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,0,SR4156


In [27]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 63) (253,) (253, 64)


In [28]:
df_sel['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [29]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [30]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel_over)

In [31]:
dat5 = pd.DataFrame(X_sel_test_over[:,-1])
dat5['test'] = y_sel_test_over

In [32]:
dat5

,0,test
0,NRS245,1
1,NY439,2
2,CA544,1
3,CA541,2
4,EUH15,1
...,...,...
112,NRS112,0
113,CFBRSa51,2
114,NRS383,1
115,NRS247,0


In [33]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [83]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [84]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 464us/step - loss: 30.6432 - accuracy: 0.3259 - val_loss: 37.7811 - val_accuracy: 0.3162
Epoch 2/1000
270/270 [==============================] - 0s 55us/step - loss: 27.9954 - accuracy: 0.3333 - val_loss: 34.3036 - val_accuracy: 0.3162
Epoch 3/1000
270/270 [==============================] - 0s 79us/step - loss: 25.4551 - accuracy: 0.3296 - val_loss: 30.8973 - val_accuracy: 0.3077
Epoch 4/1000
270/270 [==============================] - 0s 73us/step - loss: 22.9558 - accuracy: 0.3259 - val_loss: 27.5251 - val_accuracy: 0.3077
Epoch 5/1000
270/270 [==============================] - 0s 97us/step - loss: 20.5438 - accuracy: 0.3556 - val_loss: 24.2923 - val_accuracy: 0.3846
Epoch 6/1000
270/270 [==============================] - 0s 95us/step - loss: 17.9406 - accuracy: 0.4222 - val_loss: 21.3313 - val_accuracy: 0.3846
Epoch 7/1000
270/270 [==============================] - 0s 108us/step -

270/270 [==============================] - 0s 75us/step - loss: 0.6507 - accuracy: 0.7704 - val_loss: 0.8979 - val_accuracy: 0.6923
Epoch 113/1000
270/270 [==============================] - 0s 104us/step - loss: 0.6193 - accuracy: 0.7519 - val_loss: 0.8992 - val_accuracy: 0.7009
Epoch 114/1000
270/270 [==============================] - 0s 103us/step - loss: 0.7801 - accuracy: 0.7333 - val_loss: 1.2537 - val_accuracy: 0.6496
Epoch 115/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8174 - accuracy: 0.7259 - val_loss: 0.9903 - val_accuracy: 0.7009
Epoch 116/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6770 - accuracy: 0.7481 - val_loss: 0.9236 - val_accuracy: 0.5897
Epoch 117/1000
270/270 [==============================] - 0s 76us/step - loss: 0.6628 - accuracy: 0.7000 - val_loss: 1.0520 - val_accuracy: 0.5641
Epoch 118/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6354 - accuracy: 0.7222 - val_loss: 0.9388 - val_a

Epoch 223/1000
270/270 [==============================] - 0s 58us/step - loss: 0.4774 - accuracy: 0.8259 - val_loss: 0.9425 - val_accuracy: 0.7094
Epoch 224/1000
270/270 [==============================] - 0s 56us/step - loss: 0.5823 - accuracy: 0.7963 - val_loss: 1.1374 - val_accuracy: 0.6496
Epoch 225/1000
270/270 [==============================] - 0s 49us/step - loss: 0.6179 - accuracy: 0.7778 - val_loss: 0.9764 - val_accuracy: 0.7265
Epoch 226/1000
270/270 [==============================] - 0s 55us/step - loss: 0.5206 - accuracy: 0.8074 - val_loss: 0.9622 - val_accuracy: 0.6410
Epoch 227/1000
270/270 [==============================] - 0s 54us/step - loss: 0.5255 - accuracy: 0.7704 - val_loss: 1.1341 - val_accuracy: 0.6154
Epoch 228/1000
270/270 [==============================] - 0s 56us/step - loss: 0.5983 - accuracy: 0.7815 - val_loss: 0.9443 - val_accuracy: 0.7094
Epoch 229/1000
270/270 [==============================] - 0s 60us/step - loss: 0.5026 - accuracy: 0.8111 - val_loss: 0

Epoch 279/1000
270/270 [==============================] - 0s 64us/step - loss: 0.6054 - accuracy: 0.8000 - val_loss: 1.0850 - val_accuracy: 0.6923
Epoch 280/1000
270/270 [==============================] - 0s 54us/step - loss: 0.4669 - accuracy: 0.8296 - val_loss: 1.0832 - val_accuracy: 0.7009
Epoch 281/1000
270/270 [==============================] - 0s 57us/step - loss: 0.5483 - accuracy: 0.8000 - val_loss: 1.0168 - val_accuracy: 0.7094
Epoch 282/1000
270/270 [==============================] - 0s 51us/step - loss: 0.5129 - accuracy: 0.8259 - val_loss: 1.0951 - val_accuracy: 0.7009
Epoch 283/1000
270/270 [==============================] - 0s 58us/step - loss: 0.4923 - accuracy: 0.8333 - val_loss: 0.9878 - val_accuracy: 0.7179
Epoch 284/1000
270/270 [==============================] - 0s 51us/step - loss: 0.4886 - accuracy: 0.8037 - val_loss: 0.9962 - val_accuracy: 0.7350
Epoch 285/1000
270/270 [==============================] - 0s 51us/step - loss: 0.4690 - accuracy: 0.8259 - val_loss: 1

Epoch 335/1000
270/270 [==============================] - 0s 50us/step - loss: 0.4224 - accuracy: 0.8407 - val_loss: 1.0245 - val_accuracy: 0.7436
Epoch 336/1000
270/270 [==============================] - 0s 55us/step - loss: 0.4184 - accuracy: 0.8481 - val_loss: 1.0168 - val_accuracy: 0.7436
Epoch 337/1000
270/270 [==============================] - 0s 52us/step - loss: 0.4174 - accuracy: 0.8593 - val_loss: 1.0101 - val_accuracy: 0.7521
Epoch 338/1000
270/270 [==============================] - 0s 62us/step - loss: 0.4152 - accuracy: 0.8556 - val_loss: 1.0122 - val_accuracy: 0.7521
Epoch 339/1000
270/270 [==============================] - 0s 55us/step - loss: 0.4137 - accuracy: 0.8593 - val_loss: 1.0153 - val_accuracy: 0.7521
Epoch 340/1000
270/270 [==============================] - 0s 72us/step - loss: 0.4127 - accuracy: 0.8593 - val_loss: 1.0241 - val_accuracy: 0.7521
Epoch 341/1000
270/270 [==============================] - 0s 64us/step - loss: 0.4151 - accuracy: 0.8481 - val_loss: 1

...

Epoch 502/1000
270/270 [==============================] - 0s 84us/step - loss: 0.4070 - accuracy: 0.8481 - val_loss: 1.2040 - val_accuracy: 0.7265
Epoch 503/1000
270/270 [==============================] - 0s 68us/step - loss: 0.3820 - accuracy: 0.8630 - val_loss: 1.1562 - val_accuracy: 0.7521
Epoch 504/1000
270/270 [==============================] - 0s 68us/step - loss: 0.3890 - accuracy: 0.8519 - val_loss: 1.1718 - val_accuracy: 0.7436
Epoch 505/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3791 - accuracy: 0.8630 - val_loss: 1.1466 - val_accuracy: 0.7265
Epoch 506/1000
270/270 [==============================] - 0s 508us/step - loss: 0.3650 - accuracy: 0.8630 - val_loss: 1.1393 - val_accuracy: 0.7436
Epoch 507/1000
270/270 [==============================] - 0s 89us/step - loss: 0.3744 - accuracy: 0.8519 - val_loss: 1.1408 - val_accuracy: 0.7265
Epoch 508/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3755 - accuracy: 0.8630 - val_loss: 

Epoch 613/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3488 - accuracy: 0.8778 - val_loss: 1.2087 - val_accuracy: 0.7521
Epoch 614/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3435 - accuracy: 0.8778 - val_loss: 1.2139 - val_accuracy: 0.7521
Epoch 615/1000
270/270 [==============================] - 0s 122us/step - loss: 0.3417 - accuracy: 0.8815 - val_loss: 1.2152 - val_accuracy: 0.7436
Epoch 616/1000
270/270 [==============================] - 0s 94us/step - loss: 0.3459 - accuracy: 0.8741 - val_loss: 1.2173 - val_accuracy: 0.7350
Epoch 617/1000
270/270 [==============================] - 0s 99us/step - loss: 0.3423 - accuracy: 0.8778 - val_loss: 1.2246 - val_accuracy: 0.7436
Epoch 618/1000
270/270 [==============================] - 0s 76us/step - loss: 0.3541 - accuracy: 0.8704 - val_loss: 1.2420 - val_accuracy: 0.7436
Epoch 619/1000
270/270 [==============================] - 0s 124us/step - loss: 0.3478 - accuracy: 0.8741 - val_loss:

Epoch 669/1000
270/270 [==============================] - 0s 52us/step - loss: 0.3374 - accuracy: 0.8815 - val_loss: 1.2991 - val_accuracy: 0.7350
Epoch 670/1000
270/270 [==============================] - 0s 57us/step - loss: 0.3530 - accuracy: 0.8778 - val_loss: 1.2908 - val_accuracy: 0.7436
Epoch 671/1000
270/270 [==============================] - 0s 66us/step - loss: 0.3433 - accuracy: 0.8741 - val_loss: 1.2750 - val_accuracy: 0.7350
Epoch 672/1000
270/270 [==============================] - 0s 56us/step - loss: 0.3549 - accuracy: 0.8593 - val_loss: 1.2727 - val_accuracy: 0.7350
Epoch 673/1000
270/270 [==============================] - 0s 54us/step - loss: 0.3485 - accuracy: 0.8704 - val_loss: 1.2728 - val_accuracy: 0.7350
Epoch 674/1000
270/270 [==============================] - 0s 65us/step - loss: 0.3440 - accuracy: 0.8704 - val_loss: 1.2581 - val_accuracy: 0.7350
Epoch 675/1000
270/270 [==============================] - 0s 55us/step - loss: 0.3400 - accuracy: 0.8630 - val_loss: 1

Epoch 780/1000
270/270 [==============================] - 0s 60us/step - loss: 0.3233 - accuracy: 0.8704 - val_loss: 1.3195 - val_accuracy: 0.7778
Epoch 781/1000
270/270 [==============================] - 0s 74us/step - loss: 0.3225 - accuracy: 0.8815 - val_loss: 1.3246 - val_accuracy: 0.7607
Epoch 782/1000
270/270 [==============================] - 0s 66us/step - loss: 0.3262 - accuracy: 0.8667 - val_loss: 1.3420 - val_accuracy: 0.7350
Epoch 783/1000
270/270 [==============================] - 0s 89us/step - loss: 0.3364 - accuracy: 0.8667 - val_loss: 1.3763 - val_accuracy: 0.7179
Epoch 784/1000
270/270 [==============================] - 0s 90us/step - loss: 0.3534 - accuracy: 0.8593 - val_loss: 1.3330 - val_accuracy: 0.7607
Epoch 785/1000
270/270 [==============================] - 0s 72us/step - loss: 0.3208 - accuracy: 0.8815 - val_loss: 1.3135 - val_accuracy: 0.7436
Epoch 786/1000
270/270 [==============================] - 0s 81us/step - loss: 0.3287 - accuracy: 0.8741 - val_loss: 1

270/270 [==============================] - 0s 172us/step - loss: 0.3245 - accuracy: 0.8741 - val_loss: 1.4045 - val_accuracy: 0.7521
Epoch 891/1000
270/270 [==============================] - 0s 74us/step - loss: 0.3289 - accuracy: 0.8778 - val_loss: 1.3557 - val_accuracy: 0.7521
Epoch 892/1000
270/270 [==============================] - 0s 94us/step - loss: 0.3168 - accuracy: 0.8778 - val_loss: 1.3612 - val_accuracy: 0.7350
Epoch 893/1000
270/270 [==============================] - 0s 84us/step - loss: 0.3116 - accuracy: 0.8815 - val_loss: 1.3724 - val_accuracy: 0.7521
Epoch 894/1000
270/270 [==============================] - 0s 86us/step - loss: 0.3129 - accuracy: 0.8852 - val_loss: 1.3687 - val_accuracy: 0.7436
Epoch 895/1000
270/270 [==============================] - 0s 208us/step - loss: 0.3052 - accuracy: 0.8852 - val_loss: 1.3873 - val_accuracy: 0.7436
Epoch 896/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3078 - accuracy: 0.8852 - val_loss: 1.3808 - val_a

In [164]:
acc_test2_over = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over*100))

117/117 [==============================] - 0s 60us/step
over-sampling test accuracy: 72.65%


In [86]:
pred5 = model2_over.predict_classes(X_sel_test_over)
pred5

array([1, 2, 1, 1, 1, 2, 0, 1, 0, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 1,
       0, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 1, 1,
       1, 2, 2, 0, 2, 1, 0, 2, 2, 0, 1, 2, 0, 1, 1, 0, 1, 2, 1, 2, 0, 1,
       1, 1, 0, 2, 2, 1, 1, 1, 2, 1, 2, 0, 1, 1, 2, 0, 0, 2, 1, 2, 1, 0,
       1, 0, 1, 2, 0, 0, 1, 2, 2, 2, 1, 0, 2, 1, 2, 1, 0, 1, 0, 1, 0, 1,
       2, 0, 0, 1, 0, 0, 0])

In [87]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS245,1,1
1,NY439,2,2
2,CA544,1,1
3,CA541,2,1
4,EUH15,1,1
...,...,...,...
112,NRS112,0,0
113,CFBRSa51,2,1
114,NRS383,1,0
115,NRS247,0,0


In [88]:
proba5 = model2_over.predict_proba(X_sel_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,0.220197,0.444876,3.349269e-01
1,0.016598,0.057210,9.261913e-01
2,0.037655,0.586365,3.759793e-01
3,0.037655,0.586365,3.759793e-01
4,0.229057,0.755724,1.521926e-02
...,...,...,...
112,0.791234,0.207402,1.364317e-03
113,0.037655,0.586365,3.759793e-01
114,0.931316,0.068343,3.413378e-04
115,0.679530,0.318582,1.888539e-03


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p006pST.csv", index = False,
         header=None)

In [168]:
hist2_over = model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 119us/step - loss: 0.2826 - accuracy: 0.8889 - val_loss: 1.5015 - val_accuracy: 0.7350
Epoch 2/1000
270/270 [==============================] - 0s 128us/step - loss: 0.2928 - accuracy: 0.8778 - val_loss: 1.5098 - val_accuracy: 0.7350
Epoch 3/1000
270/270 [==============================] - 0s 146us/step - loss: 0.2922 - accuracy: 0.8778 - val_loss: 1.5693 - val_accuracy: 0.7094
Epoch 4/1000
270/270 [==============================] - 0s 140us/step - loss: 0.4549 - accuracy: 0.8519 - val_loss: 1.4903 - val_accuracy: 0.7009
Epoch 5/1000
270/270 [==============================] - 0s 128us/step - loss: 0.3648 - accuracy: 0.8667 - val_loss: 1.6033 - val_accuracy: 0.7094
Epoch 6/1000
270/270 [==============================] - 0s 150us/step - loss: 0.3266 - accuracy: 0.8741 - val_loss: 1.5216 - val_accuracy: 0.7179
Epoch 7/1000
270/270 [==============================] - 0s 127us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 146us/step - loss: 0.2772 - accuracy: 0.8889 - val_loss: 1.6616 - val_accuracy: 0.7265
Epoch 113/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2678 - accuracy: 0.8963 - val_loss: 1.6605 - val_accuracy: 0.7350
Epoch 114/1000
270/270 [==============================] - 0s 160us/step - loss: 0.2611 - accuracy: 0.8963 - val_loss: 1.6650 - val_accuracy: 0.7265
Epoch 115/1000
270/270 [==============================] - 0s 369us/step - loss: 0.2627 - accuracy: 0.8963 - val_loss: 1.6463 - val_accuracy: 0.7350
Epoch 116/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2575 - accuracy: 0.8926 - val_loss: 1.6484 - val_accuracy: 0.7350
Epoch 117/1000
270/270 [==============================] - 0s 131us/step - loss: 0.2582 - accuracy: 0.9000 - val_loss: 1.6579 - val_accuracy: 0.7350
Epoch 118/1000
270/270 [==============================] - 0s 166us/step - loss: 0.2585 - accuracy: 0.9000 - val_l

270/270 [==============================] - 0s 147us/step - loss: 0.3926 - accuracy: 0.8667 - val_loss: 1.8542 - val_accuracy: 0.7265
Epoch 223/1000
270/270 [==============================] - 0s 170us/step - loss: 0.3079 - accuracy: 0.8778 - val_loss: 1.7877 - val_accuracy: 0.7265
Epoch 224/1000
270/270 [==============================] - 0s 165us/step - loss: 0.3230 - accuracy: 0.8778 - val_loss: 1.8963 - val_accuracy: 0.7265
Epoch 225/1000
270/270 [==============================] - 0s 172us/step - loss: 0.2746 - accuracy: 0.8815 - val_loss: 1.9171 - val_accuracy: 0.6923
Epoch 226/1000
270/270 [==============================] - 0s 156us/step - loss: 0.3031 - accuracy: 0.8704 - val_loss: 1.8316 - val_accuracy: 0.7350
Epoch 227/1000
270/270 [==============================] - 0s 154us/step - loss: 0.2826 - accuracy: 0.8926 - val_loss: 1.8072 - val_accuracy: 0.7265
Epoch 228/1000
270/270 [==============================] - 0s 231us/step - loss: 0.2535 - accuracy: 0.8963 - val_loss: 1.7967 - 

270/270 [==============================] - 0s 388us/step - loss: 0.2700 - accuracy: 0.8852 - val_loss: 1.9540 - val_accuracy: 0.7094
Epoch 333/1000
270/270 [==============================] - 0s 203us/step - loss: 0.2529 - accuracy: 0.8852 - val_loss: 1.9460 - val_accuracy: 0.7179
Epoch 334/1000
270/270 [==============================] - 0s 211us/step - loss: 0.2565 - accuracy: 0.8778 - val_loss: 1.9221 - val_accuracy: 0.7179
Epoch 335/1000
270/270 [==============================] - 0s 182us/step - loss: 0.2600 - accuracy: 0.8926 - val_loss: 1.9368 - val_accuracy: 0.7265
Epoch 336/1000
270/270 [==============================] - 0s 241us/step - loss: 0.2701 - accuracy: 0.8963 - val_loss: 1.9178 - val_accuracy: 0.7179
Epoch 337/1000
270/270 [==============================] - 0s 147us/step - loss: 0.2636 - accuracy: 0.8926 - val_loss: 1.9935 - val_accuracy: 0.7265
Epoch 338/1000
270/270 [==============================] - 0s 192us/step - loss: 0.2913 - accuracy: 0.8889 - val_loss: 1.8954 - 

270/270 [==============================] - 0s 124us/step - loss: 0.3201 - accuracy: 0.8815 - val_loss: 2.0904 - val_accuracy: 0.7350
Epoch 443/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2576 - accuracy: 0.8963 - val_loss: 2.0736 - val_accuracy: 0.7350
Epoch 444/1000
270/270 [==============================] - 0s 128us/step - loss: 0.2935 - accuracy: 0.8630 - val_loss: 2.2764 - val_accuracy: 0.7179
Epoch 445/1000
270/270 [==============================] - 0s 128us/step - loss: 0.4914 - accuracy: 0.8630 - val_loss: 2.0807 - val_accuracy: 0.7350
Epoch 446/1000
270/270 [==============================] - 0s 125us/step - loss: 0.4131 - accuracy: 0.8481 - val_loss: 2.0297 - val_accuracy: 0.7265
Epoch 447/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2717 - accuracy: 0.8926 - val_loss: 2.1039 - val_accuracy: 0.7350
Epoch 448/1000
270/270 [==============================] - 0s 146us/step - loss: 0.2727 - accuracy: 0.8963 - val_loss: 2.0770 - 

Epoch 498/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2476 - accuracy: 0.8926 - val_loss: 2.1318 - val_accuracy: 0.7265
Epoch 499/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3061 - accuracy: 0.8815 - val_loss: 2.0669 - val_accuracy: 0.7265
Epoch 500/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2368 - accuracy: 0.9000 - val_loss: 2.0650 - val_accuracy: 0.7350
Epoch 501/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2319 - accuracy: 0.9000 - val_loss: 2.0728 - val_accuracy: 0.7350
Epoch 502/1000
270/270 [==============================] - 0s 82us/step - loss: 0.2352 - accuracy: 0.8963 - val_loss: 2.0660 - val_accuracy: 0.7350
Epoch 503/1000
270/270 [==============================] - 0s 174us/step - loss: 0.2433 - accuracy: 0.8963 - val_loss: 2.0495 - val_accuracy: 0.7265
Epoch 504/1000
270/270 [==============================] - 0s 172us/step - loss: 0.2324 - accuracy: 0.8963 - val_loss:

270/270 [==============================] - 0s 169us/step - loss: 0.3113 - accuracy: 0.8778 - val_loss: 2.1999 - val_accuracy: 0.7265
Epoch 609/1000
270/270 [==============================] - 0s 163us/step - loss: 0.3892 - accuracy: 0.8667 - val_loss: 2.2779 - val_accuracy: 0.7350
Epoch 610/1000
270/270 [==============================] - 0s 149us/step - loss: 0.3113 - accuracy: 0.8778 - val_loss: 2.2180 - val_accuracy: 0.7179
Epoch 611/1000
270/270 [==============================] - 0s 149us/step - loss: 0.2726 - accuracy: 0.8741 - val_loss: 2.2602 - val_accuracy: 0.7350
Epoch 612/1000
270/270 [==============================] - 0s 206us/step - loss: 0.3040 - accuracy: 0.8963 - val_loss: 2.1734 - val_accuracy: 0.7436
Epoch 613/1000
270/270 [==============================] - 0s 185us/step - loss: 0.2521 - accuracy: 0.8815 - val_loss: 2.2024 - val_accuracy: 0.7265
Epoch 614/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2321 - accuracy: 0.8963 - val_loss: 2.1664 - 

270/270 [==============================] - 0s 256us/step - loss: 0.2233 - accuracy: 0.8963 - val_loss: 2.2549 - val_accuracy: 0.7436
Epoch 719/1000
270/270 [==============================] - 0s 219us/step - loss: 0.2330 - accuracy: 0.9037 - val_loss: 2.2596 - val_accuracy: 0.7521
Epoch 720/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2523 - accuracy: 0.8963 - val_loss: 2.2925 - val_accuracy: 0.7265
Epoch 721/1000
270/270 [==============================] - 0s 96us/step - loss: 0.3065 - accuracy: 0.8741 - val_loss: 2.2542 - val_accuracy: 0.7265
Epoch 722/1000
270/270 [==============================] - 0s 118us/step - loss: 0.2260 - accuracy: 0.8963 - val_loss: 2.3029 - val_accuracy: 0.7436
Epoch 723/1000
270/270 [==============================] - 0s 146us/step - loss: 0.2426 - accuracy: 0.8963 - val_loss: 2.3501 - val_accuracy: 0.7436
Epoch 724/1000
270/270 [==============================] - 0s 187us/step - loss: 0.2912 - accuracy: 0.8926 - val_loss: 2.2506 - v

270/270 [==============================] - 0s 139us/step - loss: 0.2292 - accuracy: 0.8963 - val_loss: 2.3740 - val_accuracy: 0.7436
Epoch 829/1000
270/270 [==============================] - 0s 175us/step - loss: 0.2268 - accuracy: 0.8926 - val_loss: 2.3839 - val_accuracy: 0.7436
Epoch 830/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2542 - accuracy: 0.8889 - val_loss: 2.3826 - val_accuracy: 0.7436
Epoch 831/1000
270/270 [==============================] - 0s 123us/step - loss: 0.2256 - accuracy: 0.8926 - val_loss: 2.3660 - val_accuracy: 0.7436
Epoch 832/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2386 - accuracy: 0.8963 - val_loss: 2.3694 - val_accuracy: 0.7436
Epoch 833/1000
270/270 [==============================] - 0s 109us/step - loss: 0.2226 - accuracy: 0.8963 - val_loss: 2.3981 - val_accuracy: 0.7436
Epoch 834/1000
270/270 [==============================] - 0s 139us/step - loss: 0.4027 - accuracy: 0.8481 - val_loss: 2.7455 - 

270/270 [==============================] - 0s 105us/step - loss: 0.2800 - accuracy: 0.8926 - val_loss: 2.5151 - val_accuracy: 0.7436
Epoch 939/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2359 - accuracy: 0.8815 - val_loss: 2.5483 - val_accuracy: 0.7436
Epoch 940/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2277 - accuracy: 0.9000 - val_loss: 2.5514 - val_accuracy: 0.7350
Epoch 941/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2279 - accuracy: 0.8963 - val_loss: 2.5519 - val_accuracy: 0.7350
Epoch 942/1000
270/270 [==============================] - 0s 141us/step - loss: 0.2241 - accuracy: 0.8963 - val_loss: 2.5264 - val_accuracy: 0.7350
Epoch 943/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2316 - accuracy: 0.8963 - val_loss: 2.5073 - val_accuracy: 0.7436
Epoch 944/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2270 - accuracy: 0.8963 - val_loss: 2.5451 - val

In [169]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over.history['accuracy'])*100))

over-sampling train accuracy: 88.33%


In [34]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [35]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS245,1,2,1.345807e-02,2.164788e-01,7.700630e-01
1,p0006kpresabs_qual,NY439,2,2,2.674153e-02,9.294230e-04,9.723290e-01
2,p0006kpresabs_qual,CA544,1,0,4.147484e-01,3.626331e-01,2.226184e-01
3,p0006kpresabs_qual,CA541,2,0,4.147484e-01,3.626331e-01,2.226184e-01
4,p0006kpresabs_qual,EUH15,1,0,4.147484e-01,3.626331e-01,2.226184e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CA541,1,1,3.723218e-01,6.276781e-01,1.945911e-08
985,p0017Skpresabs_qual,SR4152,1,0,7.372800e-01,2.627200e-01,4.197748e-08
986,p0017Skpresabs_qual,NRS110,2,2,4.194510e-08,7.508231e-09,1.000000e+00
987,p0017Skpresabs_qual,CFBRSa70,0,0,7.372800e-01,2.627200e-01,4.197748e-08


In [36]:
y_prob5 = df_proba5[df_proba5['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[2.20196680e-01, 4.44876430e-01, 3.34926900e-01],
       [1.65982120e-02, 5.72104640e-02, 9.26191300e-01],
       [3.76553100e-02, 5.86365340e-01, 3.75979330e-01],
       [3.76553100e-02, 5.86365340e-01, 3.75979330e-01],
       [2.29057010e-01, 7.55723700e-01, 1.52192560e-02],
       [4.34341240e-03, 1.30369370e-01, 8.65287200e-01],
       [6.65978430e-01, 3.25485000e-01, 8.53646550e-03],
       [4.26822520e-04, 6.20363500e-01, 3.79209730e-01],
       [1.00000000e+00, 2.86358160e-10, 3.71487850e-14],
       [3.11875970e-01, 5.35741450e-01, 1.52382570e-01],
       [1.00000000e+00, 6.06132370e-10, 1.84966770e-14],
       [1.86845520e-05, 4.43183200e-02, 9.55662970e-01],
       [7.97487970e-04, 3.25040730e-03, 9.95952100e-01],
       [4.84200430e-03, 9.94669500e-01, 4.88545400e-04],
       [3.44319410e-04, 3.55005430e-07, 9.99655370e-01],
       [8.38377600e-05, 3.63673160e-02, 9.63548840e-01],
       [1.20127454e-01, 1.11250760e-01, 7.68621740e-01],
       [1.28481050e-03, 1.61363

In [37]:
ovo5 = rocauc_ovo(y_sel_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.8345386806925269

In [38]:
ovr5 = rocauc_ovr(y_sel_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.8345386806925269

In [39]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel_over)

In [40]:
dat6 = pd.DataFrame(X_sel_test_over[:,-1])
dat6['test'] = y_sel_test_over

In [41]:
dat6

,0,test
0,NRS249,2
1,NRS188,1
2,NRS232,2
3,NY439,2
4,GA27,2
...,...,...
112,SR3569,0
113,NRS204,0
114,NRS203,0
115,CFBRSa25,1


In [42]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [43]:
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 618us/step - loss: 40.4823 - accuracy: 0.3519 - val_loss: 33.3464 - val_accuracy: 0.3419
Epoch 2/1000
270/270 [==============================] - 0s 120us/step - loss: 33.9010 - accuracy: 0.3519 - val_loss: 28.7881 - val_accuracy: 0.3504
Epoch 3/1000
270/270 [==============================] - 0s 117us/step - loss: 27.4396 - accuracy: 0.3593 - val_loss: 24.2371 - val_accuracy: 0.4188
Epoch 4/1000
270/270 [==============================] - 0s 153us/step - loss: 21.9314 - accuracy: 0.4259 - val_loss: 19.7920 - val_accuracy: 0.3932
Epoch 5/1000
270/270 [==============================] - 0s 113us/step - loss: 16.2334 - accuracy: 0.3926 - val_loss: 15.8726 - val_accuracy: 0.3590
Epoch 6/1000
270/270 [==============================] - 0s 199us/step - loss: 11.2529 - accuracy: 0.2852 - val_loss: 12.7805 - val_accuracy: 0.3248
Epoch 7/1000
270/270 [==============================] - 0s 94us/st

Epoch 112/1000
270/270 [==============================] - 0s 53us/step - loss: 0.6394 - accuracy: 0.7556 - val_loss: 0.9209 - val_accuracy: 0.6410
Epoch 113/1000
270/270 [==============================] - 0s 61us/step - loss: 0.6093 - accuracy: 0.7630 - val_loss: 0.8606 - val_accuracy: 0.6496
Epoch 114/1000
270/270 [==============================] - 0s 50us/step - loss: 0.6265 - accuracy: 0.7556 - val_loss: 0.9275 - val_accuracy: 0.5983
Epoch 115/1000
270/270 [==============================] - 0s 72us/step - loss: 0.6923 - accuracy: 0.7481 - val_loss: 1.0238 - val_accuracy: 0.6325
Epoch 116/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8460 - accuracy: 0.7222 - val_loss: 2.0329 - val_accuracy: 0.6239
Epoch 117/1000
270/270 [==============================] - 0s 51us/step - loss: 1.1632 - accuracy: 0.7148 - val_loss: 1.8400 - val_accuracy: 0.6325
Epoch 118/1000
270/270 [==============================] - 0s 50us/step - loss: 1.0359 - accuracy: 0.7185 - val_loss: 1

Epoch 223/1000
270/270 [==============================] - ETA: 0s - loss: 0.5756 - accuracy: 0.76 - 0s 49us/step - loss: 0.6140 - accuracy: 0.7963 - val_loss: 1.3580 - val_accuracy: 0.6325
Epoch 224/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7307 - accuracy: 0.7852 - val_loss: 1.4295 - val_accuracy: 0.6667
Epoch 225/1000
270/270 [==============================] - 0s 51us/step - loss: 0.6583 - accuracy: 0.7778 - val_loss: 0.9573 - val_accuracy: 0.5897
Epoch 226/1000
270/270 [==============================] - 0s 54us/step - loss: 0.5973 - accuracy: 0.7741 - val_loss: 1.1460 - val_accuracy: 0.5812
Epoch 227/1000
270/270 [==============================] - 0s 54us/step - loss: 0.6413 - accuracy: 0.7704 - val_loss: 1.0810 - val_accuracy: 0.6325
Epoch 228/1000
270/270 [==============================] - 0s 50us/step - loss: 0.5284 - accuracy: 0.8037 - val_loss: 0.9490 - val_accuracy: 0.6068
Epoch 229/1000
270/270 [==============================] - 0s 53us/step - los

Epoch 334/1000
270/270 [==============================] - 0s 69us/step - loss: 0.4270 - accuracy: 0.8556 - val_loss: 0.9530 - val_accuracy: 0.5983
Epoch 335/1000
270/270 [==============================] - 0s 70us/step - loss: 0.4357 - accuracy: 0.8407 - val_loss: 0.9416 - val_accuracy: 0.6154
Epoch 336/1000
270/270 [==============================] - 0s 72us/step - loss: 0.4259 - accuracy: 0.8407 - val_loss: 0.9983 - val_accuracy: 0.6581
Epoch 337/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4510 - accuracy: 0.8333 - val_loss: 0.9906 - val_accuracy: 0.6325
Epoch 338/1000
270/270 [==============================] - 0s 74us/step - loss: 0.4223 - accuracy: 0.8444 - val_loss: 0.9355 - val_accuracy: 0.5983
Epoch 339/1000
270/270 [==============================] - 0s 87us/step - loss: 0.4221 - accuracy: 0.8519 - val_loss: 0.9362 - val_accuracy: 0.6068
Epoch 340/1000
270/270 [==============================] - 0s 105us/step - loss: 0.4241 - accuracy: 0.8370 - val_loss:

Epoch 390/1000
270/270 [==============================] - 0s 58us/step - loss: 0.3937 - accuracy: 0.8667 - val_loss: 0.9353 - val_accuracy: 0.6667
Epoch 391/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3953 - accuracy: 0.8741 - val_loss: 0.9479 - val_accuracy: 0.6923
Epoch 392/1000
270/270 [==============================] - 0s 60us/step - loss: 0.3902 - accuracy: 0.8741 - val_loss: 0.9284 - val_accuracy: 0.6667
Epoch 393/1000
270/270 [==============================] - 0s 70us/step - loss: 0.3918 - accuracy: 0.8630 - val_loss: 0.9299 - val_accuracy: 0.6496
Epoch 394/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3928 - accuracy: 0.8704 - val_loss: 0.9342 - val_accuracy: 0.6581
Epoch 395/1000
270/270 [==============================] - ETA: 0s - loss: 0.4202 - accuracy: 0.82 - 0s 69us/step - loss: 0.4008 - accuracy: 0.8667 - val_loss: 0.9139 - val_accuracy: 0.6496
Epoch 396/1000
270/270 [==============================] - 0s 79us/step - los

...

Epoch 613/1000
270/270 [==============================] - 0s 49us/step - loss: 0.3478 - accuracy: 0.8926 - val_loss: 1.1216 - val_accuracy: 0.6410
Epoch 614/1000
270/270 [==============================] - 0s 67us/step - loss: 0.3354 - accuracy: 0.8926 - val_loss: 1.0354 - val_accuracy: 0.6154
Epoch 615/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3255 - accuracy: 0.8778 - val_loss: 1.0333 - val_accuracy: 0.6154
Epoch 616/1000
270/270 [==============================] - 0s 72us/step - loss: 0.3329 - accuracy: 0.8815 - val_loss: 1.1206 - val_accuracy: 0.6325
Epoch 617/1000
270/270 [==============================] - 0s 80us/step - loss: 0.3741 - accuracy: 0.8741 - val_loss: 1.1494 - val_accuracy: 0.6239
Epoch 618/1000
270/270 [==============================] - 0s 77us/step - loss: 0.3365 - accuracy: 0.8963 - val_loss: 1.0868 - val_accuracy: 0.6239
Epoch 619/1000
270/270 [==============================] - 0s 83us/step - loss: 0.3452 - accuracy: 0.8778 - val_loss: 1

270/270 [==============================] - 0s 52us/step - loss: 0.4682 - accuracy: 0.8074 - val_loss: 1.5168 - val_accuracy: 0.6068
Epoch 724/1000
270/270 [==============================] - 0s 54us/step - loss: 0.7851 - accuracy: 0.8222 - val_loss: 1.3699 - val_accuracy: 0.5983
Epoch 725/1000
270/270 [==============================] - 0s 51us/step - loss: 0.6181 - accuracy: 0.8185 - val_loss: 1.8665 - val_accuracy: 0.6154
Epoch 726/1000
270/270 [==============================] - 0s 54us/step - loss: 0.5765 - accuracy: 0.8370 - val_loss: 1.3636 - val_accuracy: 0.6154
Epoch 727/1000
270/270 [==============================] - 0s 62us/step - loss: 0.4715 - accuracy: 0.8444 - val_loss: 1.4943 - val_accuracy: 0.5726
Epoch 728/1000
270/270 [==============================] - 0s 64us/step - loss: 0.6498 - accuracy: 0.8333 - val_loss: 1.5802 - val_accuracy: 0.5897
Epoch 729/1000
270/270 [==============================] - 0s 62us/step - loss: 0.4401 - accuracy: 0.8444 - val_loss: 1.2183 - val_acc

Epoch 779/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8793 - accuracy: 0.8333 - val_loss: 2.0517 - val_accuracy: 0.6325
Epoch 780/1000
270/270 [==============================] - 0s 78us/step - loss: 0.6541 - accuracy: 0.8593 - val_loss: 1.4280 - val_accuracy: 0.5983
Epoch 781/1000
270/270 [==============================] - 0s 68us/step - loss: 0.3467 - accuracy: 0.8667 - val_loss: 1.2889 - val_accuracy: 0.5983
Epoch 782/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3632 - accuracy: 0.8593 - val_loss: 1.2420 - val_accuracy: 0.6068
Epoch 783/1000
270/270 [==============================] - 0s 66us/step - loss: 0.3349 - accuracy: 0.8778 - val_loss: 1.2296 - val_accuracy: 0.6239
Epoch 784/1000
270/270 [==============================] - 0s 62us/step - loss: 0.3291 - accuracy: 0.8741 - val_loss: 1.1924 - val_accuracy: 0.6154
Epoch 785/1000
270/270 [==============================] - 0s 67us/step - loss: 0.3072 - accuracy: 0.8889 - val_loss: 1

Epoch 835/1000
270/270 [==============================] - 0s 55us/step - loss: 0.3025 - accuracy: 0.8963 - val_loss: 1.1195 - val_accuracy: 0.6325
Epoch 836/1000
270/270 [==============================] - 0s 60us/step - loss: 0.2911 - accuracy: 0.8963 - val_loss: 1.1318 - val_accuracy: 0.6410
Epoch 837/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2887 - accuracy: 0.9074 - val_loss: 1.1593 - val_accuracy: 0.6581
Epoch 838/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2959 - accuracy: 0.9111 - val_loss: 1.1462 - val_accuracy: 0.6581
Epoch 839/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2884 - accuracy: 0.9074 - val_loss: 1.1347 - val_accuracy: 0.6410
Epoch 840/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2871 - accuracy: 0.8963 - val_loss: 1.1290 - val_accuracy: 0.6410
Epoch 841/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2868 - accuracy: 0.8963 - val_loss: 1

Epoch 891/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2868 - accuracy: 0.9000 - val_loss: 1.2075 - val_accuracy: 0.6410
Epoch 892/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3023 - accuracy: 0.8889 - val_loss: 1.2211 - val_accuracy: 0.6410
Epoch 893/1000
270/270 [==============================] - 0s 79us/step - loss: 0.3022 - accuracy: 0.8889 - val_loss: 1.2147 - val_accuracy: 0.6496
Epoch 894/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2833 - accuracy: 0.9000 - val_loss: 1.1750 - val_accuracy: 0.6410
Epoch 895/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2847 - accuracy: 0.9000 - val_loss: 1.1696 - val_accuracy: 0.6410
Epoch 896/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2839 - accuracy: 0.8963 - val_loss: 1.2070 - val_accuracy: 0.6496
Epoch 897/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2830 - accuracy: 0.9000 - val_loss: 1

In [197]:
acc_test2_over2 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over2*100))

117/117 [==============================] - 0s 74us/step
over-sampling test accuracy: 64.10%


In [99]:
pred6 = model2_over2.predict_classes(X_sel_test_over)
pred6

array([2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1,
       0, 0, 2, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 0,
       0, 2, 0, 1, 2, 2, 1, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 2, 1, 2, 1, 0,
       1, 1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 0, 1, 2, 2, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 2, 2, 0, 1, 2, 1, 1, 0, 1, 2,
       0, 0, 0, 0, 0, 1, 2])

In [100]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS249,2,2
1,NRS188,1,0
2,NRS232,2,2
3,NY439,2,2
4,GA27,2,2
...,...,...,...
112,SR3569,0,0
113,NRS204,0,0
114,NRS203,0,0
115,CFBRSa25,1,1


In [101]:
proba6 = model2_over2.predict_proba(X_sel_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,0.007408,5.862936e-04,0.992006
1,0.954154,4.573350e-02,0.000112
2,0.000035,7.795578e-09,0.999965
3,0.025416,2.911933e-02,0.945464
4,0.004203,1.008085e-01,0.894989
...,...,...,...
112,0.984458,1.260230e-02,0.002940
113,0.976584,2.290322e-02,0.000512
114,0.728446,3.116752e-02,0.240386
115,0.265580,7.110654e-01,0.023355


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p006pST.csv", index = False,
         header=None)

In [201]:
hist2_over2 = model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 71us/step - loss: 0.2804 - accuracy: 0.8889 - val_loss: 1.2217 - val_accuracy: 0.6239
Epoch 2/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2972 - accuracy: 0.8815 - val_loss: 1.1747 - val_accuracy: 0.6154
Epoch 3/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2788 - accuracy: 0.9000 - val_loss: 1.2517 - val_accuracy: 0.6410
Epoch 4/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2846 - accuracy: 0.8963 - val_loss: 1.2142 - val_accuracy: 0.6154
Epoch 5/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2748 - accuracy: 0.9000 - val_loss: 1.2000 - val_accuracy: 0.6239
Epoch 6/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2760 - accuracy: 0.8926 - val_loss: 1.1763 - val_accuracy: 0.6239
Epoch 7/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2692 

Epoch 57/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2652 - accuracy: 0.9074 - val_loss: 1.1607 - val_accuracy: 0.6410
Epoch 58/1000
270/270 [==============================] - 0s 62us/step - loss: 0.2610 - accuracy: 0.9000 - val_loss: 1.1668 - val_accuracy: 0.6410
Epoch 59/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2660 - accuracy: 0.9000 - val_loss: 1.2180 - val_accuracy: 0.6410
Epoch 60/1000
270/270 [==============================] - 0s 59us/step - loss: 0.2655 - accuracy: 0.9000 - val_loss: 1.1736 - val_accuracy: 0.6410
Epoch 61/1000
270/270 [==============================] - 0s 47us/step - loss: 0.2680 - accuracy: 0.8963 - val_loss: 1.1986 - val_accuracy: 0.6410
Epoch 62/1000
270/270 [==============================] - 0s 61us/step - loss: 0.3071 - accuracy: 0.8778 - val_loss: 1.3622 - val_accuracy: 0.6410
Epoch 63/1000
270/270 [==============================] - 0s 70us/step - loss: 0.3040 - accuracy: 0.8889 - val_loss: 1.2096 -

270/270 [==============================] - 0s 58us/step - loss: 0.2603 - accuracy: 0.9074 - val_loss: 1.2178 - val_accuracy: 0.6410
Epoch 114/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2622 - accuracy: 0.9074 - val_loss: 1.1997 - val_accuracy: 0.6410
Epoch 115/1000
270/270 [==============================] - 0s 123us/step - loss: 0.2565 - accuracy: 0.9111 - val_loss: 1.1812 - val_accuracy: 0.6496
Epoch 116/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2557 - accuracy: 0.9074 - val_loss: 1.1909 - val_accuracy: 0.6496
Epoch 117/1000
270/270 [==============================] - ETA: 0s - loss: 0.2485 - accuracy: 0.92 - 0s 120us/step - loss: 0.2613 - accuracy: 0.9074 - val_loss: 1.1983 - val_accuracy: 0.6410
Epoch 118/1000
270/270 [==============================] - 0s 126us/step - loss: 0.2601 - accuracy: 0.9074 - val_loss: 1.2049 - val_accuracy: 0.6496
Epoch 119/1000
270/270 [==============================] - 0s 126us/step - loss: 0.2622 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.131394). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


270/270 [==============================] - 0s 347us/step - loss: 0.2648 - accuracy: 0.8963 - val_loss: 1.2111 - val_accuracy: 0.6496
Epoch 146/1000
270/270 [==============================] - 0s 113us/step - loss: 0.2574 - accuracy: 0.9037 - val_loss: 1.1922 - val_accuracy: 0.6496
Epoch 147/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2718 - accuracy: 0.9037 - val_loss: 1.2529 - val_accuracy: 0.6496
Epoch 148/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2569 - accuracy: 0.9037 - val_loss: 1.2515 - val_accuracy: 0.6325
Epoch 149/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2666 - accuracy: 0.8963 - val_loss: 1.4015 - val_accuracy: 0.6325
Epoch 150/1000
270/270 [==============================] - 0s 86us/step - loss: 0.3216 - accuracy: 0.8815 - val_loss: 1.3213 - val_accuracy: 0.6410
Epoch 151/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2610 - accuracy: 0.9000 - val_loss: 1.2629 - val_a

...

Epoch 534/1000
270/270 [==============================] - 0s 113us/step - loss: 0.3832 - accuracy: 0.8741 - val_loss: 1.8051 - val_accuracy: 0.6410
Epoch 535/1000
270/270 [==============================] - 0s 70us/step - loss: 0.3343 - accuracy: 0.8926 - val_loss: 1.4365 - val_accuracy: 0.6496
Epoch 536/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2901 - accuracy: 0.8852 - val_loss: 1.4866 - val_accuracy: 0.6325
Epoch 537/1000
270/270 [==============================] - 0s 99us/step - loss: 0.3436 - accuracy: 0.8815 - val_loss: 1.6787 - val_accuracy: 0.6239
Epoch 538/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2756 - accuracy: 0.8889 - val_loss: 1.4336 - val_accuracy: 0.6410
Epoch 539/1000
270/270 [==============================] - 0s 68us/step - loss: 0.2538 - accuracy: 0.9037 - val_loss: 1.3982 - val_accuracy: 0.6410
Epoch 540/1000
270/270 [==============================] - 0s 247us/step - loss: 0.2568 - accuracy: 0.9111 - val_loss:

Epoch 645/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2343 - accuracy: 0.9111 - val_loss: 1.3662 - val_accuracy: 0.6667
Epoch 646/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2354 - accuracy: 0.9111 - val_loss: 1.3785 - val_accuracy: 0.6496
Epoch 647/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2288 - accuracy: 0.9148 - val_loss: 1.4072 - val_accuracy: 0.6496
Epoch 648/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2264 - accuracy: 0.9148 - val_loss: 1.3728 - val_accuracy: 0.6496
Epoch 649/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2235 - accuracy: 0.9185 - val_loss: 1.3739 - val_accuracy: 0.6581
Epoch 650/1000
270/270 [==============================] - 0s 69us/step - loss: 0.2289 - accuracy: 0.9111 - val_loss: 1.3922 - val_accuracy: 0.6496
Epoch 651/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2257 - accuracy: 0.9185 - val_loss: 

270/270 [==============================] - 0s 78us/step - loss: 0.2237 - accuracy: 0.9111 - val_loss: 1.4321 - val_accuracy: 0.6325
Epoch 756/1000
270/270 [==============================] - 0s 210us/step - loss: 0.2245 - accuracy: 0.9148 - val_loss: 1.4219 - val_accuracy: 0.6325
Epoch 757/1000
270/270 [==============================] - 0s 122us/step - loss: 0.2220 - accuracy: 0.9148 - val_loss: 1.4059 - val_accuracy: 0.6325
Epoch 758/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2202 - accuracy: 0.9111 - val_loss: 1.3825 - val_accuracy: 0.6410
Epoch 759/1000
270/270 [==============================] - 0s 76us/step - loss: 0.2205 - accuracy: 0.9148 - val_loss: 1.3883 - val_accuracy: 0.6325
Epoch 760/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2203 - accuracy: 0.9111 - val_loss: 1.4049 - val_accuracy: 0.6325
Epoch 761/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2194 - accuracy: 0.9148 - val_loss: 1.4081 - val_a

Epoch 811/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2299 - accuracy: 0.9111 - val_loss: 1.4618 - val_accuracy: 0.6410
Epoch 812/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2298 - accuracy: 0.9148 - val_loss: 1.4516 - val_accuracy: 0.6410
Epoch 813/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2245 - accuracy: 0.9148 - val_loss: 1.4612 - val_accuracy: 0.6496
Epoch 814/1000
270/270 [==============================] - 0s 73us/step - loss: 0.2237 - accuracy: 0.9111 - val_loss: 1.4635 - val_accuracy: 0.6581
Epoch 815/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2238 - accuracy: 0.9148 - val_loss: 1.4407 - val_accuracy: 0.6581
Epoch 816/1000
270/270 [==============================] - 0s 56us/step - loss: 0.2210 - accuracy: 0.9185 - val_loss: 1.4296 - val_accuracy: 0.6410
Epoch 817/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2235 - accuracy: 0.9111 - val_loss: 1

270/270 [==============================] - 0s 73us/step - loss: 0.2281 - accuracy: 0.9111 - val_loss: 1.4848 - val_accuracy: 0.6581
Epoch 922/1000
270/270 [==============================] - 0s 61us/step - loss: 0.2224 - accuracy: 0.9148 - val_loss: 1.4967 - val_accuracy: 0.6581
Epoch 923/1000
270/270 [==============================] - 0s 57us/step - loss: 0.2238 - accuracy: 0.9148 - val_loss: 1.5401 - val_accuracy: 0.6581
Epoch 924/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2263 - accuracy: 0.9111 - val_loss: 1.5287 - val_accuracy: 0.6325
Epoch 925/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2241 - accuracy: 0.9148 - val_loss: 1.5049 - val_accuracy: 0.6410
Epoch 926/1000
270/270 [==============================] - 0s 80us/step - loss: 0.2154 - accuracy: 0.9185 - val_loss: 1.4566 - val_accuracy: 0.6496
Epoch 927/1000
270/270 [==============================] - 0s 79us/step - loss: 0.2176 - accuracy: 0.9185 - val_loss: 1.4617 - val_acc

In [202]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over2.history['accuracy'])*100))

over-sampling train accuracy: 90.72%


In [44]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [45]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS249,2,1,1.888869e-01,5.108038e-01,3.003094e-01
1,p0006kpresabs_qual,NRS188,1,1,1.888869e-01,5.108038e-01,3.003094e-01
2,p0006kpresabs_qual,NRS232,2,2,4.222906e-01,7.029924e-02,5.074101e-01
3,p0006kpresabs_qual,NY439,2,2,3.558408e-04,2.976018e-04,9.993465e-01
4,p0006kpresabs_qual,GA27,2,1,3.940971e-01,4.184215e-01,1.874814e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS252,0,0,7.239556e-01,2.760444e-01,1.176030e-09
985,p0017Skpresabs_qual,SR2852,1,1,1.052276e-07,9.999999e-01,1.101559e-28
986,p0017Skpresabs_qual,NRS108,1,1,1.540350e-17,1.000000e+00,9.011977e-16
987,p0017Skpresabs_qual,NRS202,0,0,6.888959e-01,3.111042e-01,2.228958e-09


In [46]:
y_prob6 = df_proba6[df_proba6['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[7.40805730e-03, 5.86293600e-04, 9.92005650e-01],
       [9.54154300e-01, 4.57334970e-02, 1.12171310e-04],
       [3.51895100e-05, 7.79557800e-09, 9.99964830e-01],
       [2.54162100e-02, 2.91193280e-02, 9.45464500e-01],
       [4.20283570e-03, 1.00808510e-01, 8.94988660e-01],
       [7.40805730e-03, 5.86293600e-04, 9.92005650e-01],
       [7.62119400e-02, 4.61425900e-02, 8.77645430e-01],
       [4.59457070e-01, 1.81036210e-03, 5.38732650e-01],
       [1.98232250e-01, 8.01758770e-01, 8.98703600e-06],
       [2.54162100e-02, 2.91193280e-02, 9.45464500e-01],
       [6.02058880e-02, 6.57663050e-01, 2.82131050e-01],
       [1.21570740e-01, 7.17245160e-01, 1.61184130e-01],
       [2.48006240e-01, 1.00885615e-01, 6.51108150e-01],
       [1.24078990e-03, 2.08422220e-06, 9.98757100e-01],
       [8.36917200e-02, 9.16261550e-01, 4.67285700e-05],
       [1.54871830e-04, 2.22620910e-05, 9.99822900e-01],
       [1.54780390e-01, 8.25319300e-01, 1.99002870e-02],
       [1.45105900e-01, 8.54894

In [47]:
ovo6 = rocauc_ovo(y_sel_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.7965154503616042

In [48]:
ovr6 = rocauc_ovr(y_sel_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.7965154503616042

In [49]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel_over)

In [50]:
dat7 = pd.DataFrame(X_sel_test_over[:,-1])
dat7['test'] = y_sel_test_over

In [51]:
dat7

,0,test
0,NRS210,0
1,NRS205,2
2,312,2
3,GA15,2
4,SR4035,0
...,...,...
112,NRS265,2
113,CFBREBSa108,1
114,NY224,1
115,NRS386,2


In [52]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [53]:
model2_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [110]:
model2_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 834us/step - loss: 10.5304 - accuracy: 0.2667 - val_loss: 11.8254 - val_accuracy: 0.2906
Epoch 2/1000
270/270 [==============================] - 0s 111us/step - loss: 8.7116 - accuracy: 0.3296 - val_loss: 9.4643 - val_accuracy: 0.3248
Epoch 3/1000
270/270 [==============================] - 0s 142us/step - loss: 6.8718 - accuracy: 0.3222 - val_loss: 7.2825 - val_accuracy: 0.3248
Epoch 4/1000
270/270 [==============================] - 0s 96us/step - loss: 5.1380 - accuracy: 0.3148 - val_loss: 5.2124 - val_accuracy: 0.3162
Epoch 5/1000
270/270 [==============================] - 0s 128us/step - loss: 3.4726 - accuracy: 0.3259 - val_loss: 3.4098 - val_accuracy: 0.4103
Epoch 6/1000
270/270 [==============================] - 0s 72us/step - loss: 2.6422 - accuracy: 0.3296 - val_loss: 3.4586 - val_accuracy: 0.3333
Epoch 7/1000
270/270 [==============================] - 0s 111us/step - loss: 

Epoch 57/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8190 - accuracy: 0.7000 - val_loss: 0.7887 - val_accuracy: 0.7094
Epoch 58/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7272 - accuracy: 0.7222 - val_loss: 0.7933 - val_accuracy: 0.6838
Epoch 59/1000
270/270 [==============================] - 0s 73us/step - loss: 0.7303 - accuracy: 0.7037 - val_loss: 0.9177 - val_accuracy: 0.6667
Epoch 60/1000
270/270 [==============================] - 0s 87us/step - loss: 0.7508 - accuracy: 0.7148 - val_loss: 0.7865 - val_accuracy: 0.6667
Epoch 61/1000
270/270 [==============================] - 0s 180us/step - loss: 0.7167 - accuracy: 0.7148 - val_loss: 0.8045 - val_accuracy: 0.6667
Epoch 62/1000
270/270 [==============================] - 0s 122us/step - loss: 0.7058 - accuracy: 0.7222 - val_loss: 0.7887 - val_accuracy: 0.6410
Epoch 63/1000
270/270 [==============================] - 0s 165us/step - loss: 0.7266 - accuracy: 0.7259 - val_loss: 0.779

Epoch 113/1000
270/270 [==============================] - 0s 48us/step - loss: 0.6054 - accuracy: 0.7519 - val_loss: 0.7268 - val_accuracy: 0.7009
Epoch 114/1000
270/270 [==============================] - 0s 53us/step - loss: 0.6759 - accuracy: 0.7407 - val_loss: 0.7646 - val_accuracy: 0.7009
Epoch 115/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6596 - accuracy: 0.7481 - val_loss: 0.7305 - val_accuracy: 0.7009
Epoch 116/1000
270/270 [==============================] - 0s 48us/step - loss: 0.6111 - accuracy: 0.7630 - val_loss: 0.7262 - val_accuracy: 0.7265
Epoch 117/1000
270/270 [==============================] - 0s 51us/step - loss: 0.6025 - accuracy: 0.7630 - val_loss: 0.7144 - val_accuracy: 0.7179
Epoch 118/1000
270/270 [==============================] - 0s 53us/step - loss: 0.6468 - accuracy: 0.7481 - val_loss: 0.7076 - val_accuracy: 0.7179
Epoch 119/1000
270/270 [==============================] - 0s 48us/step - loss: 0.6129 - accuracy: 0.7481 - val_loss: 0

Epoch 169/1000
270/270 [==============================] - 0s 63us/step - loss: 0.5595 - accuracy: 0.7667 - val_loss: 0.7293 - val_accuracy: 0.7179
Epoch 170/1000
270/270 [==============================] - 0s 61us/step - loss: 0.5515 - accuracy: 0.7704 - val_loss: 0.6645 - val_accuracy: 0.7436
Epoch 171/1000
270/270 [==============================] - 0s 72us/step - loss: 0.5536 - accuracy: 0.7778 - val_loss: 0.6743 - val_accuracy: 0.7436
Epoch 172/1000
270/270 [==============================] - 0s 64us/step - loss: 0.5736 - accuracy: 0.7667 - val_loss: 0.6795 - val_accuracy: 0.7094
Epoch 173/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6978 - accuracy: 0.7593 - val_loss: 0.7548 - val_accuracy: 0.7009
Epoch 174/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6232 - accuracy: 0.7519 - val_loss: 0.7108 - val_accuracy: 0.7350
Epoch 175/1000
270/270 [==============================] - 0s 79us/step - loss: 0.5519 - accuracy: 0.7667 - val_loss: 0

Epoch 225/1000
270/270 [==============================] - 0s 70us/step - loss: 0.5380 - accuracy: 0.7815 - val_loss: 0.7034 - val_accuracy: 0.7436
Epoch 226/1000
270/270 [==============================] - 0s 55us/step - loss: 0.5101 - accuracy: 0.7926 - val_loss: 0.6468 - val_accuracy: 0.7521
Epoch 227/1000
270/270 [==============================] - 0s 77us/step - loss: 0.5253 - accuracy: 0.8000 - val_loss: 0.6601 - val_accuracy: 0.7521
Epoch 228/1000
270/270 [==============================] - 0s 90us/step - loss: 0.5167 - accuracy: 0.7815 - val_loss: 0.6879 - val_accuracy: 0.7350
Epoch 229/1000
270/270 [==============================] - 0s 54us/step - loss: 0.5198 - accuracy: 0.7926 - val_loss: 0.6654 - val_accuracy: 0.7436
Epoch 230/1000
270/270 [==============================] - 0s 55us/step - loss: 0.5520 - accuracy: 0.7778 - val_loss: 0.6566 - val_accuracy: 0.7265
Epoch 231/1000
270/270 [==============================] - 0s 57us/step - loss: 0.5022 - accuracy: 0.7963 - val_loss: 0

...

Epoch 728/1000
270/270 [==============================] - 0s 65us/step - loss: 0.3516 - accuracy: 0.8704 - val_loss: 0.6589 - val_accuracy: 0.7778
Epoch 729/1000
270/270 [==============================] - 0s 66us/step - loss: 0.4612 - accuracy: 0.8296 - val_loss: 0.8072 - val_accuracy: 0.7607
Epoch 730/1000
270/270 [==============================] - 0s 64us/step - loss: 0.5690 - accuracy: 0.8444 - val_loss: 0.7168 - val_accuracy: 0.7436
Epoch 731/1000
270/270 [==============================] - 0s 72us/step - loss: 0.3791 - accuracy: 0.8593 - val_loss: 0.8380 - val_accuracy: 0.7607
Epoch 732/1000
270/270 [==============================] - 0s 70us/step - loss: 0.4169 - accuracy: 0.8630 - val_loss: 0.6579 - val_accuracy: 0.7692
Epoch 733/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3743 - accuracy: 0.8407 - val_loss: 0.6580 - val_accuracy: 0.7863
Epoch 734/1000
270/270 [==============================] - 0s 68us/step - loss: 0.3453 - accuracy: 0.8704 - val_loss: 0

Epoch 784/1000
270/270 [==============================] - 0s 74us/step - loss: 0.3878 - accuracy: 0.8667 - val_loss: 0.7308 - val_accuracy: 0.7607
Epoch 785/1000
270/270 [==============================] - 0s 71us/step - loss: 0.4170 - accuracy: 0.8519 - val_loss: 0.6838 - val_accuracy: 0.8120
Epoch 786/1000
270/270 [==============================] - 0s 76us/step - loss: 0.3576 - accuracy: 0.8519 - val_loss: 0.6819 - val_accuracy: 0.8034
Epoch 787/1000
270/270 [==============================] - 0s 75us/step - loss: 0.3391 - accuracy: 0.8704 - val_loss: 0.6613 - val_accuracy: 0.7692
Epoch 788/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3643 - accuracy: 0.8556 - val_loss: 0.6602 - val_accuracy: 0.7863
Epoch 789/1000
270/270 [==============================] - 0s 93us/step - loss: 0.3277 - accuracy: 0.8741 - val_loss: 0.7235 - val_accuracy: 0.7863
Epoch 790/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3457 - accuracy: 0.8630 - val_loss: 0

Epoch 840/1000
270/270 [==============================] - 0s 62us/step - loss: 0.3175 - accuracy: 0.8741 - val_loss: 0.7086 - val_accuracy: 0.8205
Epoch 841/1000
270/270 [==============================] - 0s 62us/step - loss: 0.3199 - accuracy: 0.8704 - val_loss: 0.6918 - val_accuracy: 0.8034
Epoch 842/1000
270/270 [==============================] - 0s 78us/step - loss: 0.3310 - accuracy: 0.8667 - val_loss: 0.6842 - val_accuracy: 0.7778
Epoch 843/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3453 - accuracy: 0.8630 - val_loss: 0.6846 - val_accuracy: 0.7778
Epoch 844/1000
270/270 [==============================] - 0s 83us/step - loss: 0.3228 - accuracy: 0.8815 - val_loss: 0.7521 - val_accuracy: 0.7863
Epoch 845/1000
270/270 [==============================] - 0s 112us/step - loss: 0.4096 - accuracy: 0.8630 - val_loss: 0.7277 - val_accuracy: 0.7436
Epoch 846/1000
270/270 [==============================] - 0s 77us/step - loss: 0.4479 - accuracy: 0.8259 - val_loss: 

Epoch 896/1000
270/270 [==============================] - 0s 96us/step - loss: 0.3240 - accuracy: 0.8704 - val_loss: 0.7226 - val_accuracy: 0.8120
Epoch 897/1000
270/270 [==============================] - 0s 81us/step - loss: 0.3196 - accuracy: 0.8778 - val_loss: 0.7490 - val_accuracy: 0.7863
Epoch 898/1000
270/270 [==============================] - 0s 85us/step - loss: 0.3306 - accuracy: 0.8667 - val_loss: 0.7369 - val_accuracy: 0.7607
Epoch 899/1000
270/270 [==============================] - 0s 90us/step - loss: 0.3459 - accuracy: 0.8593 - val_loss: 0.7266 - val_accuracy: 0.7863
Epoch 900/1000
270/270 [==============================] - 0s 101us/step - loss: 0.3149 - accuracy: 0.8778 - val_loss: 0.7101 - val_accuracy: 0.8120
Epoch 901/1000
270/270 [==============================] - 0s 100us/step - loss: 0.3087 - accuracy: 0.8815 - val_loss: 0.7034 - val_accuracy: 0.8120
Epoch 902/1000
270/270 [==============================] - 0s 90us/step - loss: 0.3185 - accuracy: 0.8852 - val_loss:

Epoch 952/1000
270/270 [==============================] - 0s 62us/step - loss: 0.3000 - accuracy: 0.8889 - val_loss: 0.7296 - val_accuracy: 0.8034
Epoch 953/1000
270/270 [==============================] - 0s 65us/step - loss: 0.3048 - accuracy: 0.8852 - val_loss: 0.7217 - val_accuracy: 0.7949
Epoch 954/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3031 - accuracy: 0.8815 - val_loss: 0.7377 - val_accuracy: 0.7949
Epoch 955/1000
270/270 [==============================] - 0s 61us/step - loss: 0.3011 - accuracy: 0.8815 - val_loss: 0.7359 - val_accuracy: 0.8120
Epoch 956/1000
270/270 [==============================] - 0s 63us/step - loss: 0.3041 - accuracy: 0.8815 - val_loss: 0.7267 - val_accuracy: 0.7949
Epoch 957/1000
270/270 [==============================] - 0s 63us/step - loss: 0.3053 - accuracy: 0.8815 - val_loss: 0.7230 - val_accuracy: 0.7949
Epoch 958/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2976 - accuracy: 0.8889 - val_loss: 0

In [222]:
acc_test2_over3 = model2_over3.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over3*100))

117/117 [==============================] - 0s 79us/step
over-sampling test accuracy: 81.20%


In [112]:
pred7 = model2_over3.predict_classes(X_sel_test_over)
pred7

array([0, 2, 2, 2, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 2, 2, 0, 0,
       2, 2, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 1, 2, 2,
       1, 1, 1, 2, 0, 2, 0, 0, 1, 1, 1, 0, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1,
       2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 2, 2, 0, 1, 0, 0, 0, 1, 0,
       2, 1, 0, 2, 2, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 0, 1, 2, 2, 2, 2, 2,
       2, 0, 2, 0, 1, 2, 2])

In [113]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS210,0,0
1,NRS205,2,2
2,312,2,2
3,GA15,2,2
4,SR4035,0,0
...,...,...,...
112,NRS265,2,2
113,CFBREBSa108,1,0
114,NY224,1,1
115,NRS386,2,2


In [114]:
proba7 = model2_over3.predict_proba(X_sel_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,0.800126,1.996603e-01,2.131956e-04
1,0.000009,5.095759e-05,9.999399e-01
2,0.000048,4.146676e-05,9.999100e-01
3,0.123158,2.964625e-03,8.738779e-01
4,1.000000,2.883267e-11,6.108685e-15
...,...,...,...
112,0.000532,1.998170e-02,9.794863e-01
113,0.614736,3.797406e-01,5.523312e-03
114,0.049790,7.340332e-01,2.161765e-01
115,0.000373,1.516442e-01,8.479825e-01


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p006pST.csv", index = False,
         header=None)

In [226]:
hist2_over3 = model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 71us/step - loss: 0.3264 - accuracy: 0.8704 - val_loss: 0.5788 - val_accuracy: 0.8291
Epoch 2/1000
270/270 [==============================] - 0s 67us/step - loss: 0.3242 - accuracy: 0.8667 - val_loss: 0.5829 - val_accuracy: 0.8291
Epoch 3/1000
270/270 [==============================] - 0s 63us/step - loss: 0.3206 - accuracy: 0.8704 - val_loss: 0.6067 - val_accuracy: 0.8120
Epoch 4/1000
270/270 [==============================] - 0s 94us/step - loss: 0.3138 - accuracy: 0.8667 - val_loss: 0.6516 - val_accuracy: 0.8120
Epoch 5/1000
270/270 [==============================] - 0s 102us/step - loss: 0.3226 - accuracy: 0.8704 - val_loss: 0.6340 - val_accuracy: 0.8120
Epoch 6/1000
270/270 [==============================] - 0s 96us/step - loss: 0.3242 - accuracy: 0.8704 - val_loss: 0.6055 - val_accuracy: 0.8120
Epoch 7/1000
270/270 [==============================] - 0s 79us/step - loss: 0.3240

Epoch 112/1000
270/270 [==============================] - 0s 64us/step - loss: 0.3362 - accuracy: 0.8741 - val_loss: 0.6621 - val_accuracy: 0.7863
Epoch 113/1000
270/270 [==============================] - 0s 61us/step - loss: 0.3496 - accuracy: 0.8556 - val_loss: 0.6332 - val_accuracy: 0.8034
Epoch 114/1000
270/270 [==============================] - 0s 68us/step - loss: 0.3235 - accuracy: 0.8704 - val_loss: 0.6146 - val_accuracy: 0.7949
Epoch 115/1000
270/270 [==============================] - 0s 61us/step - loss: 0.3024 - accuracy: 0.8815 - val_loss: 0.6110 - val_accuracy: 0.8120
Epoch 116/1000
270/270 [==============================] - 0s 66us/step - loss: 0.3180 - accuracy: 0.8741 - val_loss: 0.6064 - val_accuracy: 0.8120
Epoch 117/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3076 - accuracy: 0.8778 - val_loss: 0.6107 - val_accuracy: 0.8120
Epoch 118/1000
270/270 [==============================] - 0s 62us/step - loss: 0.3017 - accuracy: 0.8704 - val_loss: 0

Epoch 223/1000
270/270 [==============================] - 0s 113us/step - loss: 0.3014 - accuracy: 0.8704 - val_loss: 0.6217 - val_accuracy: 0.8034
Epoch 224/1000
270/270 [==============================] - 0s 127us/step - loss: 0.3233 - accuracy: 0.8815 - val_loss: 0.6329 - val_accuracy: 0.8034
Epoch 225/1000
270/270 [==============================] - 0s 138us/step - loss: 0.2971 - accuracy: 0.8704 - val_loss: 0.6431 - val_accuracy: 0.8034
Epoch 226/1000
270/270 [==============================] - 0s 129us/step - loss: 0.3336 - accuracy: 0.8704 - val_loss: 0.6737 - val_accuracy: 0.7863
Epoch 227/1000
270/270 [==============================] - 0s 120us/step - loss: 0.3337 - accuracy: 0.8630 - val_loss: 0.6887 - val_accuracy: 0.7692
Epoch 228/1000
270/270 [==============================] - 0s 214us/step - loss: 0.3367 - accuracy: 0.8741 - val_loss: 0.6243 - val_accuracy: 0.7949
Epoch 229/1000
270/270 [==============================] - 0s 208us/step - loss: 0.3271 - accuracy: 0.8704 - val_

Epoch 334/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2801 - accuracy: 0.8778 - val_loss: 0.6364 - val_accuracy: 0.8034
Epoch 335/1000
270/270 [==============================] - 0s 134us/step - loss: 0.2853 - accuracy: 0.8741 - val_loss: 0.6428 - val_accuracy: 0.8120
Epoch 336/1000
270/270 [==============================] - 0s 363us/step - loss: 0.2838 - accuracy: 0.8815 - val_loss: 0.6471 - val_accuracy: 0.8120
Epoch 337/1000
270/270 [==============================] - 0s 241us/step - loss: 0.2859 - accuracy: 0.8778 - val_loss: 0.6377 - val_accuracy: 0.8120
Epoch 338/1000
270/270 [==============================] - 0s 78us/step - loss: 0.3489 - accuracy: 0.8704 - val_loss: 0.6737 - val_accuracy: 0.7949
Epoch 339/1000
270/270 [==============================] - 0s 75us/step - loss: 0.3535 - accuracy: 0.8704 - val_loss: 0.6505 - val_accuracy: 0.7863
Epoch 340/1000
270/270 [==============================] - 0s 81us/step - loss: 0.2956 - accuracy: 0.8630 - val_loss

270/270 [==============================] - 0s 87us/step - loss: 0.2765 - accuracy: 0.8778 - val_loss: 0.6693 - val_accuracy: 0.8034
Epoch 445/1000
270/270 [==============================] - 0s 66us/step - loss: 0.2832 - accuracy: 0.8741 - val_loss: 0.6844 - val_accuracy: 0.8120
Epoch 446/1000
270/270 [==============================] - 0s 63us/step - loss: 0.3198 - accuracy: 0.8778 - val_loss: 0.6810 - val_accuracy: 0.7949
Epoch 447/1000
270/270 [==============================] - 0s 62us/step - loss: 0.3243 - accuracy: 0.8741 - val_loss: 0.8727 - val_accuracy: 0.7949
Epoch 448/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3954 - accuracy: 0.8704 - val_loss: 0.6882 - val_accuracy: 0.7863
Epoch 449/1000
270/270 [==============================] - 0s 65us/step - loss: 0.3894 - accuracy: 0.8519 - val_loss: 0.6467 - val_accuracy: 0.7949
Epoch 450/1000
270/270 [==============================] - 0s 54us/step - loss: 0.2763 - accuracy: 0.8926 - val_loss: 0.9456 - val_acc

...

270/270 [==============================] - 0s 189us/step - loss: 0.2573 - accuracy: 0.8852 - val_loss: 0.6882 - val_accuracy: 0.8034
Epoch 611/1000
270/270 [==============================] - 0s 136us/step - loss: 0.2604 - accuracy: 0.8926 - val_loss: 0.6838 - val_accuracy: 0.8034
Epoch 612/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2587 - accuracy: 0.8926 - val_loss: 0.6964 - val_accuracy: 0.8034
Epoch 613/1000
270/270 [==============================] - 0s 53us/step - loss: 0.2606 - accuracy: 0.8852 - val_loss: 0.6960 - val_accuracy: 0.8034
Epoch 614/1000
270/270 [==============================] - 0s 65us/step - loss: 0.2584 - accuracy: 0.8926 - val_loss: 0.6865 - val_accuracy: 0.8034
Epoch 615/1000
270/270 [==============================] - 0s 51us/step - loss: 0.2573 - accuracy: 0.8852 - val_loss: 0.6769 - val_accuracy: 0.8034
Epoch 616/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2595 - accuracy: 0.8889 - val_loss: 0.6727 - val_a

Epoch 666/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2762 - accuracy: 0.8852 - val_loss: 0.6637 - val_accuracy: 0.8205
Epoch 667/1000
270/270 [==============================] - 0s 58us/step - loss: 0.2727 - accuracy: 0.8852 - val_loss: 0.7207 - val_accuracy: 0.7949
Epoch 668/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8365 - accuracy: 0.8519 - val_loss: 1.3615 - val_accuracy: 0.7692
Epoch 669/1000
270/270 [==============================] - 0s 82us/step - loss: 1.2239 - accuracy: 0.8370 - val_loss: 1.1926 - val_accuracy: 0.7692
Epoch 670/1000
270/270 [==============================] - 0s 144us/step - loss: 0.7504 - accuracy: 0.8444 - val_loss: 0.9048 - val_accuracy: 0.7692
Epoch 671/1000
270/270 [==============================] - 0s 92us/step - loss: 0.6715 - accuracy: 0.8370 - val_loss: 0.6708 - val_accuracy: 0.8291
Epoch 672/1000
270/270 [==============================] - 0s 69us/step - loss: 0.3333 - accuracy: 0.8667 - val_loss: 

Epoch 722/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2517 - accuracy: 0.8926 - val_loss: 0.7112 - val_accuracy: 0.7863
Epoch 723/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2512 - accuracy: 0.8926 - val_loss: 0.6989 - val_accuracy: 0.8034
Epoch 724/1000
270/270 [==============================] - 0s 74us/step - loss: 0.2535 - accuracy: 0.8963 - val_loss: 0.6854 - val_accuracy: 0.7863
Epoch 725/1000
270/270 [==============================] - 0s 167us/step - loss: 0.2487 - accuracy: 0.8963 - val_loss: 0.6918 - val_accuracy: 0.8034
Epoch 726/1000
270/270 [==============================] - 0s 168us/step - loss: 0.2562 - accuracy: 0.8741 - val_loss: 0.7070 - val_accuracy: 0.8034
Epoch 727/1000
270/270 [==============================] - 0s 75us/step - loss: 0.2518 - accuracy: 0.8926 - val_loss: 0.7171 - val_accuracy: 0.7949
Epoch 728/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2543 - accuracy: 0.8926 - val_loss

270/270 [==============================] - 0s 61us/step - loss: 0.2436 - accuracy: 0.9000 - val_loss: 0.7478 - val_accuracy: 0.8120
Epoch 833/1000
270/270 [==============================] - 0s 78us/step - loss: 0.2538 - accuracy: 0.8852 - val_loss: 0.7193 - val_accuracy: 0.8205
Epoch 834/1000
270/270 [==============================] - 0s 63us/step - loss: 0.2541 - accuracy: 0.8963 - val_loss: 0.7202 - val_accuracy: 0.8205
Epoch 835/1000
270/270 [==============================] - 0s 72us/step - loss: 0.3241 - accuracy: 0.8926 - val_loss: 0.8159 - val_accuracy: 0.7863
Epoch 836/1000
270/270 [==============================] - 0s 67us/step - loss: 0.2727 - accuracy: 0.8889 - val_loss: 0.7892 - val_accuracy: 0.7949
Epoch 837/1000
270/270 [==============================] - 0s 73us/step - loss: 0.3276 - accuracy: 0.8778 - val_loss: 0.6930 - val_accuracy: 0.7949
Epoch 838/1000
270/270 [==============================] - 0s 64us/step - loss: 0.2708 - accuracy: 0.8926 - val_loss: 0.7147 - val_acc

Epoch 943/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2425 - accuracy: 0.8926 - val_loss: 0.7289 - val_accuracy: 0.8205
Epoch 944/1000
270/270 [==============================] - 0s 100us/step - loss: 0.2405 - accuracy: 0.9000 - val_loss: 0.7610 - val_accuracy: 0.8034
Epoch 945/1000
270/270 [==============================] - 0s 110us/step - loss: 0.2395 - accuracy: 0.8963 - val_loss: 0.7535 - val_accuracy: 0.8034
Epoch 946/1000
270/270 [==============================] - 0s 108us/step - loss: 0.2432 - accuracy: 0.9000 - val_loss: 0.7696 - val_accuracy: 0.7949
Epoch 947/1000
270/270 [==============================] - 0s 115us/step - loss: 0.2376 - accuracy: 0.9000 - val_loss: 0.7356 - val_accuracy: 0.8205
Epoch 948/1000
270/270 [==============================] - 0s 72us/step - loss: 0.2464 - accuracy: 0.8963 - val_loss: 0.7294 - val_accuracy: 0.8034
Epoch 949/1000
270/270 [==============================] - 0s 77us/step - loss: 0.2413 - accuracy: 0.9000 - val_lo

In [227]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over3.history['accuracy'])*100))

over-sampling train accuracy: 87.77%


In [54]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [55]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS210,0,0,6.132076e-01,2.812180e-01,1.055744e-01
1,p0006kpresabs_qual,NRS205,2,2,1.993202e-04,6.834937e-07,9.998000e-01
2,p0006kpresabs_qual,312,2,1,3.589463e-01,3.982787e-01,2.427750e-01
3,p0006kpresabs_qual,GA15,2,1,3.589463e-01,3.982787e-01,2.427750e-01
4,p0006kpresabs_qual,SR4035,0,1,3.589463e-01,3.982787e-01,2.427750e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS383,1,0,5.477194e-01,4.522807e-01,1.761374e-08
985,p0017Skpresabs_qual,NRS218,1,1,6.953657e-05,9.999305e-01,3.132419e-10
986,p0017Skpresabs_qual,NRS209,2,2,2.713214e-09,6.656316e-09,1.000000e+00
987,p0017Skpresabs_qual,SR2852,1,1,9.956684e-12,1.000000e+00,7.441288e-26


In [56]:
y_prob7 = df_proba7[df_proba7['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[8.0012650e-01, 1.9966029e-01, 2.1319556e-04],
       [9.1607270e-06, 5.0957595e-05, 9.9993990e-01],
       [4.8487780e-05, 4.1466756e-05, 9.9991000e-01],
       [1.2315752e-01, 2.9646250e-03, 8.7387790e-01],
       [1.0000000e+00, 2.8832665e-11, 6.1086847e-15],
       [2.4728544e-02, 4.2997734e-03, 9.7097170e-01],
       [9.9943510e-01, 1.8774886e-05, 5.4607884e-04],
       [4.4669630e-01, 5.5093290e-01, 2.3707827e-03],
       [9.9999810e-01, 1.9248296e-06, 1.8591079e-14],
       [9.4720674e-01, 5.2792820e-02, 4.7561224e-07],
       [4.5006310e-02, 9.5192030e-01, 3.0734178e-03],
       [9.9504770e-01, 4.9414970e-03, 1.0786558e-05],
       [9.9994100e-01, 5.9044523e-05, 1.1313044e-11],
       [9.0729684e-01, 8.6046790e-02, 6.6563976e-03],
       [4.5006310e-02, 9.5192030e-01, 3.0734178e-03],
       [2.9479584e-01, 6.8180860e-01, 2.3395572e-02],
       [7.7497900e-05, 2.1593415e-03, 9.9776316e-01],
       [6.2537676e-01, 3.3334124e-01, 4.1281990e-02],
       [8.8690040e-02, 7.115

In [57]:
ovo7 = rocauc_ovo(y_sel_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.903791365329827

In [58]:
ovr7 = rocauc_ovr(y_sel_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.903791365329827

In [59]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel_over)

In [60]:
dat8 = pd.DataFrame(X_sel_test_over[:,-1])
dat8['test'] = y_sel_test_over

In [61]:
dat8

,0,test
0,NRS236,1
1,NRS113,2
2,CFBRSa23,0
3,NRS249,2
4,107,1
...,...,...
112,NY439,2
113,NRS106,0
114,221,0
115,NRS386,2


In [62]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [104]:
model2_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [123]:
model2_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 438us/step - loss: 5.8467 - accuracy: 0.3852 - val_loss: 3.8121 - val_accuracy: 0.4359
Epoch 2/1000
270/270 [==============================] - 0s 170us/step - loss: 3.5348 - accuracy: 0.3370 - val_loss: 2.3722 - val_accuracy: 0.3162
Epoch 3/1000
270/270 [==============================] - 0s 361us/step - loss: 2.0852 - accuracy: 0.3741 - val_loss: 1.2614 - val_accuracy: 0.3675
Epoch 4/1000
270/270 [==============================] - 0s 378us/step - loss: 1.2897 - accuracy: 0.4074 - val_loss: 1.1647 - val_accuracy: 0.3761
Epoch 5/1000
270/270 [==============================] - 0s 420us/step - loss: 1.1178 - accuracy: 0.4963 - val_loss: 1.1515 - val_accuracy: 0.5043
Epoch 6/1000
270/270 [==============================] - 0s 188us/step - loss: 1.0336 - accuracy: 0.5074 - val_loss: 1.0236 - val_accuracy: 0.5299
Epoch 7/1000
270/270 [==============================] - 0s 147us/step - loss: 

270/270 [==============================] - 0s 105us/step - loss: 0.6458 - accuracy: 0.7593 - val_loss: 0.9313 - val_accuracy: 0.6410
Epoch 113/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6555 - accuracy: 0.7667 - val_loss: 0.8827 - val_accuracy: 0.6410
Epoch 114/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6200 - accuracy: 0.7778 - val_loss: 0.7375 - val_accuracy: 0.6581
Epoch 115/1000
270/270 [==============================] - 0s 72us/step - loss: 0.5385 - accuracy: 0.7815 - val_loss: 0.7811 - val_accuracy: 0.6239
Epoch 116/1000
270/270 [==============================] - 0s 83us/step - loss: 0.5494 - accuracy: 0.7926 - val_loss: 0.7513 - val_accuracy: 0.6667
Epoch 117/1000
270/270 [==============================] - 0s 72us/step - loss: 0.5079 - accuracy: 0.8037 - val_loss: 0.7392 - val_accuracy: 0.6325
Epoch 118/1000
270/270 [==============================] - 0s 71us/step - loss: 0.5186 - accuracy: 0.8000 - val_loss: 0.7643 - val_ac

Epoch 168/1000
270/270 [==============================] - 0s 85us/step - loss: 0.4580 - accuracy: 0.8259 - val_loss: 0.7191 - val_accuracy: 0.6667
Epoch 169/1000
270/270 [==============================] - 0s 81us/step - loss: 0.4792 - accuracy: 0.8148 - val_loss: 0.7090 - val_accuracy: 0.7009
Epoch 170/1000
270/270 [==============================] - 0s 97us/step - loss: 0.5820 - accuracy: 0.7889 - val_loss: 0.7994 - val_accuracy: 0.6410
Epoch 171/1000
270/270 [==============================] - 0s 73us/step - loss: 0.5140 - accuracy: 0.7889 - val_loss: 0.7195 - val_accuracy: 0.6496
Epoch 172/1000
270/270 [==============================] - 0s 86us/step - loss: 0.4719 - accuracy: 0.7963 - val_loss: 0.7460 - val_accuracy: 0.6752
Epoch 173/1000
270/270 [==============================] - 0s 179us/step - loss: 0.5329 - accuracy: 0.8037 - val_loss: 0.9444 - val_accuracy: 0.6496
Epoch 174/1000
270/270 [==============================] - 0s 79us/step - loss: 0.5610 - accuracy: 0.8000 - val_loss: 

270/270 [==============================] - 0s 105us/step - loss: 0.4073 - accuracy: 0.8370 - val_loss: 0.7710 - val_accuracy: 0.6923
Epoch 279/1000
270/270 [==============================] - 0s 121us/step - loss: 0.4354 - accuracy: 0.8333 - val_loss: 0.7842 - val_accuracy: 0.6923
Epoch 280/1000
270/270 [==============================] - 0s 106us/step - loss: 0.6222 - accuracy: 0.8148 - val_loss: 0.9017 - val_accuracy: 0.6923
Epoch 281/1000
270/270 [==============================] - 0s 110us/step - loss: 0.5796 - accuracy: 0.8296 - val_loss: 0.7183 - val_accuracy: 0.6667
Epoch 282/1000
270/270 [==============================] - 0s 90us/step - loss: 0.6492 - accuracy: 0.8222 - val_loss: 0.7203 - val_accuracy: 0.7009
Epoch 283/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4616 - accuracy: 0.8074 - val_loss: 0.7100 - val_accuracy: 0.7179
Epoch 284/1000
270/270 [==============================] - 0s 112us/step - loss: 0.4723 - accuracy: 0.8333 - val_loss: 0.7152 - v

Epoch 334/1000
270/270 [==============================] - 0s 87us/step - loss: 0.4098 - accuracy: 0.8333 - val_loss: 0.7279 - val_accuracy: 0.6752
Epoch 335/1000
270/270 [==============================] - 0s 70us/step - loss: 0.3850 - accuracy: 0.8556 - val_loss: 0.7029 - val_accuracy: 0.6923
Epoch 336/1000
270/270 [==============================] - 0s 63us/step - loss: 0.4159 - accuracy: 0.8407 - val_loss: 0.7424 - val_accuracy: 0.6923
Epoch 337/1000
270/270 [==============================] - 0s 68us/step - loss: 0.3676 - accuracy: 0.8667 - val_loss: 0.7297 - val_accuracy: 0.6923
Epoch 338/1000
270/270 [==============================] - 0s 59us/step - loss: 0.3800 - accuracy: 0.8407 - val_loss: 0.7485 - val_accuracy: 0.6923
Epoch 339/1000
270/270 [==============================] - 0s 56us/step - loss: 0.4309 - accuracy: 0.8519 - val_loss: 0.8377 - val_accuracy: 0.6838
Epoch 340/1000
270/270 [==============================] - 0s 55us/step - loss: 0.4474 - accuracy: 0.8519 - val_loss: 0

...

Epoch 724/1000
270/270 [==============================] - 0s 91us/step - loss: 0.4595 - accuracy: 0.8667 - val_loss: 1.0447 - val_accuracy: 0.6410
Epoch 725/1000
270/270 [==============================] - 0s 90us/step - loss: 0.5017 - accuracy: 0.8556 - val_loss: 0.7882 - val_accuracy: 0.7350
Epoch 726/1000
270/270 [==============================] - 0s 88us/step - loss: 0.3522 - accuracy: 0.8704 - val_loss: 0.8030 - val_accuracy: 0.7179
Epoch 727/1000
270/270 [==============================] - 0s 84us/step - loss: 0.3133 - accuracy: 0.8741 - val_loss: 0.9010 - val_accuracy: 0.7094
Epoch 728/1000
270/270 [==============================] - 0s 89us/step - loss: 0.4241 - accuracy: 0.8704 - val_loss: 0.8164 - val_accuracy: 0.6923
Epoch 729/1000
270/270 [==============================] - 0s 93us/step - loss: 0.3146 - accuracy: 0.8630 - val_loss: 0.8335 - val_accuracy: 0.7265
Epoch 730/1000
270/270 [==============================] - 0s 87us/step - loss: 0.3387 - accuracy: 0.8926 - val_loss: 0

Epoch 780/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2409 - accuracy: 0.9037 - val_loss: 0.8087 - val_accuracy: 0.7350
Epoch 781/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2626 - accuracy: 0.8889 - val_loss: 0.8568 - val_accuracy: 0.7265
Epoch 782/1000
270/270 [==============================] - 0s 86us/step - loss: 0.3542 - accuracy: 0.8852 - val_loss: 0.8742 - val_accuracy: 0.7094
Epoch 783/1000
270/270 [==============================] - 0s 85us/step - loss: 0.3103 - accuracy: 0.8815 - val_loss: 0.9815 - val_accuracy: 0.6496
Epoch 784/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3204 - accuracy: 0.8778 - val_loss: 0.8242 - val_accuracy: 0.7265
Epoch 785/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2715 - accuracy: 0.8889 - val_loss: 0.8622 - val_accuracy: 0.7265
Epoch 786/1000
270/270 [==============================] - 0s 89us/step - loss: 0.3400 - accuracy: 0.8815 - val_loss: 0

Epoch 836/1000
270/270 [==============================] - 0s 101us/step - loss: 0.4974 - accuracy: 0.8593 - val_loss: 0.8483 - val_accuracy: 0.7094
Epoch 837/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2554 - accuracy: 0.9037 - val_loss: 0.8671 - val_accuracy: 0.7265
Epoch 838/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2645 - accuracy: 0.8815 - val_loss: 0.9085 - val_accuracy: 0.6752
Epoch 839/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2399 - accuracy: 0.8852 - val_loss: 0.8725 - val_accuracy: 0.6923
Epoch 840/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2404 - accuracy: 0.8852 - val_loss: 0.8605 - val_accuracy: 0.6752
Epoch 841/1000
270/270 [==============================] - 0s 85us/step - loss: 0.2338 - accuracy: 0.8926 - val_loss: 0.8576 - val_accuracy: 0.7350
Epoch 842/1000
270/270 [==============================] - 0s 95us/step - loss: 0.2381 - accuracy: 0.9037 - val_loss: 

Epoch 892/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2333 - accuracy: 0.8852 - val_loss: 0.8727 - val_accuracy: 0.6838
Epoch 893/1000
270/270 [==============================] - 0s 90us/step - loss: 0.2265 - accuracy: 0.9111 - val_loss: 0.8548 - val_accuracy: 0.7179
Epoch 894/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2354 - accuracy: 0.9000 - val_loss: 0.8711 - val_accuracy: 0.7265
Epoch 895/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2296 - accuracy: 0.9111 - val_loss: 0.8544 - val_accuracy: 0.7179
Epoch 896/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2244 - accuracy: 0.9074 - val_loss: 0.8529 - val_accuracy: 0.7179
Epoch 897/1000
270/270 [==============================] - 0s 89us/step - loss: 0.2259 - accuracy: 0.9037 - val_loss: 0.8624 - val_accuracy: 0.7265
Epoch 898/1000
270/270 [==============================] - 0s 83us/step - loss: 0.2294 - accuracy: 0.9074 - val_loss: 0

Epoch 948/1000
270/270 [==============================] - 0s 91us/step - loss: 0.2845 - accuracy: 0.8852 - val_loss: 1.0811 - val_accuracy: 0.6838
Epoch 949/1000
270/270 [==============================] - 0s 92us/step - loss: 0.3721 - accuracy: 0.8593 - val_loss: 1.1383 - val_accuracy: 0.6410
Epoch 950/1000
270/270 [==============================] - 0s 91us/step - loss: 0.4536 - accuracy: 0.8704 - val_loss: 1.2469 - val_accuracy: 0.6752
Epoch 951/1000
270/270 [==============================] - 0s 89us/step - loss: 0.3244 - accuracy: 0.8778 - val_loss: 0.9736 - val_accuracy: 0.6667
Epoch 952/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2571 - accuracy: 0.8889 - val_loss: 1.0297 - val_accuracy: 0.7179
Epoch 953/1000
270/270 [==============================] - 0s 85us/step - loss: 0.3220 - accuracy: 0.8963 - val_loss: 1.0273 - val_accuracy: 0.7009
Epoch 954/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3958 - accuracy: 0.8741 - val_loss: 0

In [251]:
acc_test2_over4 = model2_over4.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over4*100))

117/117 [==============================] - 0s 71us/step
over-sampling test accuracy: 67.52%


In [125]:
pred8 = model2_over4.predict_classes(X_sel_test_over)
pred8

array([2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 1, 0, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 1, 1, 0, 1, 2, 0, 2, 0, 1, 2, 2,
       2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 0, 2, 0, 0, 0, 0, 2, 2,
       2, 1, 1, 2, 0, 1, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 1, 0, 2, 0,
       0, 0, 0, 0, 0, 1, 2, 2, 0, 2, 0, 2, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2,
       1, 2, 2, 1, 1, 2, 1])

In [126]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS236,1,2
1,NRS113,2,2
2,CFBRSa23,0,0
3,NRS249,2,2
4,107,1,0
...,...,...,...
112,NY439,2,2
113,NRS106,0,1
114,221,0,1
115,NRS386,2,2


In [127]:
proba8 = model2_over4.predict_proba(X_sel_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,0.000503,0.056169,0.943327
1,0.010403,0.000670,0.988927
2,0.978578,0.014319,0.007103
3,0.000550,0.000128,0.999322
4,0.726844,0.050102,0.223053
...,...,...,...
112,0.003017,0.001974,0.995009
113,0.118740,0.877459,0.003802
114,0.147418,0.833585,0.018997
115,0.000002,0.471317,0.528681


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p006pST.csv", index = False,
         header=None)

In [255]:
hist2_over4 = model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 107us/step - loss: 0.3353 - accuracy: 0.8741 - val_loss: 1.0247 - val_accuracy: 0.7179
Epoch 2/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2762 - accuracy: 0.9000 - val_loss: 1.0159 - val_accuracy: 0.7179
Epoch 3/1000
270/270 [==============================] - 0s 129us/step - loss: 0.2917 - accuracy: 0.8963 - val_loss: 1.0330 - val_accuracy: 0.6838
Epoch 4/1000
270/270 [==============================] - 0s 114us/step - loss: 0.3175 - accuracy: 0.8667 - val_loss: 1.1134 - val_accuracy: 0.6496
Epoch 5/1000
270/270 [==============================] - 0s 120us/step - loss: 0.4474 - accuracy: 0.8630 - val_loss: 1.0309 - val_accuracy: 0.6838
Epoch 6/1000
270/270 [==============================] - 0s 115us/step - loss: 0.4674 - accuracy: 0.8593 - val_loss: 1.0731 - val_accuracy: 0.6752
Epoch 7/1000
270/270 [==============================] - 0s 141us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 117us/step - loss: 0.2625 - accuracy: 0.8852 - val_loss: 1.0799 - val_accuracy: 0.7179
Epoch 113/1000
270/270 [==============================] - 0s 105us/step - loss: 0.2530 - accuracy: 0.8815 - val_loss: 1.0758 - val_accuracy: 0.7094
Epoch 114/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2476 - accuracy: 0.9000 - val_loss: 1.0641 - val_accuracy: 0.7009
Epoch 115/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2536 - accuracy: 0.9037 - val_loss: 1.0585 - val_accuracy: 0.7009
Epoch 116/1000
270/270 [==============================] - 0s 107us/step - loss: 0.2473 - accuracy: 0.9037 - val_loss: 1.0608 - val_accuracy: 0.7094
Epoch 117/1000
270/270 [==============================] - 0s 93us/step - loss: 0.2458 - accuracy: 0.9000 - val_loss: 1.0767 - val_accuracy: 0.7094
Epoch 118/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2551 - accuracy: 0.8963 - val_loss

270/270 [==============================] - 0s 111us/step - loss: 0.2486 - accuracy: 0.8926 - val_loss: 1.0872 - val_accuracy: 0.7094
Epoch 223/1000
270/270 [==============================] - 0s 117us/step - loss: 0.2490 - accuracy: 0.9000 - val_loss: 1.0752 - val_accuracy: 0.7179
Epoch 224/1000
270/270 [==============================] - 0s 96us/step - loss: 0.2461 - accuracy: 0.9074 - val_loss: 1.0839 - val_accuracy: 0.7094
Epoch 225/1000
270/270 [==============================] - 0s 92us/step - loss: 0.2406 - accuracy: 0.9037 - val_loss: 1.0914 - val_accuracy: 0.7094
Epoch 226/1000
270/270 [==============================] - 0s 97us/step - loss: 0.2858 - accuracy: 0.8741 - val_loss: 1.0894 - val_accuracy: 0.7009
Epoch 227/1000
270/270 [==============================] - 0s 87us/step - loss: 0.2910 - accuracy: 0.9037 - val_loss: 1.0748 - val_accuracy: 0.7009
Epoch 228/1000
270/270 [==============================] - 0s 88us/step - loss: 0.2854 - accuracy: 0.8963 - val_loss: 1.0987 - val_a

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.154811). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


270/270 [==============================] - 0s 172us/step - loss: 0.2459 - accuracy: 0.8926 - val_loss: 1.0971 - val_accuracy: 0.7094
Epoch 242/1000
270/270 [==============================] - 0s 319us/step - loss: 0.2859 - accuracy: 0.9000 - val_loss: 1.0939 - val_accuracy: 0.7094
Epoch 243/1000
270/270 [==============================] - 0s 165us/step - loss: 0.2696 - accuracy: 0.8963 - val_loss: 1.1012 - val_accuracy: 0.7179
Epoch 244/1000
270/270 [==============================] - 0s 185us/step - loss: 0.2473 - accuracy: 0.8963 - val_loss: 1.0994 - val_accuracy: 0.7009
Epoch 245/1000
270/270 [==============================] - 0s 156us/step - loss: 0.2412 - accuracy: 0.9000 - val_loss: 1.0960 - val_accuracy: 0.7094
Epoch 246/1000
270/270 [==============================] - 0s 211us/step - loss: 0.2378 - accuracy: 0.8926 - val_loss: 1.0959 - val_accuracy: 0.7094
Epoch 247/1000
270/270 [==============================] - 0s 198us/step - loss: 0.2399 - accuracy: 0.9037 - val_loss: 1.0825 - 

...

270/270 [==============================] - 0s 108us/step - loss: 0.2254 - accuracy: 0.9074 - val_loss: 1.2269 - val_accuracy: 0.7179
Epoch 682/1000
270/270 [==============================] - 0s 99us/step - loss: 0.2324 - accuracy: 0.9037 - val_loss: 1.2224 - val_accuracy: 0.7265
Epoch 683/1000
270/270 [==============================] - 0s 94us/step - loss: 0.2416 - accuracy: 0.8926 - val_loss: 1.2308 - val_accuracy: 0.7179
Epoch 684/1000
270/270 [==============================] - 0s 91us/step - loss: 0.3015 - accuracy: 0.8926 - val_loss: 1.2083 - val_accuracy: 0.7179
Epoch 685/1000
270/270 [==============================] - 0s 86us/step - loss: 0.2299 - accuracy: 0.9074 - val_loss: 1.2122 - val_accuracy: 0.7179
Epoch 686/1000
270/270 [==============================] - 0s 235us/step - loss: 0.2233 - accuracy: 0.9074 - val_loss: 1.2167 - val_accuracy: 0.7179
Epoch 687/1000
270/270 [==============================] - 0s 263us/step - loss: 0.2273 - accuracy: 0.9074 - val_loss: 1.2167 - val_

270/270 [==============================] - 0s 97us/step - loss: 0.2215 - accuracy: 0.9074 - val_loss: 1.2599 - val_accuracy: 0.7265
Epoch 792/1000
270/270 [==============================] - 0s 134us/step - loss: 0.2483 - accuracy: 0.8852 - val_loss: 1.2689 - val_accuracy: 0.6752
Epoch 793/1000
270/270 [==============================] - 0s 120us/step - loss: 0.2386 - accuracy: 0.9074 - val_loss: 1.2694 - val_accuracy: 0.7265
Epoch 794/1000
270/270 [==============================] - 0s 199us/step - loss: 0.2533 - accuracy: 0.8926 - val_loss: 1.2430 - val_accuracy: 0.7179
Epoch 795/1000
270/270 [==============================] - 0s 111us/step - loss: 0.2691 - accuracy: 0.8926 - val_loss: 1.2829 - val_accuracy: 0.7265
Epoch 796/1000
270/270 [==============================] - 0s 124us/step - loss: 0.3167 - accuracy: 0.8704 - val_loss: 1.2838 - val_accuracy: 0.7179
Epoch 797/1000
270/270 [==============================] - 0s 95us/step - loss: 0.3259 - accuracy: 0.8926 - val_loss: 1.2750 - va

270/270 [==============================] - 0s 167us/step - loss: 0.3062 - accuracy: 0.9000 - val_loss: 1.3068 - val_accuracy: 0.7265
Epoch 902/1000
270/270 [==============================] - 0s 158us/step - loss: 0.4340 - accuracy: 0.8852 - val_loss: 1.2706 - val_accuracy: 0.7179
Epoch 903/1000
270/270 [==============================] - 0s 208us/step - loss: 0.2473 - accuracy: 0.8926 - val_loss: 1.2758 - val_accuracy: 0.7265
Epoch 904/1000
270/270 [==============================] - 0s 137us/step - loss: 0.2436 - accuracy: 0.8889 - val_loss: 1.3217 - val_accuracy: 0.6838
Epoch 905/1000
270/270 [==============================] - 0s 138us/step - loss: 0.2529 - accuracy: 0.9037 - val_loss: 1.2624 - val_accuracy: 0.7094
Epoch 906/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2520 - accuracy: 0.8815 - val_loss: 1.3040 - val_accuracy: 0.6838
Epoch 907/1000
270/270 [==============================] - 0s 178us/step - loss: 0.2768 - accuracy: 0.8889 - val_loss: 1.2797 - 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.254633). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.128448). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 976/1000
270/270 [==============================] - 0s 318us/step - loss: 0.2306 - accuracy: 0.8926 - val_loss: 1.3342 - val_accuracy: 0.7179
Epoch 977/1000
270/270 [==============================] - 0s 126us/step - loss: 0.2292 - accuracy: 0.9074 - val_loss: 1.3455 - val_accuracy: 0.7265
Epoch 978/1000
270/270 [==============================] - 0s 116us/step - loss: 0.2215 - accuracy: 0.9111 - val_loss: 1.3300 - val_accuracy: 0.7094
Epoch 979/1000
270/270 [==============================] - 0s 192us/step - loss: 0.2239 - accuracy: 0.9037 - val_loss: 1.3167 - val_accuracy: 0.7009
Epoch 980/1000
270/270 [==============================] - 0s 703us/step - loss: 0.2264 - accuracy: 0.9074 - val_loss: 1.3357 - val_accuracy: 0.7179
Epoch 981/1000
270/270 [==============================] - 0s 297us/step - loss: 0.2283 - accuracy: 0.9037 - val_loss: 1.3335 - val_accuracy: 0.7094
Epoch 982/1000
270/270 [==============================] - 0s 133us/step - loss: 0.2234 - accuracy: 0.9037 - val_

In [256]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over4.history['accuracy'])*100))

over-sampling train accuracy: 89.19%


In [63]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [64]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS236,1,2,1.321970e-02,2.446264e-01,7.421539e-01
1,p0006kpresabs_qual,NRS113,2,2,3.478230e-02,2.806685e-01,6.845492e-01
2,p0006kpresabs_qual,CFBRSa23,0,0,4.090251e-01,3.405008e-01,2.504741e-01
3,p0006kpresabs_qual,NRS249,2,1,1.987907e-01,5.331044e-01,2.681049e-01
4,p0006kpresabs_qual,107,1,0,4.090251e-01,3.405008e-01,2.504741e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CFBRSa30,0,0,7.207667e-01,2.792331e-01,2.571588e-07
985,p0017Skpresabs_qual,NRS383,1,0,6.129044e-01,3.870795e-01,1.601290e-05
986,p0017Skpresabs_qual,NRS110,2,2,3.260306e-07,7.910664e-07,9.999989e-01
987,p0017Skpresabs_qual,NRS209,2,2,3.604249e-12,2.698129e-07,9.999998e-01


In [65]:
y_prob8 = df_proba8[df_proba8['phage']=='p0006presabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[5.03265000e-04, 5.61693650e-02, 9.43327300e-01],
       [1.04032390e-02, 6.70049500e-04, 9.88926650e-01],
       [9.78578100e-01, 1.43186760e-02, 7.10328440e-03],
       [5.50247550e-04, 1.27662090e-04, 9.99322060e-01],
       [7.26844500e-01, 5.01021560e-02, 2.23053290e-01],
       [1.94693530e-05, 2.33604520e-02, 9.76620100e-01],
       [7.26844500e-01, 5.01021560e-02, 2.23053290e-01],
       [8.22069760e-01, 1.74542350e-01, 3.38793130e-03],
       [9.99806000e-01, 1.93385190e-04, 5.42968400e-07],
       [9.99809560e-01, 1.88622940e-04, 1.84084700e-06],
       [7.26844500e-01, 5.01021560e-02, 2.23053290e-01],
       [1.34047710e-03, 9.01287800e-01, 9.73717800e-02],
       [1.59369290e-01, 4.17955370e-01, 4.22675300e-01],
       [1.25269030e-02, 6.09113630e-01, 3.78359530e-01],
       [9.99922900e-01, 7.69331160e-05, 2.50158820e-07],
       [8.94587160e-08, 1.79730520e-07, 9.99999760e-01],
       [7.67720500e-04, 2.41396450e-02, 9.75092650e-01],
       [1.59369290e-01, 4.17955

In [66]:
ovo8 = rocauc_ovo(y_sel_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.8666447512601358

In [67]:
ovr8 = rocauc_ovr(y_sel_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.8666447512601358

In [68]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.8503725619110235

In [69]:
np.std(ovos2)

0.03959057010134543

In [70]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.8503725619110235

In [71]:
np.std(ovrs2)

0.03959057010134543

In [257]:
accs_l_over = [acc_test2_over, acc_test2_over2, acc_test2_over3, acc_test2_over4]

In [258]:
mean_l_over = np.mean(accs_l_over)
print('over-sampling test accuracy mean after lasso: %.2f%%' % (mean_l_over*100))

over-sampling test accuracy mean after lasso: 71.37%


In [259]:
std_l_over = np.std(accs_l_over)
print('over-sampling test accuracy standard deviation after lasso:', std_l_over)

over-sampling test accuracy standard deviation after lasso: 0.06438683023626


In [260]:
accs_train_l_over = [np.mean(hist2_over.history['accuracy']), np.mean(hist2_over2.history['accuracy']), np.mean(hist2_over3.history['accuracy']),
             np.mean(hist2_over4.history['accuracy'])]

In [261]:
mean_train_l_over = np.mean(accs_train_l_over)
print('over-sampling train accuracy mean after lasso: %.2f%%' % (mean_train_l_over*100))

over-sampling train accuracy mean after lasso: 89.00%


In [262]:
std_train_l_over = np.std(accs_train_l_over)
print('over-sampling train accuracy standard deviation after lasso:', std_train_l_over)

over-sampling train accuracy standard deviation after lasso: 0.011120621
